### Web Scraper

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import re

In [ ]:
BASE_URL = "https://manshurat.org/node/"

In [ ]:
def parse_normal_page(soup):
    articles = []
    # Locate the div containing the articles
    content_div = soup.select_one(".field.field-name-body .field-items")
    if not content_div:
        print("No content div found. Check the structure of the page.")
        return articles

    # Find all <p> tags inside the content div
    paragraphs = content_div.find_all("p")
    print(f"Found {len(paragraphs)} <p> tags.")  # Debugging: Log the number of <p> tags found

    i = 0
    while i < len(paragraphs):
        text = paragraphs[i].get_text(strip=True)  # Extract text
        print(f"Paragraph {i+1}: {text}")  # Debugging: Log the text of each paragraph

        # Check if the paragraph starts with "المادة"
        if text.startswith("المادة"):
            # Extract the article number
            split_text = text.split("المادة", 1)
            if len(split_text) > 1:
                article_number = split_text[1].strip().split(" ")[0]  # Get only the number
                article = f"المادة {article_number}"

                # Check if the rest of the law is in the same paragraph
                law = split_text[1].strip()[len(article_number):].strip()

                # If no law in the same paragraph, check the next paragraph
                if not law and i + 1 < len(paragraphs):
                    next_text = paragraphs[i + 1].get_text(strip=True)
                    if not next_text.startswith("المادة"):  # If the next paragraph isn't a new article
                        law = next_text
                        i += 1  # Skip the next paragraph since it's part of the current law

                articles.append({"Article": article, "Law": law})
            else:
                print(f"Could not split article and law: {text}")  # Debugging

        # Handle cases where <br> tags are used within a single paragraph
        elif "<br" in str(paragraphs[i]):  # Check raw HTML for <br>
            parts = str(paragraphs[i]).split("<br/>")
            for part in parts:
                if "المادة" in part:
                    text = BeautifulSoup(part, "html.parser").get_text(strip=True)
                    if text.startswith("المادة"):
                        split_text = text.split("المادة", 1)
                        if len(split_text) > 1:
                            article_number = split_text[1].strip().split(" ")[0]
                            article = f"المادة {article_number}"
                            law = split_text[1].strip()[len(article_number):].strip()
                            articles.append({"Article": article, "Law": law})

        i += 1  # Move to the next paragraph
    return articles

def save_csv(filename, rows, fieldnames):
    # Overwrite the file if it exists
    if os.path.exists(filename):
        os.remove(filename)
        print(f"Overwriting existing file: {filename}")
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

def main():
    for page_id in range(14677, 14720):  # Use a specific range for testing
        print(f"Processing page ID: {page_id}")
        soup = fetch_page_content(page_id)
        if not soup:
            print(f"Page {page_id} not found. Skipping...")
            continue

        # Extract page title
        page_title_element = soup.find("h2")
        if not page_title_element:
            print(f"Page {page_id} has no title. Skipping...")
            continue
        page_title = page_title_element.text.strip()

        # Skip pages with "الصفحة غير موجودة"
        if "الصفحة غير موجودة" in page_title:
            print(f"Page {page_id} not found. Skipping...")
            continue

        # Ignore pages with PDF links
        if soup.find("a", string="Download"):
            print(f"Page {page_id} contains a PDF. Skipping...")
            continue

        # Handle normal pages
        articles = parse_normal_page(soup)
        if articles:
            # Sanitize the title and create a unique filename
            sanitized_title = f"ID_{page_id}_{page_title}".replace("/", "-").replace("\\", "-").replace(":", "-")
            csv_filename = f"{sanitized_title}.csv"
            
            # Save the extracted data into a CSV file
            save_csv(csv_filename, articles, ["Article", "Law"])
            print(f"Saved normal page with ID {page_id}: {csv_filename}")
        else:
            print(f"No articles found on page {page_id}. Skipping...")

if __name__ == "__main__":
    main()


Processing page ID: 14677
Found 557 <p> tags.
Paragraph 1: **آخر تعديل: 15 أغسطس 2021 بالقانون 141 لسنة 2021 **
Paragraph 2: القانون رقم 58 لسنة 1937إصدار قانون العقوباتنحن فاروق الأول ملك مصرقرر مجلس الشيوخ ومجلس النواب القانون الآتي نصه وقد صدقنا عليه وأصدرناه:
Paragraph 3: المادة 1يلغى قانون العقوبات الجاري العمل به أمام المحاكم الأهلية وقانون العقوبات الذي تطبقه المحاكم المختلطة ويستعاض عنهما بقانون العقوبات المرافق لهذا القانون.
Paragraph 4: المادة 2على وزير الحقانية تنفيذ هذا القانون ويعمل به من 15 أكتوبر سنة 1937.نأمر بأن يبصم هذا القانون بخاتم الدولة وأن ينشر في الجريدة الرسمية وينفذ كقانون من قوانين الدولة.
Paragraph 5: صدر بسراي عابدين في 23 جمادي الأول ( 21 يوليو سنة 1937 )فاروقبأمر حضرة صاحب الجلالةرئيس الوزراءمصطفي النحاس
Paragraph 6: قانون العقوباتالكتاب الأول أحكام ابتدائيةالباب الأول – قواعد عمومية
Paragraph 7: المادة 1تسري أحكام هذا القانون على كل من يرتكب في القطر المصري جريمة من الجرائم المنصوص عليها فيه.
Paragraph 8: المادة 2تسري أحكام هذا القانون أيضاً على الأشخاص 

In [27]:
URL = "https://manshurat.org/node/63711"

def parse_normal_page(soup):
    """Parse the content of a normal page to extract articles and laws."""
    articles = []
    # Locate the div containing the articles
    content_div = soup.select_one(".field.field-name-body .field-items")
    if not content_div:
        print("No content div found. Check the structure of the page.")
        return articles

    # Find all <p> tags inside the content div
    paragraphs = content_div.find_all("p")
    print(f"Found {len(paragraphs)} <p> tags.")  # Debugging: Log the number of <p> tags found

    i = 0
    while i < len(paragraphs):
        text = paragraphs[i].get_text(strip=True)  # Extract text
        print(f"Paragraph {i+1}: {text}")  # Debugging: Log the text of each paragraph

        # Check if the paragraph starts with "المادة"
        if text.startswith("المادة"):
            # Extract the article number
            split_text = text.split("المادة", 1)
            if len(split_text) > 1:
                article_number = split_text[1].strip().split(" ")[0]  # Get only the number
                article = f"المادة {article_number}"

                # Check if the rest of the law is in the same paragraph
                law = split_text[1].strip()[len(article_number):].strip()

                # If no law in the same paragraph, check the next paragraph
                if not law and i + 1 < len(paragraphs):
                    next_text = paragraphs[i + 1].get_text(strip=True)
                    if not next_text.startswith("المادة"):  # If the next paragraph isn't a new article
                        law = next_text
                        i += 1  # Skip the next paragraph since it's part of the current law

                articles.append({"Article": article, "Law": law})
            else:
                print(f"Could not split article and law: {text}")  # Debugging

        # Handle cases where <br> tags are used within a single paragraph
        elif "<br" in str(paragraphs[i]):  # Check raw HTML for <br>
            parts = str(paragraphs[i]).split("<br/>")
            for part in parts:
                if "المادة" in part:
                    text = BeautifulSoup(part, "html.parser").get_text(strip=True)
                    if text.startswith("المادة"):
                        split_text = text.split("المادة", 1)
                        if len(split_text) > 1:
                            article_number = split_text[1].strip().split(" ")[0]
                            article = f"المادة {article_number}"
                            law = split_text[1].strip()[len(article_number):].strip()
                            articles.append({"Article": article, "Law": law})

        i += 1  # Move to the next paragraph
    return articles

def save_csv(filename, rows, fieldnames):
    """Save the extracted data into a CSV file."""
    # Overwrite the file if it exists
    if os.path.exists(filename):
        os.remove(filename)
        print(f"Overwriting existing file: {filename}")
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

def main():
    print(f"Processing URL: {URL}")
    soup = fetch_page_content(URL)
    if not soup:
        print(f"Failed to fetch the page. Please check the URL and try again.")
        return

    # Extract the page title
    page_title_element = soup.find("h2")
    if not page_title_element:
        print(f"The page has no title. Skipping...")
        return

    page_title = page_title_element.text.strip()

    # Skip pages with "الصفحة غير موجودة"
    if "الصفحة غير موجودة" in page_title:
        print(f"The page does not exist. Skipping...")
        return

    # Process normal pages
    articles = parse_normal_page(soup)
    if articles:
        # Sanitize the title and create a unique filename
        sanitized_title = page_title.replace("/", "-").replace("\\", "-").replace(":", "-")
        csv_filename = f"{sanitized_title}.csv"

        # Save the extracted data into a CSV file
        save_csv(csv_filename, articles, ["Article", "Law"])
        print(f"Saved articles for the page in {csv_filename}")
    else:
        print(f"No articles found on the page.")

if __name__ == "__main__":
    main()


Processing URL: https://manshurat.org/node/63711
Found 88 <p> tags.
Paragraph 1: قانون الطوارئ رقم 162 لسنة 1958
Paragraph 2: **النص الساري للقانون وفقاً لأحدث تعديلاته**
Paragraph 3: ** مضاف إليه التعديل الصادر بالقانون 22 لسنة 2020 بتاريخ 6 مايو 2020 **
Paragraph 4: بعد الاطلاع على الدستور المؤقت؛وعلى المرسوم التشريعي رقم 150 الصادر في الإقليم السوري بتاريخ 22/6/1949 المتضمن تنظيم الإدارة العرفية؛وعلى القانون رقم 533 لسنة 1954 في شأن الأحكام العرفية الصادر في الإقليم المصري والقوانين المعدلة له؛قرر القانون الآتي:
Paragraph 5: مادة 1:
Paragraph 6: يعمل بالقانون المرافق في شأن حالة الطوارئ.
Paragraph 7: مادة 2:
Paragraph 8: يلغى المرسوم التشريعي رقم 150 الصادر في 22/6/1949 والقانون رقم 533 لسنة 1954 المشار إليهما وكذلك كل نص يخالف أحكام هذا القانون.
Paragraph 9: مادة 3:
Paragraph 10: ينشر هذا القانون في الجريدة الرسمية, ويعمل به في إقليمي الجمهورية من تاريخ نشره.
Paragraph 11: رئيس الجمهورية العربية المتحدة
Paragraph 12: جمال عبد الناصر
Paragraph 13: مادة 1:
Paragraph 14: يجوز إعلان حا

In [ ]:
URL = "https://manshurat.org/node/71081"

def parse_normal_page(soup):
    
    articles = []
    
    # Locate the container for the main content
    content_div = soup.select_one(".field.field-name-body .field-items")
    if not content_div:
        print("No content div found. Check the page structure.")
        return articles

    paragraphs = content_div.find_all("p")
    print(f"Found {len(paragraphs)} <p> tags.")

    current_article_title = None
    current_law_lines = []  # Accumulates law text for the current article

    for p_tag in paragraphs:
        # Split the paragraph into separate lines (if any) using newline as the separator.
        lines = p_tag.get_text(separator="\n", strip=True).split("\n")
        header_found_in_this_paragraph = False
        
        for idx, line in enumerate(lines):
            # Count the number of words in the line
            words = line.split()
            if "المادة" in line and len(words) <= 5:
                # A valid header line is found.
                # Finalize the previous article (if any)
                if current_article_title or current_law_lines:
                    articles.append({
                        "Article": current_article_title if current_article_title else "",
                        "Law": "\n".join(current_law_lines).strip()
                    })
                # The whole line becomes the new article header
                current_article_title = line.strip()
                header_found_in_this_paragraph = True
                # If there are more lines in this paragraph, treat them as part of the law text.
                if idx < len(lines) - 1:
                    remaining_lines = [l.strip() for l in lines[idx+1:] if l.strip()]
                    current_law_lines = remaining_lines
                else:
                    current_law_lines = []
                # Only process the first valid header per paragraph
                break
        
        # If no valid header was found in the current paragraph,
        # consider the entire paragraph as law text
        if not header_found_in_this_paragraph:
            # Append the full paragraph text to the current law lines.
            # (If there is no current article header yet, the text will accumulate until one is found)
            current_law_lines.append(" ".join(lines))
    
    # Finalize the last article (if any)
    if current_article_title or current_law_lines:
        articles.append({
            "Article": current_article_title if current_article_title else "",
            "Law": "\n".join(current_law_lines).strip()
        })
    
    return articles



def save_csv(filename, rows, fieldnames):
    """Save the extracted data into a CSV file."""
    # Overwrite the file if it exists
    if os.path.exists(filename):
        os.remove(filename)
        print(f"Overwriting existing file: {filename}")
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

def main():
    print(f"Processing URL: {URL}")
    soup = fetch_page_content(URL)
    if not soup:
        print(f"Failed to fetch the page. Please check the URL and try again.")
        return

    # Extract the page title
    page_title_element = soup.find("h2")
    if not page_title_element:
        print(f"The page has no title. Skipping...")
        return

    page_title = page_title_element.text.strip()

    # Skip pages with "الصفحة غير موجودة"
    if "الصفحة غير موجودة" in page_title:
        print(f"The page does not exist. Skipping...")
        return

    # Process normal pages
    articles = parse_normal_page(soup)
    if articles:
        # Sanitize the title and create a unique filename
        sanitized_title = page_title.replace("/", "-").replace("\\", "-").replace(":", "-")
        csv_filename = f"new.csv"

        # Save the extracted data into a CSV file
        save_csv(csv_filename, articles, ["Article", "Law"])
        print(f"Saved articles for the page in {csv_filename}")
    else:
        print(f"No articles found on the page.")

if __name__ == "__main__":
    main()


Processing URL: https://manshurat.org/node/71081
Found 1059 <p> tags.
Overwriting existing file: new.csv
Saved articles for the page in new.csv


In [ ]:
html = r'''

<div class="ds-main col-md-9 ">
  <div class="field field-name-body field-type-text-with-summary field-label-hidden"><div class="field-items"><div class="field-item even" property="content:encoded"><p>قانون رقم 81 لسنة 2016 بإصدار قانون الخدمة المدنية باسم الشعب رئيس الجمهورية قرر مجلس النواب القانون الآتي نصه, وقد أصدرناه: المادة 1 يُعمل بأحكام القانون المرافق في شأن الخدمة المدنية, وتسري أحكامه على الوظائف في الوزارات ومصالحها والأجهزة الحكومية ووحدات الإدارة المحلية, والهيئات العامة وذلك ما لم تنص قوانين أو قرارات إنشائها على ما يخالف ذلك. المادة 2 يُلغى قانون نظام العاملين المدنيين بالدولة الصادر بالقانون رقم 47 لسنة 1978, كما يُلغى كل حكم يخالف أحكام القانون المرافق. المادة 3 يُصدر رئيس مجلس الوزراء بعد أخذ رأي مجلس الخدمة المدنية اللائحة التنفيذية للقانون المرافق خلال ثلاثة أشهر من تاريخ العمل به وإلى أن تصدر هذه اللائحة يستمر العمل باللوائح والقرارات القائمة حاليًا فيما لا يتعارض وأحكام القانون المرافق. المادة 4 يُنقل الموظفون المعينون الموجودون بالخدمة قبل العمل بأحكام هذا القانون إلى الوظائف المعادلة لوظائفهم الحالية على النحو الموضح بالجداول أرقام (1, 2, 3) الملحقة بالقانون المرافق بما فيها المستوى الوظيفي الأول (أ), ويكون ترتيب الأقدمية بين المنقولين لوظيفة واحدة بحسب أوضاعهم السابقة. ويحتفظ كل منهم بالأجر المقرر له قانونًا والذي كان يتقاضاه إذا زاد على الأجر الوظيفي المقرر لمستوى وظيفته في الجداول الملحقة بالقانون المرافق, أما إذا قل الأجر المحتفظ به عن الأجر الوظيفي المقرر لمستوى وظيفته يصرف له الأجر الوظيفي المقرر في الجداول المشار إليها. ذلك كله مع عدم الإخلال بالقوانين والقرارات المنظمة للحدين الأدنى والأقصى للدخول. المادة 5 يُنشر هذا القانون في الجريدة الرسمية, ويُعمل به من اليوم التالي لتاريخ نشره. يُبصم هذا القانون بخاتم الدولة, ويُنفذ كقانون من قوانينها. قانون الخدمة المدنية الباب الأول الأحكام العامة المادة 1 الوظائف المدنية حق للمواطنين على أساس الكفاءة والجدارة, وهي تكليف للقائمين بها لخدمة الشعب, وتكفل الدولة حقوقهم وحمايتهم, وقيامهم بأداء واجباتهم في رعاية مصالح الشعب. ويُحظر التمييز بين الموظفين في تطبيق أحكام هذا القانون بسبب الدين أو الجنس أو لأي سبب آخر. المادة 2 يُقصد في تطبيق أحكام هذا القانون بالكلمات والعبارات التالية المعنى المبين قرين كل منها: 1- السلطة المختصة: الوزير أو المحافظ أو رئيس مجلس إدارة الهيئة بحسب الأحوال. 2- الوحدة: الوزارة أو المصلحة أو الجهاز الحكومي أو المحافظة أو الهيئة العامة. 3- الوظائف القيادية: وظائف المستويات الثلاثة التالية للسلطة المختصة والتي يرأس شاغلوها وحدات تقسيمات تنظيمية بالوحدة من مستوى إدارة عامة أو إدارة مركزية أو قطاعات, وما يعادلها من تقسيمات. 4- وظائف الإدارة الإشرافية: وظائف المستوى التالي للوظائف القيادية, والتي يرأس شاغلوها إدارات بالوحدة. 5- الموظف: كل من يشغل إحدى الوظائف الواردة بموازنة الوحدة. 6- الأجر الوظيفي: الأجر المنصوص عليه في الجداول الملحقة بهذا القانون مضموًما إليه جميع العلاوات المقررة بمقتضى هذا القانون. 7- الأجر المكمل: كل ما يحصل عليه الموظف نظير عمله بخلاف الأجر الوظيفي. 8- كامل الأجر: كل ما يحصل عليه الموظف نظير عمله من أجر وظيفي وأجر مكمل. 9- السنة: السنة المالية للدولة. 10- الوزير المختص: الوزير المعني بالخدمة المدنية. 11- الجهاز: الجهاز المركزي للتنظيم والإدارة. مجلس الخدمة المدنية المادة 3 يُنشأ مجلس للخدمة المدنية بغرض تقديم المقترحات الخاصة بتطوير الخدمة المدنية وتحسين الخدمات العامة في البلاد, ويقوم على وجه الخصوص بالآتي: (أ) إبداء المشورة فيما يطرح عليه من قضايا الخدمة المدنية, سواء من رئيس مجلس الوزراء أو الوزير المختص أو رئيس الجهاز. (ب) إبداء الرأي في مشروعات القوانين واللوائح المتعلقة بالخدمة المدنية. (جـ) إبداء الرأي في طريقة ومعايير تقييم الجهات الحكومية وموظفي الخدمة المدنية. (د) إبداء الرأي في البرامج التدريبية المقدمة لموظفي الخدمة المدنية. (هـ) إبداء الرأي في القضايا المتعلقة بالأخلاقيات المهنية لموظفي الخدمة المدنية. (و) تقديم المقترحات فيما يتعلق بالموازنة المخصصة للخدمة المدنية. (ز) تقديم مقترحات تحسين أداء الخدمة المدنية. ويُشكل مجلس الخدمة المدنية برئاسة رئيس الجهاز وعضوية كل من: 1- رئيس الجمعية العمومية لقسمي الفتوى والتشريع بمجلس الدولة. 2- رئيس قطاع الخدمة المدنية بالجهاز. 3- رئيس قطاع الموازنة العامة للدولة بوزارة المالية. 4- عضو من المنظمات النقابية المنتخبة يختاره الاتحاد العام لنقابات عمال مصر. 5- أربعة خبراء في الإدارة والموارد البشرية والقانون, يختارهم الوزير المختص لمدة ثلاث سنوات قابلة للتجديد ولمرة واحدة. ويكون للمجلس أمانة فنية يصدر بتشكيلها قرار من رئيس المجلس. ويضع المجلس لائحة داخلية تتضمن القواعد والإجراءات المتعلقة بسير العمل به وأمانته الفنية. وتعتمد توصيات المجلس من الوزير المختص. لجنة الموارد البشرية المادة 4 تُشكل في كل وحدة, بقرار من السلطة المختصة, لجنة أو أكثر للموارد البشرية, برئاسة أحد موظفي الوحدة من شاغلي الوظائف القيادية وعضوية أربعة أعضاء, يكون من بينهم أحد القانونيين وأحد المتخصصين في الموارد البشرية من داخل الوحدة أو خارجها, وأحد أعضاء اللجنة النقابية إن وجدت يختاره مجلس إدارة اللجنة النقابية. وتختص اللجنة بالنظر في التعيين في الوظائف من المستوى الأول (ب) فما دونها, ومنح العلاوات لشاغليها ونقلهم خارج الوحدة واعتماد تقارير تقويم أدائهم, واقتراح البرامج والدورات التدريبية اللازمة لتنمية الموارد البشرية, وتغيير مفاهيم الوظيفة وثقافتها وتطوير أساليب العمل ورفع معدلات الأداء, وغير ذلك مما يُحال إليها من السلطة المختصة. وتُرسل اللجنة اقتراحاتها إلى السلطة المختصة خلال أسبوع لاعتمادها, فإذا لم تعتمدها ولم تُبد اعتراضًا عليها خلال ثلاثين يومًا من تاريخ وصولها اعتبرت نافذة. أما إذا اعترضت على اقتراحات اللجنة كلها أو بعضها, فيتعين أن تُبدي الأسباب المبررة لذلك كتابة وتُعيد ما اعترضت عليه للجنة للنظر فيه على ضوء هذه الأسباب, وتُحدد لها أجلاً للبت فيه, فإذا انقضى هذا الأجل دون أن تبدي اللجنة رأيها اعتبر رأي السلطة المختصة نافذًا, أما إذا تمسكت اللجنة برأيها خلال الأجل المحدد, ترسل اقتراحاتها إلى السلطة المختصة لاتخاذ ما تراه بشأنها ويعتبر قرارها في هذه الحالة نهائيًا. وتحدد اللائحة التنفيذية كيفية اختيار أعضاء اللجنة ونظام العمل بها. المادة 5 تُعلن القرارات التي تصدر في شأن الخدمة المدنية في نشرة رسمية تصدرها الوحدة ورقيًا أو إلكترونيًا. وتحدد اللائحة التنفيذية كيفية وإجراءات النشر أو الإتاحة على نحو يكفل علم ذوي الشأن بها. المادة 6 يختص مجلس الدولة, دون غيره, بإبداء الرأي مسببًا في المسائل المتعلقة بتطبيق أحكام هذا القانون ولائحته التنفيذية, بناءً على طلب السلطة المختصة تنمية ثقافة الخدمة المدنية والموارد البشرية المادة 7 تعمل الوحدة على تدريب وتأهيل وإعداد الموظفين للقيام بواجباتها ومسئولياتها على نحو يكفل تنمية ثقافة الخدمة المدنية ودورها في المجتمع وتحقيق أهدافها. ولكل وحدة إنشاء مركز لتنمية الموارد البشرية, بعد موافقة الجهاز لتدريب وتأهيل وإعداد الموظفين بها وبالمصالح أو الوحدات أو الفروع التابعة لها, ويجوز إسناد عمليات التدريب والتأهيل والإعداد إلى مراكز وهيئات التدريب التي يصدر باعتمادها قرار من رئيس الجهاز. وتحدد اللائحة التنفيذية إجراءات وضوابط إنشاء مراكز تنمية الموارد البشرية ونظام التدريب والتأهيل والإعداد وضوابط الالتحاق بها والشهادات التي تمنحها. المادة 8 يجوز للوحدة أن تقوم بتدريب الشباب على الأنشطة والأعمال التخصصية بها بناءً على طلبهم دون التزامها بتعيينهم, وذلك على النحو الذي تنظمه اللائحة التنفيذية. الباب الثاني الوظائف والعلاقة الوظيفية الوظائف المادة 9 تضع كل وحدة هيكلاً تنظيميًا لها, يعتمد من السلطة المختصة, بعد أخذ رأي الجهاز, يتضمن تقسيمها إلى تقسيمات فرعية تتناسب مع أنشطتها وحجم ومجالات العمل بها. وتضع كل وحدة جدولاً للوظائف مرفقًا به بطاقات وصف كل وظيفة, تتضمن تحديد مستواها الوظيفي وطريقة شغلها والمجموعة الوظيفية التي تنتمي إليها والشروط اللازم توافرها فيمن يشغلها, والواجبات والمسئوليات والمهام المنوطة بها, ومؤشرات قياس أدائها. ويختص رئيس الجهاز باعتماد جدول وظائف كل وحدة وحجم الموارد البشرية اللازمة لها في ضوء احتياجاتها الفعلية. المادة 10 تقسم الوظائف الخاضعة لأحكام هذا القانون إلى المجموعات الوظيفية الرئيسية الآتية: 1- مجموعة الوظائف التخصصية. 2- مجموعة الوظائف الفنية. 3- مجموعة الوظائف الكتابية. 4- مجموعة الوظائف الحرفية والخدمة المعاونة. وتعتبر كل مجموعة وظيفية وحدة متميزة في مجال التعيين والترقية والنقل والندب والإعارة. وتتكون كل مجموعة وظيفية من مجموعات نوعية, وتنظم اللائحة التنفيذية معايير إنشاء هذه المجموعات النوعية والنقل بين المجموعات المتماثلة. المادة 11 يكون شغل الوظائف عن طريق التعيين أو الترقية أو النقل أو الندب أو الإعارة بمراعاة استيفاء شروط شغلها, وذلك بحسب الأحوال المبينة بهذا القانون. التعيين في الوظائف المادة 12 يكون التعيين بموجب قرار يصدر من رئيس الجمهورية أو من يفوضه, على أساس الكفاءة والجدارة, دون محاباة أو وساطة من خلال إعلان مركزي على موقع بوابة الحكومة المصرية متضمنًا البيانات المتعلقة بالوظيفة وشروط شغلها على نحو يكفل تكافؤ الفرص والمساواة بين المواطنين. وفي جميع الأحوال يشترط لشغل الوظائف أن تكون شاغرة وممولة. ويكون التعيين في تلك الوظائف بامتحان ينفذه الجهاز من خلال لجنة للاختيار, ويشرف عليه الوزير المختص, على أن يكون التعيين بحسب الأسبقية الواردة في الترتيب النهائي لنتيجة الامتحان, وعند التساوي يقدم الأعلى في مرتبة الحصول على المؤهل المطلوب لشغل الوظيفة, فالدرجة الأعلى في ذات المرتبة, فالأعلى مؤهلاً, فالأقدم في التخرج, فالأكبر سنًا. وتحدد اللائحة التنفيذية قواعد الإعلان عن الوظائف الشاغرة وكيفيته, وتشكيل لجنة الاختيار وإجراءات انعقاد الامتحان وكيفيته وقواعد المفاضلة, على أن يكون الإعلان خلال شهري يناير ويونيو من كل سنة عند الحاجة, وألا تقل مدة الإعلان والتقديم عن شهر, وتعلن النتيجة على الموقع الإلكتروني المشار إليه بالفقرة الأولى من هذه المادة. المادة 13 تلتزم كل وحدة بتخصيص نسبة خمسة في المائة من مجموع الوظائف بها للأشخاص ذوي الإعاقة. وتحدد بقرار من رئيس مجلس الوزراء الوظائف التي تحجز للمصابين في العمليات الحربية ومصابي الثورة والمحاربين القدماء ومصابي العمليات الأمنية متى سمحت حالتهم بالقيام بأعمالها, وذلك وفقًا للقواعد التي يحددها هذا القرار, على أن تلتزم الوحدة بتعيين هذه النسبة وفقًا لاحتياجاتها. كما يجوز أن يعين في هذه الوظائف أزواج الفئات المنصوص عليها في الفقرة السابقة أو أحد أولادهم أو أحد والديهم أو أحد إخوتهم, القائمين بإعالتهم, وذلك في حالة عجزهم عجزا تامًا أو وفاتهم, إذا توافرت فيهم شروط شغل هذه الوظائف, وكذلك الأمر بالنسبة لأسر الشهداء والمفقودين في العمليات الحربية وأسر شهداء العمليات الأمنية. المادة 14 يُشترط فيمن يعين في إحدى الوظائف ما يأتي: 1- أن يكون متمتعًا بالجنسية المصرية أو جنسية إحدى الدول العربية التي تعامل المصريين بالمثل في تولي الوظائف المدنية. 2- أن يكون محمود السيرة, حسن السمعة. 3- ألا يكون قد سبق الحكم عليه بعقوبة جناية أو بعقوبة مقيدة للحرية في جريمة مخلة بالشرف أو الأمانة ما لم يكن قد رُد إليه اعتباره. 4- ألا يكون قد سبق فصله من الخدمة بحكم أو قرار تأديبي نهائي, ما لم تمض على صدوره أربع سنوات على الأقل. 5- أن تثبت لياقته الصحية لشغل الوظيفة بشهادة تصدر من المجلس الطبي المختص. 6- أن يكون مستوفيًا لاشتراطات شغل الوظيفة. 7- أن يجتاز الامتحان المقرر لشغل الوظيفة. 8- ألا تقل سنه عن ثمانية عشر عامًا ميلاديًا. المادة 15 يُوضع المعين لأول مرة تحت الاختبار لمدة ستة أشهر من تاريخ تسلمه العمل تتقرر خلالها مدى صلاحيته للعمل, فإذا ثبت عدم صلاحيته أنهيت خدمته, وتحدد اللائحة التنفيذية أحوال وإجراءات عدم الصلاحية. ولا يجوز نقل أو ندب أو إعارة المعين خلال فترة الاختبار. ولا تسري أحكام هذه المادة على شاغلي الوظائف القيادية والإدارة الإشرافية. المادة 16 يجوز التعاقد في حالات الضرورة مع ذوي الخبرات في التخصصات النادرة وفقًا للشروط والضوابط الآتية: 1- ألا يوجد بالوحدة والأجهزة التابعة لها من يملك خبرة مماثلة في التخصص المطلوب ويمكن الاستعانة به. 2- ألا تقل خبرة المتعاقد معه في التخصص المطلوب عن عشر سنوات. 3- عدم الإخلال بالحد الأقصى للدخول. 4- أن يكون التعاقد لمدة أو لمدد لا تجاوز ثلاث سنوات. 5- أن يكون التعاقد بموافقة رئيس مجلس الوزراء, بناءً على عرض الوزير المختص. التعيين في الوظائف القيادية والإدارة الإشرافية المادة 17 يكون التعيين في الوظائف القيادية والإدارة الإشرافية عن طريق مسابقة يعلن عنها على موقع بوابة الحكومة المصرية أو النشر في جريدتين واسعتي الانتشار متضمنًا البيانات المتعلقة بالوظيفة. ويكون التعيين من خلال لجنة للاختيار لمدة أقصاها ثلاث سنوات, يجوز تجديدها بحد أقصى ثلاث سنوات, بناءً على تقارير تقويم الأداء, وذلك دون الإخلال بباقي الشروط اللازمة لشغل هذه الوظائف. ويشترط للتعيين في هذه الوظائف التأكد من توفر صفات النزاهة من الجهات المعنية على أن يستند الرأي بعدم توفرها إلى قرائن كافية, وأسباب جدية, واجتياز التدريب اللازم, ويحدد الجهاز مستوى البرامج التدريبية المتطلبة والجهات المعتمدة لتقديم هذه البرامج. وتحدد اللائحة التنفيذية إجراءات وقواعد اختيار شاغلي هذه الوظائف وتشكيل لجنة الاختيار والإعداد والتأهيل اللازمين لشغلها وإجراءات تقويم نتائج أعمال شاغليها. واستثناءً من أحكام هذا القانون يجوز للوزراء اختيار مساعدين ومعاونين لهم لمدة محددة وفقًا للنظام الذي يصدر به قرار من رئيس مجلس الوزراء بناءً على عرض الوزير المختص واقتراح الجهاز على أن يتضمن هذا النظام على الأخص قواعد اختيار وتقويم أداء هؤلاء والمعاملة المالية المقررة لهم. المادة 18 تُنشأ بكل وزارة وظيفة واحدة لوكيل دائم للوزارة بالمستوى الممتاز لمعاونة الوزير في مباشرة اختصاصاته. واستثناءً من أحكام المادة (17) من هذا القانون يختار الوزير الوكيل الدائم من خلال لجنة للاختيار وذلك لمدة أقصاها أربع سنوات, يجوز تجديدها بحد أقصى أربع سنوات أخرى, يكلف خلالها بضمان الاستقرار التنظيمي والمؤسسي للوزارة والهيئات والأجهزة التابعة لها, ورفع مستوى كفاءة تنفيذ سياساتها, واستمرارية البرامج والمشروعات والخطط, ومتابعتها تحت إشراف الوزير. وتحدد اللائحة التنفيذية قواعد وضوابط اختيار الوكيل الدائم. المادة 19 يؤدي كل موظف يعين في وظيفة من الوظائف القيادية أمام السلطة المختصة وقبل أن يباشر عمله اليمين الآتية: ((أقسم بالله العظيم أن أحترم الدستور والقانون, وأن أخدم الدولة, وأن أحافظ على المال العام, وأن أؤدي واجباتي الوظيفية بنزاهة وشفافية وبروح فريق العمل وعلى الوجه الأكمل لخدمة الشعب)). المادة 20 تنتهي مدة شغل الوظائف القيادية والإدارة الإشرافية بانقضاء المدة المحددة في قرار شغلها ما لم يصدر قرار بتجديدها, وبانتهاء هذه المدة يشغل الموظف وظيفة أخرى لا يقل مستواها عن مستوى الوظيفة التي كان يشغلها إذا كان من موظفي الدولة قبل شغله لإحدى هذه الوظائف. ويجوز للموظف خلال الثلاثين يومًا التالية لانتهاء مدة شغله لإحدى الوظائف المشار إليها طلب إنهاء خدمته, وفي هذه الحالة تسوى حقوقه التأمينية على أساس مدة اشتراكه في التأمين الاجتماعي مضافًا إليها مدة خمس سنوات أو المدة الباقية لبلوغه السن المقررة قانونًا لترك الخدمة أيهما أقل, ويعامل فيما يتعلق بالمعاش الذي يستحقه في وظيفته السابقة معاملة من تنتهي خدمته ببلوغ هذه السن. وتتحمل الخزانة العامة للدولة الزيادة في الحقوق التأمينية الناتجة عن تطبيق هذه المادة. ويجب أن تتخذ الإجراءات اللازمة لتجديد مدة شغل الوظائف القيادية والإدارة الإشرافية أو النقل منها طبقًا للأحكام السابقة قبل انتهاء المدة المحددة لشغل الوظيفة بستين يوماً على الأقل. المادة 21 لا تسري أحكام المادتين (17, 20) من هذا القانون على الجهات والوظائف ذات الطبيعة الخاصة التي يصدر بتحديدها قرار من رئيس الجمهورية, ويكون التعيين في الوظائف القيادية والإدارة الإشرافية في هذه الجهات والوظائف بقرار من رئيس الجمهورية أو من يفوضه. ويكون شغل هذه الوظائف عن طريق الترقية بالاختيار وذلك على أساس بيانات تقويم الأداء وما ورد في ملف الخدمة من عناصر الامتياز. العلاقة الوظيفية المادة 22 تعتبر الأقدمية في الوظيفة من تاريخ شغلها, فإذا اتحد تاريخ شغل الوظيفة لأكثر من موظف اعتبرت الأقدمية وفقًا لما يأتي: 1- إذا كان شغل الوظيفة لأول مرة اعتبرت الأقدمية بحسب الأسبقية في التعيين طبقًا لما ورد في المادة (12) من هذا القانون. 2- إذا كان شغل الوظيفة بطريق الترقية اعتبرت الأقدمية على أساس الأقدمية في الوظيفة السابقة. المادة 23 مع عدم الإخلال بأحكام المادة (76) من هذا القانون, يجوز للموظفين الحاصلين على مؤهلات أعلى قبل الخدمة أو أثنائها, التقدم للوظائف الخالية بالوحدات التي يعملون بها, أو غيرها من الوحدات, متى كانت تلك المؤهلات متطلبة لشغلها, وبشرط استيفائهم الشروط اللازمة لشغل هذه الوظائف. المادة 24 لا يجوز بأية حال من الأحوال أن يعمل موظف تحت الرئاسة المباشرة لأحد أقاربه من الدرجة الأولى في ذات الوحدة, وتحدد اللائحة التنفيذية الإجراءات الواجب اتخاذها عند توافر هذه الحالة. الباب الثالث تقويم الأداء المادة 25 تضع السلطة المختصة نظاماً يكفل تقويم أداء الموظف بالوحدة بما يتفق وطبيعة نشاطها وأهدافها ونوعية وظائفها. ويكون تقويم أداء الموظف عن سنة مالية على مرتين على الأقل قبل وضع التقرير النهائي, ويقتصر تقويم الأداء على القائمين بالعمل فعلاً بالوحدة مدة ستة أشهر على الأقل. ويكون الأداء العادي هو الأساس المُعول عليه في تقويم أداء الموظفين بما يحقق أهداف الوحدة ونشاطها ونوعية الوظائف بها. ويكون تقويم الأداء بمرتبة ممتاز, أو كفء, أو فوق المتوسط, أو متوسط, أو ضعيف. وتُحدد اللائحة التنفيذية ضوابط وإجراءات التقويم بما يكفل الحيادية والدقة في القياس وصولاً للمنحنى الطبيعي للأداء, وكذا ميعاد وضع تقارير التقويم وكيفية اعتمادها والتظلم منها ومعادلة هذه المراتب بالمراتب المعمول بها في تاريخ العمل بهذا القانون. ويقدر تقويم أداء الموظف الذي لم يقم بالعمل فعلياً بالوحدة لمدة ستة أشهر على الأقل بسبب التجنيد, أو للاستدعاء للاحتياط أو للاستبقاء, أو للمرض, أو لإجازة رعاية الطفل, أو لعضوية أحد المجالس النقابية, أو لعضوية مجلس النواب بمرتبة كفء حكماً, فإذا كان تقويم أدائه في العام السابق بمرتبة ممتاز يقدر بمرتبة ممتاز حكماً. المادة 26 تُعلن إدارة الموارد البشرية الموظف بصورة من تقرير تقويم أدائه بمجرد اعتماده من السلطة المختصة. وله أن يتظلم منه خلال خمسة عشر يوماً من تاريخ إعلانه. ويكون تظلم الموظفين شاغلي الوظائف القيادية والإدارة الإشرافية من التقارير المقدمة عن أدائهم إلى السلطة المختصة. ويكون تظلم باقي الموظفين إلى لجنة تظلمات، تنشأ لهذا الغرض، وتُشكل بقرار من السلطة المختصة من ثلاثة من شاغلي الوظائف القيادية، وعضو تختاره اللجنة النقابية بالوحدة إن وُجدت. ويُبت في التظلم خلال ستين يوماً من تاريخ تقديمه، ويجب على إدارة الموارد البشرية إعلان الموظف بنتيجة تظلمه والأسباب التي بُني عليها، ويكون قرار السلطة المختصة أو اللجنة نهائياً، وذلك مع عدم الإخلال بحقه في التقاضي. ولا يُعتبر تقرير تقويم الأداء نهائياً إلا بعد انقضاء ميعاد التظلم منه أو البت فيه. وتحدد اللائحة التنفيذية كيفية إعلان الموظف بتقرير تقويم الأداء ونتيجة التظلم منه. المادة 27 يُعرض أمر الموظف الذي يُقدم عنه تقريران سنويان متتاليان بمرتبة ضعيف على لجنة الموارد البشرية، لنقله لوظيفة أخرى ملائمة في ذات مستوى وظيفته لمدة سنة. وإذا تبين للجنة بعد انقضاء المدة المشار إليها في الفقرة السابقة أنه غير صالح للعمل بها بطريقة مرضية، اقترحت خصم (50%) من الأجر المكمل لمدة ستة أشهر. وإذا تبين بعدها أنه غير صالح للعمل، اقترحت اللجنة إنهاء خدمته لعدم الصلاحية للوظيفة مع حفظ حقوقه التأمينية. وفي جميع الأحوال ترفع اللجنة تقريرها للسلطة المختصة للاعتماد. المادة 28 تنتهي لعدم الصلاحية للوظيفة خدمة شاغلي الوظائف القيادية الذين يُقدم عنهم تقريران متتاليان بمرتبة أقل من فوق المتوسط من اليوم التالي لتاريخ صدور آخر تقرير نهائي مع حفظ حقهم في المعاش. الباب الرابع الترقية، والنقل، والندب، والحلول والإعارة الترقية المادة 29 مع مراعاة استيفاء الموظف لشروط شغل الوظيفة المرقى إليها, تكون الترقية بموجب قرار يصدر من السلطة المختصة من الوظيفة التي تسبقها مباشرة في المستوى والمجموعة الوظيفية التي تنتمي إليها. وتكون الترقية للوظائف التخصصية من المستوى الأول (ب) بالاختيار على أساس بيانات تقويم الأداء وما ورد في ملف الخدمة من عناصر الامتياز, وتكون الترقية للوظائف التخصصية الأخرى بالاختيار في حدود النسب الواردة في الجدول رقم (1) الملحق بهذا القانون. وتكون الترقية لباقي الوظائف بالأقدمية. ويُشترط للترقية أن يحصل الموظف على تقرير تقويم أداء بمرتبة كفء في السنتين السابقتين مباشرة على الترقية, أما الترقية بالاختيار في الوظائف التخصصية فيجب الحصول على تقرير تقويم أداء بمرتبة ممتاز. فإذا كان عدد من تتوافر فيهم شروط الترقية بالاختيار من الحاصلين على مرتبة ممتاز أقل من العدد المخصص للترقية بالاختيار تكون الترقية في الجزء الباقي من الحاصلين على مرتبة كفء على الأقل عن ذات المدة السابقة. فإذا كان عدد من تتوافر فيهم شروط الترقية بالاختيار أقل من العدد المخصص لها تؤجل الترقية وتحجز الوظائف في الجزء المتبقي في أول ترقية تالية. وباستثناء جزاءي الإنذار والخصم من الأجر مدة لا تزيد على عشرة أيام, لا تجوز ترقية الموظف قبل محو الجزاء الموقع عليه. وتُحدد اللائحة التنفيذية ضوابط وإجراءات الترقية. المادة 30 تُفضل عند الترقية بالاختيار ترقية الأعلى في مجموع درجات تقويم أداء السنتين السابقتين مباشرة على الترقية, وعند التساوي يفضل الأعلى في مجموع درجات تقويم أداء السنة السابقة عليهما, فالحاصل على درجة علمية أعلى متى كانت متصلة بطبيعة العمل طبقًا لما تقرره السلطة المختصة بناءً على اقتراح لجنة الموارد البشرية, وعند التساوي يفضل الأعلى في التقدير العام لهذه الدرجة, فالأقدم في المستوى الوظيفي المرقى منه. المادة 31 يصدر قرار الترقية من السلطة المختصة, وتعتبر الترقية نافذة من تاريخ صدور القرار بها. ويستحق الموظف اعتبارًا من هذا التاريخ الأجر الوظيفي المقرر للوظيفة المرقى إليها أو أجره السابق مضافًا إليه علاوة ترقية بنسبة (5%) من هذا الأجر الوظيفي أيهما أكبر. النقل المادة 32 يجوز بقرار من السلطة المختصة نقل الموظف من وحدة إلى أخرى وذلك إذا كان النقل لا يفوت عليه دوره في الترقية أو كان بناءً على طلبه. ويكون نقل شاغلي الوظائف القيادية إلى خارج الوحدة بقرار من رئيس مجلس الوزراء. ولا يجوز نقل الموظف من وظيفة إلى أخرى تقل في مستواها عن مستوى وظيفته الأصلية. وتحدد اللائحة التنفيذية القواعد الخاصة بالنقل. الندب المادة 33 يجوز بقرار من السلطة المختصة, ندب الموظف للقيام مؤقتًا بعمل وظيفة أخرى من ذات المستوى الوظيفي لوظيفته أو من المستوى الذي يعلوه مباشرةً في ذات الوحدة التي يعمل بها أو في وحدة أخرى, إذا كانت حاجة العمل في الوظيفة الأصلية تسمح بذلك. ولا يجوز ندب الموظف خارج الوحدة إلا بناءً على طلبه. وتُحدد اللائحة التنفيذية القواعد الخاصة بالندب, على ألا تزيد مدته على أربع سنوات. وللوحدة المنتدب إليها الموظف اتخاذ إجراءات نقله من الوحدة المنتدب منها, بعد هذه المدة, وفي حالة رغبة الموظف, ووفقًا لحاجة العمل. واستثناءً مما تقدم, يجوز بقرار من السلطة المختصة ندب الموظف بعد موافقته إلى الجمعيات والمؤسسات الأهلية ذات النفع العام, وتتحمل الوحدة بكامل الأجر أو بعضه, وذلك على النحو الذي تحدده اللائحة التنفيذية. الحلول المادة 34 عند غياب شاغل وظيفة من الوظائف القيادية والإدارة الإشرافية عن العمل, يحل محله في مباشرة واجبات ومسئوليات وظيفته من يليه مباشرة في ترتيب الأقدمية ما لم تُحدد السلطة المختصة من يحل محله, على أن يكون من ذات مستواه أو من المستوى الأدنى مباشرة. الإعارة المادة 35 يجوز بقرار من السلطة المختصة إعارة الموظف للعمل بالداخل أو الخارج بعد موافقة كتابية منه، ويُحدد القرار الصادر بالإعارة مدتها. ويترتب على إعارة شاغل وظيفة من الوظائف القيادية أو الإدارة الإشرافية انتهاء مدة شغله لها. ويكون أجر الموظف المعار بكامله على الجهة المُستعيرة، وتدخل مدة الإعارة ضمن مدة خدمته، ولا يجوز ترقية المُعار إلا بعد عودته من الإعارة واستكمال المدة البينية اللازمة لشغل الوظيفة الأعلى مباشرةً ولا تدخل مدة الإعارة ضمن المدة البينية اللازمة للترقية. وتدخل مدة الإعارة ضمن مدة اشتراك الموظف في نظام التأمين الاجتماعي واستحقاق العلاوة، وذلك مع مراعاة أحكام قانون التأمين الاجتماعي الصادر بالقانون رقم 79 لسنة 1975 وتحدد اللائحة التنفيذية القواعد الخاصة بالإعارة. الباب الخامس (الأجور والعلاوات) الأجر الوظيفي المادة 36 يُحدد الأجر الوظيفي للوظائف وفقاً للجداول أرقام (1، 2، 3) الملحقة بهذا القانون. ويستحق الموظف أجره من تاريخ تسلمه العمل، ما لم يكن مُستبقًى بالقوات المسلحة فيستحق أجره من تاريخ التعيين. العلاوات المادة 37 يستحق الموظف علاوة دورية سنوية في الأول من يوليو التالي لانقضاء سنة من تاريخ شغل الوظيفة أو من تاريخ استحقاق العلاوة الدورية السابقة بنسبة (7%) من الأجر الوظيفي، على أن يعاد النظر في هذه النسبة بصفة دورية منتظمة. المادة 38 يجوز للسلطة المختصة منح الموظف علاوة تشجيعية بنسبة (5%) من أجره الوظيفي، وذلك طبقاً للشروط الآتية: 1- أن تكون كفاية الموظف قد حُددت بمرتبة كفء على الأقل عن العامين الأخيرين. 2- ألا يمنح الموظف هذه العلاوة أكثر من مرة كل ثلاثة أعوام. 3- ألا يزيد عدد الموظفين الذين يُمنحون هذه العلاوة في سنة واحدة على (10%) من عدد الموظفين في وظائف كل مستوى من كل مجموعة نوعية على حدة، فإذا كان عدد الموظفين في تلك الوظائف أقل من عشرة، تُمنح العلاوة لواحد منهم. المادة 39 يستحق الموظف الذي يحصل على مؤهل أعلى أثناء الخدمة حافز تميز علمي. ويمنح الموظف هذا الحافز إذا حصل على دبلومة مدتها سنتان دراسيتان على الأقل، أو على درجة الماجستير أو ما يعادلها أو دبلومتين من دبلومات الدراسات العليا مدة كل منهما سنة دراسية على الأقل، كما يمنح الموظف حافز تميز آخر إذا حصل على درجة الدكتوراة أو ما يعادلها. ويكون حافز التميز العلمي المشار إليه بنسبة (7%) من الأجر الوظيفي، أو الفئات المالية التالية أيهما أكبر: 25 جنيهاً شهرياً لمن يحصل على مؤهل متوسط أو فوق المتوسط. 50 جنيهاً شهرياً لمن يحصل على مؤهل عال. 75 جنيهاً شهرياً لمن يحصل على دبلومة مدتها سنتان دراسيتان على الأقل. 100 جنيه شهرياً لمن يحصل على درجة الماجستير أو ما يعادلها، أو دبلومتين من دبلومات الدراسات العليا مدة كل منها سنة دراسية على الأقل. 200 جنيه شهرياً لمن يحصل على درجة الدكتوراة أو ما يعادلها. وتحدد اللائحة التنفيذية شروط وضوابط منح حافز التميز، على ألا يجوز منح هذا الحافز أكثر من مرة عن ذات المستوى العلمي. المادة 40 تُضم العلاوات المقررة بمقتضى هذا القانون إلى الأجر الوظيفي للموظف. الأجر المكمل المادة 41 يصدر بنظام الأجر المكمل قرار من رئيس مجلس الوزراء بمراعاة طبيعة عمل كل وحدة ونوعية الوظائف بها وطبيعة اختصاصاتها ومعدلات أداء موظفيها بحسب الأحوال بناءً على عرض الوزير المختص وبعد موافقة وزير المالية ودراسة الجهاز. المادة 42 يجوز للسلطة المختصة تقرير مكافآت تشجيعية للموظف الذي يقدم خدمات ممتازة أو أعمالاً أو بحوثًا أو اقتراحات تساعد على تحسين طرق العمل, أو رفع كفاءة الأداء, أو توفير في النفقات, وذلك كله بشرط سماح البند المخصص لذلك في الموازنة العامة. المادة 43 يجوز لرئيس الجمهورية في الحالات التي يُقدرها الاحتفاظ لمن يُعين بوظيفة أخرى بكامل أو بعض الأجر الذي كان يتقاضاه قبل التعيين بها. المادة 44 تُشجع الدولة زيادة وعي الموظفين بالعلوم والتكنولوجيا, والعمل على نشر المعارف بينهم, وتطوير القدرات الابتكارية. وتكون الاختراعات والمصنفات التي يبتكرها الموظف أثناء تأدية وظيفته أو بسببها ملكًا للدولة إذا كان الاختراع نتيجة تجارب رسمية أو له صلة بالشئون العسكرية, أو إذا كان الاختراع أو المصنف يدخل في نطاق أعمال الوظيفة. وفي جميع الأحوال يكون للموظف الحق في تعويض عادل, يُراعى عند تقديره تشجيع البحث والاختراع. ويجوز أن يُنشأ صندوق خاص في الوحدة, تتكون موارده من حصيلة استغلال حق هذه الاختراعات والمصنفات, ويكون الصرف من حصيلة هذا الصندوق طبقًا للائحة المالية التي تضعها السلطة المختصة. المادة 45 تضع السلطة المختصة بالاشتراك مع اللجنة النقابية للوحدة نظامًا للرعاية الاجتماعية والثقافية والرياضية للموظفين بها, وذلك بمراعاة أحكام التشريعات ذات الصلة. الباب السادس الإجازات المادة 46 تُحدد السلطة المختصة أيام العمل في الأسبوع ومواقيته وتوزيع ساعاته وفقًا لمقتضيات المصلحة العامة, على ألا يقل عدد ساعات العمل الأسبوعية عن خمس وثلاثين ساعة ولا يزيد على اثنتين وأربعين ساعة. وتخفض عدد ساعات العمل اليومية بمقدار ساعة للموظف ذي الإعاقة, والموظفة التي ترضع طفلها وحتى بلوغه العامين, والحالات الأخرى التي تبينها اللائحة التنفيذية. ولا يجوز للموظف أن ينقطع عن عمله إلا لإجازة يُرخص له بها في حدود الإجازات المقررة في هذا القانون, ووفقًا للضوابط والإجراءات التي تحددها اللائحة التنفيذية, وإلا حُرم من أجره عن مدة الانقطاع دون الإخلال بمسئوليته التأديبية. إجازة بأجر كامل المادة 47 يستحق الموظف إجازة بأجر كامل عن أيام عطلات الأعياد والمناسبات الرسمية التي تحدد بقرار من رئيس مجلس الوزراء, ويجوز تشغيل الموظف في هذه العطلات إذا اقتضت الضرورة ذلك, مع منحه أجرًا مماثلاً مضافًا إلى أجره المستحق أو إجازة عوضًا عنها. وتسري بالنسبة للأعياد الدينية لغير المسلمين أحكام قرار رئيس مجلس الوزراء الصادر في هذا الشأن. المادة 48 للموظف أن ينقطع عن العمل لسبب عارض لمدة لا تتجاوز سبعة أيام خلال السنة وبحد أقصى يومان في المرة الواحدة. المادة 49 يستحق الموظف إجازة اعتيادية سنوية بأجر كامل, لا يدخل في حسابها أيام عطلات الأعياد والمناسبات الرسمية فيما عدا العطلات الأسبوعية, وذلك على الوجه الآتي: 1- 15 يومًا في السنة الأولى, وذلك بعد مضي ستة أشهر من تاريخ استلام العمل. 2- 21 يومًا لمن أمضى سنة كاملة في الخدمة. 3- 30 يومًا لمن أمضى عشر سنوات في الخدمة. 4- 45 يومًا لمن تجاوزت سنه الخمسين. ويستحق الموظف من ذوي الإعاقة إجازة اعتيادية سنوية مدتها خمسة وأربعون يومًا دون التقيد بعدد سنوات الخدمة. وللسلطة المختصة أن تقرر زيادة مدة الإجازة الاعتيادية بما لا يجاوز خمسة عشر يومًا لمن يعملون في المناطق النائية, أو إذا كان العمل في أحد فروع الوحدة خارج الجمهورية. ولا يجوز تقصير أو تأجيل الإجازة الاعتيادية أو إنهاؤها إلا لأسباب قومية تقتضيها مصلحة العمل. المادة 50 يجب على الموظف أن يتقدم بطلب للحصول على كامل إجازاته الاعتيادية السنوية، ولا يجوز للوحدة ترحيلها إلا لأسباب تتعلق بمصلحة العمل وفي حدود الثلث على الأكثر ولمدة لا تزيد على ثلاث سنوات. وإذا لم يتقدم الموظف بطلب للحصول على إجازاته على النحو المشار إليه، سقط حقه فيها وفي اقتضاء مقابل عنها، أما إذا تقدم بطلب للحصول عليها ورفضته السلطة المختصة استحق مقابلاً نقدياً عنها يصرف بعد مرور ثلاث سنوات على انتهاء العام المستحق عنه الإجازة على أساس أجره الوظيفي في هذا العام. وتبين اللائحة التنفيذية إجراءات الحصول على الإجازة، وكيفية ترحيلها. المادة 51 يستحق الموظف إجازة مرضية عن كل ثلاث سنوات تقضي في الخدمة، وتُمنح بقرار من المجلس الطبي المختص في الحدود الآتية: 1- الثلاثة أشهر الأولى بأجر كامل. 2- الثلاثة أشهر التالية بأجر يعادل (75%) من الأجر الوظيفي. 3- الستة أشهر التالية بأجر يعادل (50%) من أجره الوظيفي، (75%) من الأجر الوظيفي لمن يجاوز سن الخمسين. ويحق للموظف طلب مد الإجازة المرضية بدون أجر للمدة التي يحددها المجلس الطبي المختص إذا قرر احتمال شفائه. ويحق للموظف أن يطلب تحويل الإجازة المرضية إلى إجازة اعتيادية، إذا كان له رصيد منها، وعلى الموظف المريض أن يخطر جهة عمله عن مرضه خلال أربع وعشرين ساعة من انقطاعه عن العمل للمرض إلا إذا تعذر عليه ذلك لأسباب قهرية. وتضع السلطة المختصة الإجراءات المنظمة لحصول الموظف على الإجازة المرضية، ويعتبر التمارض إخلالاً بواجبات الوظيفة. ويمنح الموظف المريض بأحد الأمراض المزمنة التي يصدر بتحديدها قرار من وزير الصحة بناءً على موافقة المجلس الطبي المختص إجازة استثنائية بأجر كامل إلى أن يشفى أو تستقر حالته استقراراً يمكنه من العودة إلى العمل أو يتبين عجزه عجزاً كاملاً، وفي هذه الحالة الأخيرة يظل الموظف في إجازة مرضية بذات الأجر حتى بلوغه سن الإحالة للمعاش. وإذا رغب الموظف المريض في إنهاء إجازته والعودة إلى عمله، وجب عليه أن يقدم طلباً كتابياً بذلك، وأن يوافق المجلس الطبي المختص على عودته. المادة 52 تكون حالات الترخيص بإجازة خاصة بأجر كامل على الوجه الآتي: 1- يستحق الموظف إجازة لمدة ثلاثين يوماً، ولمرة واحدة طوال مدة عمله بالخدمة المدنية لأداء فريضة الحج. 2- تستحق الموظفة إجازة وضع لمدة أربعة أشهر، بحد أقصى ثلاث مرات طوال مدة عملها بالخدمة المدنية، على أن تبدأ هذه الإجازة من اليوم التالي للوضع، ويجوز أن تبدأ هذه الإجازة قبل شهر من التاريخ المتوقع للوضع بناءً على طلب مقدم من الموظفة وتقرير من المجلس الطبي المختص. 3- يستحق الموظف المخالط لمريض بمرض مُعد إجازة للمدة التي يحددها المجلس الطبي المختص. 4- يستحق الموظف الذي يصاب إصابة عمل إجازة للمدة التي يحددها المجلس الطبي المختص، وذلك مع مراعاة أحكام قانون التأمين الاجتماعي المشار إليه. 5- يستحق الموظف المقيد بإحدى الكليات أو المعاهد أو المدارس إجازة عن أيام الامتحان الفعلية. الإجازة بدون أجر المادة 53 تكون حالات الترخيص بالإجازة بدون أجر على الوجه الآتي: 1- يمنح الزوج أو الزوجة إذا سافر أحدهما إلى الخارج للعمل أو للدراسة لمدة ستة أشهر على الأقل إجازة بدون أجر مدة بقاء الزوج أو الزوجة في الخارج، وفي جميع الأحوال يتعين على الوحدة أن تستجيب لطلب الزوج أو الزوجة. 2- يجوز للسلطة المختصة منح الموظف إجازة بدون أجر للأسباب التي يبديها وتقدرها السلطة المختصة ووفقاً لحاجة العمل. ولا يجوز في البندين السابقين ترقية الموظف إلا بعد عودته من الإجازة واستكمال المدة البينية اللازمة لشغل الوظيفة الأعلى مباشرة، ولا تدخل مدد الإجازات المنصوص عليها في هذين البندين السابقين ضمن المدد البينية اللازمة للترقية. 3- مع مراعاة أحكام قانون الطفل الصادر بالقانون رقم (12) لسنة 1996، تستحق الموظفة إجازة بدون أجر لرعاية طفلها لمدة عامين على الأكثر في المرة الواحدة، وبحد أقصى ستة أعوام طوال مدة عملها بالخدمة المدنية. واستثناءً من أحكام قانون التأمين الاجتماعي المشار إليه، تتحمل الوحدة اشتراكات التأمين المستحقة عليها وعلى الموظفة. المادة 54 يجوز للسلطة المختصة، وفقاً للقواعد التي تضعها، الترخيص للموظف بأن يعمل بعض الوقت بناءً على طلبه وذلك مقابل نسبة من الأجر. ويستحق الموظف في هذه الحالة الإجازات الاعتيادية والعارضة والمرضية المقررة له بما يتفق مع الجزء من الوقت الذي خصصه لعمله. وتحدد اللائحة التنفيذية قواعد احتساب الأجر المشار إليه. واستثناءً من أحكام قانون التأمين الاجتماعي المشار إليه، تؤدي الاشتراكات المستحقة وفقاً لأحكام هذا القانون من الأجر المخفض على أساس الأجر الكامل، وتدخل المدة بالكامل ضمن مدة اشتراكه. المادة 55 لا يستحق المجند والمستبقي والمستدعى للاحتياط أية إجازة تم النص عليها في هذا القانون طوال مدة وجوده بالقوات المسلحة. المادة 56 يُحظر على الموظف أن يؤدي عملاً للغير بأجر أو بدون أجر خلال مدة الإجازة بغير ترخيص من السلطة المختصة، وإلا حُرم من أجره عن مدة الإجازة، وللوحدة أن تسترد ما أدته إليه من أجر عن هذه المدة، وذلك دون الإخلال بالمسئولية التأديبية. الباب السابع السلوك الوظيفي والتأديب المادة 57 يتعين على الموظف الالتزام بأحكام هذا القانون ولائحته التنفيذية وغيرهما من القوانين واللوائح والقرارات والتعليمات المنفذة لها، وما يصدر عن الجهاز من قرارات تنظيمية أو تعليمات أو نشرات أو كتب دورية في هذا الشأن، ومدونات السلوك وأخلاقيات الخدمة المدنية الصادرة من الوزير المختص. ويحظر على الموظف بصفة خاصة مباشرة الأعمال التي تتنافى مع الحيدة، والتجرد، والالتزام الوظيفي أثناء ساعات العمل الرسمية، أو ممارسة أي عمل حزبي، أو سياسي داخل مكان عمله، أو بمناسبة تأديته لهذا العمل، أو القيام بجمع تبرعات، أو مساهمات لصالح أحزاب سياسية، أو نشر الدعاية أو الترويج لها. المادة 58 كل موظف يخرج على مقتضى الواجب في أعمال وظيفته، أو يظهر بمظهر من شأنه الإخلال بكرامة الوظيفة يجازى تأديبياً. ولا يعفى الموظف من الجزاء استناداً إلى أمر صادر إليه من رئيسه إلا إذا ثبت أن ارتكاب المخالفة كان تنفيذاً لأمر مكتوب بذلك صادر إليه من هذا الرئيس، بالرغم من تنبيهه كتابةً إلى المخالفة، وفي هذه الحالة تكون المسئولية على مُصدر الأمر وحده. ولا يسأل الموظف مدنياً إلا عن خطئه الشخصي. المادة 59 لا يجوز توقيع أي جزاء على الموظف إلا بعد التحقيق معه كتابةً، وسماع أقواله وتحقيق دفاعه، ويكون القرار الصادر بتوقيع الجزاء مسبباً. ومع ذلك، يجوز بالنسبة لجزاءي الإنذار والخصم من الأجر لمدة لا تجاوز ثلاثة أيام أن يكون التحقيق شفاهه، على أن يثبت مضمونه في القرار الصادر بتوقيع الجزاء. المادة 60 تختص النيابة الإدارية دون غيرها بالتحقيق مع شاغلي الوظائف القيادية, وكذا تختص دون غيرها بالتحقيق في المخالفات المالية التي يترتب عليها ضياع حق من الحقوق المالية للدولة أو المساس بها. كما تتولى التحقيق في المخالفات الأخرى التي تحال إليها ويكون لها بالنسبة لهذه المخالفات السلطات المقررة للسلطة المختصة في توقيع الجزاءات أو الحفظ. وعلى الجهة الإدارية المختصة بالنسبة لسائر المخالفات أن توقف ما تجريه من تحقيق في واقعة ما أو وقائع وما يرتبط بها إذا كانت النيابة الإدارية قد بدأت التحقيق فيها, ويقع باطلاً كل إجراء أو تصرف يخالف ذلك. المادة 61 الجزاءات التي يجوز توقيعها على الموظف هي: 1- الإنذار. 2- الخصم من الأجر لمدة أو مدد لا تجاوز ستين يومًا في السنة. 3- الوقف عن العمل لمدة لا تجاوز ستة أشهر مع صرف نصف الأجر الكامل. 4- تأجيل الترقية عند استحقاقها لمدة لا تزيد على سنتين. 5- الخفض إلى وظيفة في المستوى الأدنى مباشرة. 6- الخفض إلى وظيفة في المستوى الأدنى مباشرة مع خفض الأجر إلى القدر الذي كان عليه قبل الترقية. 7- الإحالة إلى المعاش. 8- الفصل من الخدمة. أما الجزاءات التي يجوز توقيعها على شاغلي الوظائف القيادية هي: 1- التنبيه. 2- اللوم. 3- الإحالة إلى المعاش. 4- الفصل من الخدمة. وللسلطة المختصة بعد توقيع جزاء تأديبي على أحد شاغلي الوظائف القيادية والإدارة الإشرافية تقدير مدى استمراره في شغل تلك الوظيفة. وتحتفظ كل وحدة في حساب خاص بحصيلة جزاءات الخصم الموقعة على العاملين, ويكون الصرف من هذه الحصيلة في الأغراض الاجتماعية, أو الثقافية, أو الرياضية للعاملين طبقًا للشروط والأوضاع التي تُحددها السلطة المختصة. المادة 62 يكون الاختصاص بالتصرف في التحقيق على النحو الآتي: 1- للرؤساء المباشرين الذين تُحددهم السلطة المختصة كل في حدود اختصاصه, حفظ التحقيق أو توقيع جزاء الإنذار أو الخصم من الأجر, بما لا يجاوز عشرين يومًا في السنة ولا يزيد على ثلاثة أيام في المرة الواحدة. 2- لشاغلي الوظائف القيادية والإدارة الإشرافية كل في حدود اختصاصه, حفظ التحقيق أو توقيع جزاء الإنذار أو الخصم من الأجر, بما لا يجاوز أربعين يومًا في السنة ولا يزيد على خمسة عشر يومًا في المرة الواحدة. 3- للسلطة المختصة حفظ التحقيق أو توقيع أي من الجزاءات المنصوص عليها في البنود من (1) إلى (5) من الفقرة الأولى من المادة (61) من هذا القانون والبندين (1, 2) من الفقرة الثانية من ذات المادة. 4- للمحكمة التأديبية المختصة توقيع أي من الجزاءات المنصوص عليها في هذا القانون. وتكون الجهة المنتدب أو المعار إليها الموظف هي المختصة بالتحقيق معه وتأديبه طبقًا لأحكام هذا القانون عن المخالفات التي يرتكبها خلال فترة الندب أو الإعارة. المادة 63 لكل من السلطة المختصة ورئيس هيئة النيابة الإدارية بحسب الأحوال أن يوقف الموظف عن عمله احتياطيًا إذا اقتضت مصلحة التحقيق معه ذلك لمدة لا تزيد على ثلاثة أشهر, ولا يجوز مد هذه المدة إلا بقرار من المحكمة التأديبية المختصة للمدة التي تحددها, ويترتب على وقف العامل عن عمله وقف صرف نصف أجره ابتداءً من تاريخ الوقف. ويجب عرض الأمر فورًا على المحكمة التأديبية المختصة لتقرير صرف أو عدم صرف المتبقي من أجره, فإذا لم يعرض الأمر عليها خلال عشرة أيام من تاريخ الوقف وجب صرف كامل أجره حتى تقرر المحكمة ما يُتبع في شأنه. وعلى المحكمة التأديبية أن تُصدر قرارها خلال عشرين يومًا من تاريخ رفع الأمر إليها, فإذا لم تصدر المحكمة قرارها في خلال هذه المدة يصرف الأجر كاملاً. فإذا برئ الموظف أو حفظ التحقيق معه أو جُوزي بجزاء الإنذار أو الخصم من الأجر لمدة لا تجاوز خمسة أيام صرف إليه ما يكون قد أوقف صرفه من أجره, وإذا جُوزي بجزاء أشد تقرر السلطة التي وقعت الجزاء ما يُتبع في شأن الأجر الموقوف صرفه, فإن جُوزي بجزاء الفصل انتهت خدمته من تاريخ وقفه ولا يجوز أن يسترد منه في هذه الحالة ما سبق أن صرف له من أجر. المادة 64 كل موظف يُحبس احتياطيًا أو تنفيذًا لحكم جنائي يُوقف عن عمله, بقوة القانون مدة حبسه, ويحرم من نصف أجره إذا كان الحبس احتياطيًا أو تنفيذًا لحكم جنائي غير نهائي, ويُحرم من كامل أجره إذا كان الحبس تنفيذًا لحكم جنائي نهائي. وإذا لم يكن من شأن الحكم الجنائي إنهاء خدمة الموظف يُعرض أمره عند عودته إلى عمله على السلطة المختصة لتقرير ما يُتبع في شأن مسئوليته التأديبية. المادة 65 لا تجوز ترقية الموظف المُحال إلى المحاكمة التأديبية أو الجنائية أو الموقوف عن العمل مدة الإحالة أو الوقف, وفي هذه الحالة تحجز وظيفة للموظف. وإذا بُرئ الموظف المُحال أو قضي بحكم نهائي بمعاقبته بالإنذار أو الخصم من الأجر لمدة لا تزيد على عشرة أيام وجب ترقيته اعتبارًا من التاريخ الذي كانت ستتم فيه الترقية لو لم يُحل إلى المحاكمة, ويُمنح أجر الوظيفة المرقى إليها من هذا التاريخ. وفي جميع الأحوال, لا يجوز تأخير ترقية الموظف لمدة تزيد على سنتين. المادة 66 لا يمنع انتهاء خدمة الموظف لأي سبب من الأسباب عدا الوفاة من محاكمته تأديبيًا إذا كان قد بُدئ في التحقيق قبل انتهاء مدة خدمته. ويجوز في المخالفات التي يترتب عليها ضياع حق من حقوق الخزانة العامة للدولة إقامة الدعوى التأديبية ولو لم يكن قد بدئ في التحقيق قبل انتهاء الخدمة وذلك لمدة خمس سنوات من تاريخ انتهائها. ويجوز أن يوقع على من انتهت خدمته غرامة لا تجاوز عشرة أضعاف أجره الوظيفي الذي كان يتقاضاه في الشهر عند انتهاء الخدمة, وذلك مع عدم الإخلال بالعقوبات الجنائية والتزامه برد قيمة الحق, واستثناءً من أحكام قانون التأمين الاجتماعي المشار إليه, تستوفى الغرامة المشار إليها بالفقرة السابقة من المعاش بما لا يجاوز ربعه, أو بطريق الحجز الإداري. المادة 67 تمحى الجزاءات التأديبية التي توقع على الموظف بانقضاء الفترات الآتية: 1- سنة في حالة الإنذار والتنبيه والخصم من الأجر مدة لا تزيد على خمسة أيام. 2- سنتان في حالة اللوم والخصم من الأجر مدة تزيد على خمسة أيام وحتى خمسة عشر يومًا. 3- ثلاث سنوات في حالة الخصم من الأجر مدة تزيد على خمسة عشر يومًا وحتى ثلاثين يومًا. 4- أربع سنوات بالنسبة إلى الجزاءات الأخرى عدا جزاءي الفصل والإحالة إلى المعاش. وتحسب فترات المحو اعتبارًا من تاريخ توقيع الجزاء. ويترتب على محو الجزاء اعتباره كأن لم يكن بالنسبة للمستقبل ولا يؤثر على الحقوق والتعويضات التي ترتبت نتيجة له. وتحدد اللائحة التنفيذية إجراءات المحو. المادة 68 تسقط الدعوى التأديبية بالنسبة للموظف الموجود بالخدمة بمضي ثلاث سنوات من تاريخ ارتكاب المخالفة. وتنقطع هذه المدة بأي إجراء من إجراءات التحقيق أو الاتهام أو المحاكمة, وتسري المدة من جديد ابتداءً من آخر إجراء. وإذا تعدد المتهمون فإن انقطاع المدة بالنسبة لأحدهم يترتب عليه انقطاعها بالنسبة للباقين ولو لم تكن قد اتخذت ضدهم إجراءات قاطعة للمدة. ومع ذلك إذا شكل الفعل جريمة جنائية, فلا تسقط الدعوى التأديبية إلا بسقوط الدعوى الجنائية. الباب الثامن انتهاء الخدمة المادة 69 تنتهي خدمة الموظف لأحد الأسباب الآتية: 1- بلوغ سن الستين بمراعاة أحكام قانون التأمين الاجتماعي المشار إليه, ويجوز بقرار من رئيس الجمهورية لاعتبارات يقدرها مد الخدمة لشاغلي الوظائف القيادية لمدة لا تجاوز ثلاث سنوات. 2- الاستقالة. 3- الإحالة إلى المعاش أو الفصل من الخدمة. 4- فقد الجنسية, أو انتفاء شرط المعاملة بالمثل بالنسبة لرعايا الدول الأخرى. 5- الانقطاع عن العمل بدون إذن خمسة عشر يومًا متتالية ما لم يقدم خلال الخمسة عشر يومًا التالية ما يثبت أن الانقطاع كان بعذر مقبول. 6- الانقطاع عن العمل بدون إذن ثلاثين يومًا غير متصلة في السنة. 7- عدم اللياقة للخدمة صحيًا وذلك بقرار من المجلس الطبي المختص. 8- الالتحاق بخدمة جهة أجنبية بغير ترخيص من حكومة جمهورية مصر العربية. 9- الحكم عليه بعقوبة جناية أو بعقوبة مقيدة للحرية في جريمة مخلة بالشرف أو الأمانة أو تفقده الثقة والاعتبار. 10- الوفاة, وفي هذه الحالة يُصرف ما يعادل الأجر الكامل لمدة شهرين لمواجهة نفقات الجنازة وذلك للأرمل أو لأرشد الأولاد أو لمن يثبت قيامه بتحمل هذه النفقات. وتُبين اللائحة التنفيذية قواعد وإجراءات إنهاء الخدمة لهذه الأسباب. المادة 70 للموظف الذي جاوز سن الخمسين أن يطلب إحالته للمعاش المبكر ما لم يكن قد اتخذت ضده إجراءات تأديبية، ويتعين على الوحدة الاستجابة لهذا الطلب وفقاً لما تحدده اللائحة التنفيذية، وفي هذه الحالة تُسوى حقوقه التأمينية على النحو الآتي: 1- إذا لم يكن قد جاوز سن الخامسة والخمسين وجاوزت مدة اشتراكه في نظام التأمين الاجتماعي عشرين عاماً ومضى على شغله الوظيفة أكثر من سنة، فيعتبر مُرقى إلى الوظيفة التالية لوظيفته من اليوم السابق على تاريخ إحالته للمعاش، وتُسوى حقوقه التأمينية بعد ترقيته على أساس مدة اشتراكه في نظام التأمين الاجتماعي مضافاً إليها خمس سنوات. 2- إذا كان قد جاوز سن الخامسة والخمسين وجاوزت مدة اشتراكه في التأمينات الاجتماعية عشرين عاماً، فتُسوى حقوقه التأمينية على أساس مدة اشتراكه في التأمينات الاجتماعية مضافاً إليها المدة الباقية لبلوغه السن المقررة لانتهاء الخدمة أو خمس سنوات أيهما أقل. ولا يجوز تعيين من يُحال للمعاش المبكر وفقاً لأحكام هذه المادة في أي من الوحدات الخاضعة لأحكام هذا القانون. الباب التاسع أحكام عامة وانتقالية المادة 71 يستحق الموظف عند انتهاء خدمته مقابلاً عن رصيد إجازاته الاعتيادية الذي تكوُن قبل العمل بأحكام هذا القانون ولم يستنفدها قبل انتهاء خدمته. ويحسب المقابل النقدي على أساس الأجر الأساسي مضافاً إليه العلاوات الخاصة التي كان يتقاضاها حتى تاريخ العمل بهذا القانون. المادة 72 يحتفظ شاغلو وظيفة كبير بوظائفهم بصفة شخصية إلى حين انتهاء مدة شغلهم لها، أو بلوغ سن التقاعد أيهما أقرب. المادة 73 يُعين في أدنى الدرجات على بند الأجور الثابتة بالباب الأول (أجور) كل من مضى على نقله على بند (أجور موسميين) على الباب الأول ثلاث سنوات على الأقل على وظائف واردة بموازنة الوحدة، بشرط استيفاء شروط شغل تلك الوظائف، وتعاقده قبل 30/6/2016. ويطبق حكم الفقرة الأولى على جميع العاملين المؤقتين والمتعاقدين بالجهات الخاضعة لأحكام قانون الخدمة المدنية المسند إليهم شغل الوظيفة العامة حتى 30/6/2016 وذلك كله على النحو الذي تبينه اللائحة التنفيذية. ويوضع نظام للتعاقد مع العمالة المؤقتة أو الموسمية على الباب الأول على أن يصدر به قرار من الوزير المختص بناءً على اقتراح الجهاز. المادة 74 يستمر العمل بالأحكام والقواعد الخاصة بتحديد المخصصات المالية للموظفين بالوظائف والجهات الصادر بتنظيم مخصصاتهم قوانين ولوائح خاصة طبقاً لجدول الأجور المقرر بها. ويستمر صرف باقي الحوافز والمكافآت والجهود غير العادية والأعمال الإضافية والبدلات وكافة المزايا النقدية والعينية وغيرها بخلاف المزايا التأمينية التي يحصل عليها الموظف بذات القواعد والشروط المقررة قبل العمل بأحكام هذا القانون بعد تحويلها من نسب مئوية مرتبطة بالأجر الأساسي إلى فئات مالية مقطوعة في 30/6/2015. المادة 75 تلتزم الوحدات المخاطبة بأحكام هذا القانون بتحديث الهياكل التنظيمية، وبطاقات الوصف، ودورات العمل، وحصر الخدمات التي تقدمها وإجراءاتها وشروطها، وذلك في مدة لا تجاوز عاماً من تاريخ العمل بهذا القانون. كما تلتزم تلك الجهات بوضع مؤشرات ومعايير الأداء، وطرق تقديم الخدمات العامة سواء بصورة مباشرة أو عن طريق إحدى الجهات غير الحكومية، وسبل تحقيق رضاء المواطنين. ويلتزم الجهاز بمتابعة تنفيذ هذه المهام في ضوء المعايير والآليات المنظمة التي يصدرها الوزير المختص، بعد العرض على مجلس الخدمة المدنية. المادة 76 يجوز للسلطة المختصة، ولمدة ثلاث سنوات اعتباراً من تاريخ العمل بهذا القانون، إعادة تعيين الموظفين المعينين قبل العمل بأحكامه، والحاصلين على مؤهلات أعلى أثناء الخدمة في الوظائف الخالية بالوحدات التي يعملون بها، متى توافرت فيهم الشروط اللازمة لشغل هذه الوظائف وفقاً لجداول الترتيب والتوصيف المعمول بها مع استثنائهم من شرطي الإعلان والامتحان اللازمين لشغل هذه الوظائف، وذلك كله وفقاً للقواعد والشروط التي تبينها اللائحة التنفيذية، على أن يتم التعيين في بداية مجموعة الوظائف المعين عليها. المادة 77 يصدر بنظام الشكاوى المتعلقة بالمخاطبين بأحكام هذا القانون، وقواعد وواجبات تعامل موظفي الوحدة مع الجمهور، قرار من رئيس الجهاز. ويكون للجهاز استئداء رسم مقداره عشرة جنيهات من المتقدم لأداء الامتحانات المشار إليها في المادة (12) من هذا القانون أو التظلم من نتائجها، على أن يزاد هذا الرسم بمقدار جنيهين كل عام ميلادي. ويُودع المبلغ المشار إليه مع المبالغ الأخرى التي يحصلها الجهاز نظير الخدمات التي يقدمها للغير في الداخل أو الخارج في حساب خاص باسم الجهاز لدى البنك المركزي بحساب الخزانة الموحد، ويصرف من الحساب المشار إليه في تطوير الجهاز، وذلك بموجب قرار يصدر من رئيس الجهاز، على أن يرحل الفائض من هذا الحساب من عام لآخر. عبد الفتاح السيسي رئيس الجمهورية</p>
</div></div></div>    </div>
'''

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Get all text from the content; using a space as separator.
content_div = soup.find("div", {"class": "ds-main col-md-9"})
text = content_div.get_text(" ", strip=True) if content_div else ""

# Define a regex pattern that looks for a header starting with "المادة" followed by a number.
# This will match "المادة" followed by whitespace, then one or more digits,
# and optionally up to 3 additional words (so that the total word count does not exceed 5).
pattern = re.compile(r'(المادة\s+\d+(?:\s+\S+){0,0})')

matches = list(pattern.finditer(text))

articles = []
for i, match in enumerate(matches):
    article_header = match.group(1).strip()  # This is our Article header (only if followed by a number)
    start_index = match.end()
    # The law text is from the end of this header until the next header or end of the text.
    if i < len(matches) - 1:
        end_index = matches[i + 1].start()
    else:
        end_index = len(text)
    law_text = text[start_index:end_index].strip()
    articles.append({
        "Article": article_header,
        "Law": law_text
    })

# Write the results to a CSV file with columns "Article" and "Law"
csv_filename = "new 2.csv"
with open(csv_filename, mode="w", encoding="utf-8", newline="") as csv_file:
    fieldnames = ["Article", "Law"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(articles)

print(f"CSV file has been saved as {csv_filename}")

CSV file has been saved as قانون التأمين الصحي الشامل رقم 2 لسنة 2018.csv


In [ ]:
# Assume your plain text is stored in the variable 'text'
text = """
قانون رقم 81 لسنة 2016 بإصدار قانون الخدمة المدنية باسم الشعب رئيس الجمهورية قرر مجلس النواب القانون الآتي نصه, وقد أصدرناه: المادة 1 يُعمل بأحكام القانون المرافق في شأن الخدمة المدنية, وتسري أحكامه على الوظائف في الوزارات ومصالحها والأجهزة الحكومية ووحدات الإدارة المحلية, والهيئات العامة وذلك ما لم تنص قوانين أو قرارات إنشائها على ما يخالف ذلك. المادة 2 يُلغى قانون نظام العاملين المدنيين بالدولة الصادر بالقانون رقم 47 لسنة 1978, كما يُلغى كل حكم يخالف أحكام القانون المرافق. المادة 3 يُصدر رئيس مجلس الوزراء بعد أخذ رأي مجلس الخدمة المدنية اللائحة التنفيذية للقانون المرافق خلال ثلاثة أشهر من تاريخ العمل به وإلى أن تصدر هذه اللائحة يستمر العمل باللوائح والقرارات القائمة حاليًا فيما لا يتعارض وأحكام القانون المرافق.
"""

# Define a regex pattern that looks for a header starting with "المادة" followed by a number.
# This regex pattern matches "المادة" followed by whitespace, then one or more digits.
pattern = re.compile(r'(مادة\s+\d+(?:\s+\S+){0,0})')

matches = list(pattern.finditer(clean_text))

articles = []
for i, match in enumerate(matches):
    article_header = match.group(1).strip()  # Article header (only if followed by a number)
    start_index = match.end()
    # The law clean_text is the clean_text from the end of this header until the next header or the end of the clean_text.
    if i < len(matches) - 1:
        end_index = matches[i + 1].start()
    else:
        end_index = len(clean_text)
    law_text = clean_text[start_index:end_index].strip()
    articles.append({
        "Article": article_header,
        "Law": law_text
    })

# Write the results to a CSV file with columns "Article" and "Law"
csv_filename = "new 2.csv"
with open(csv_filename, mode="w", encoding="utf-8", newline="") as csv_file:
    fieldnames = ["Article", "Law"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(articles)

print(f"CSV file has been saved as {csv_filename}")


CSV file has been saved as new 2.csv


In [ ]:
pattern = re.compile(r'((?:مادة\s+\d+(?:\s+مكرر(?:\s+\d+)?)?|\(المادة\s+[^\)]+\)))')

matches = list(pattern.finditer(clean_text))

articles = []
for i, match in enumerate(matches):
    article_header = match.group(1).strip()  # Extract the matched header
    start_index = match.end()
    # Define the law text as the text following this header until the next header (or end of text)
    if i < len(matches) - 1:
        end_index = matches[i + 1].start()
    else:
        end_index = len(clean_text)
    law_text = clean_text[start_index:end_index].strip()
    articles.append({
        "Article": article_header,
        "Law": law_text
    })

# Write the results to a CSV file with columns "Article" and "Law"
csv_filename = "new 3.csv"
with open(csv_filename, mode="w", encoding="utf-8", newline="") as csv_file:
    fieldnames = ["Article", "Law"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(articles)

print(f"CSV file has been saved as {csv_filename}")


CSV file has been saved as new 3.csv


In [ ]:
# Assume clean_text contains your plain text after cleaning (removing HTML tags, etc.)
# clean_text = """
# قانون رقم 81 لسنة 2016 بإصدار قانون الخدمة المدنية باسم الشعب رئيس الجمهورية قرر مجلس النواب القانون الآتي نصه, وقد أصدرناه: مادة 1 يُعمل بأحكام القانون المرافق في شأن الخدمة المدنية, وتسري أحكامه على الوظائف... (المادة الثالثة) مثال على ذلك, مادة 2 يُلغى قانون نظام العاملين المدنيين بالدولة الصادر بالقانون رقم 47 لسنة 1978, كما يُلغى كل حكم يخالف أحكام القانون المرافق. مادة 3 مكرر 2 بعض نص القانون لمزيد من الشرح.
# """

# Define the regex pattern to capture article headers that match one of these:
# 1. "\bمادة\b" followed by whitespace and a number,
#    optionally followed by the word "مكرر" and another number.
# 2. OR a header inside parentheses like "(المادة الثالثة)"
pattern = re.compile(
    r'((?:\bمادة\b\s+\d+(?:\s+مكرر(?:\s+\d+)?)?|\(المادة\s+[^\)]+\)))'
)

matches = list(pattern.finditer(clean_text))

articles = []
for i, match in enumerate(matches):
    article_header = match.group(1).strip()  # Extract the matched header
    start_index = match.end()
    # Define the law text as the text following this header until the next header or the end of the clean_text.
    if i < len(matches) - 1:
        end_index = matches[i + 1].start()
    else:
        end_index = len(clean_text)
    law_text = clean_text[start_index:end_index].strip()
    articles.append({
        "Article": article_header,
        "Law": law_text
    })

# Write the results to a CSV file with columns "Article" and "Law"
csv_filename = "قانون شركات قطاع الأعمال العام رقم 203 لسنة 1991 ولائحته التنفيذية.csv"
with open(csv_filename, mode="w", encoding="utf-8", newline="") as csv_file:
    fieldnames = ["Article", "Law"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(articles)

print(f"CSV file has been saved as {csv_filename}")


CSV file has been saved as قانون شركات قطاع الأعمال العام رقم 203 لسنة 1991 ولائحته التنفيذية.csv


In [10]:
# List the names or paths of your three CSV files
csv_files = ['قانون شركات قطاع الأعمال العام رقم 203 لسنة 1991 ولائحته التنفيذية.csv', 'new 1.csv', 'new 2.csv']

# Read each CSV file into a DataFrame and store them in a list
dataframes = [pd.read_csv(file) for file in csv_files]

# Concatenate the DataFrames vertically (row-wise)
combined_df = pd.concat(dataframes, ignore_index=True)

# Write the combined DataFrame to a new CSV file (without the index)
combined_df.to_csv('قانون شركات قطاع الأعمال العام رقم 203 لسنة 1991 ولائحته التنفيذية.csv', index=False, encoding='utf-8')

print("CSV files have been successfully combined into 'combined.csv'")


CSV files have been successfully combined into 'combined.csv'


In [ ]:
def remove_html_tags(html_text):
    """
    Remove HTML tags from a given string.
    
    Args:
        html_text (str): String containing HTML content.
    
    Returns:
        str: Cleaned text with HTML tags removed.
    """
    soup = BeautifulSoup(html_text, "html.parser")
    return soup.get_text(separator=" ", strip=True)

# Example usage:
if __name__ == "__main__":
    html_paragraph = """
<p>(المادة الأولى)</p>
<p>يعمل في شأن قطاع الأعمال العام بأحكام القانون المرافق، ويقصد بهذا القطاع الشركات القابضة والشركات التابعة لها الخاضعة لأحكام هذا القانون، وتتخذ هذه الشركات بنوعيها شكل شركات المساهمة، ويسري عليها فيما لم يرد بشأنه نص خاص في هذا القانون وبما لا يتعارض مع أحكامه نصوص قانون شركات المساهمة وشركات التوصية بالأسهم والشركات ذات المسئولية المحدودة الصادر بالقانون رقم 159 لسنة 1981.</p>
<p>لا تسري أحكام قانون هيئات القطاع العام وشركاته الصادر بالقانون رقم 97 لسنة 1983 على الشركات المشار إليها.</p>
<p> </p>
<p>(المادة الثانية)</p>
<p>تحل الشركات القابضة محل هيئات القطاع العام الخاضعة لأحكام القانون رقم 97 لسنة 1983 المشار إليه، كما تحل الشركات التابعة محل الشركات التي تشرف عليها هذه الهيئات وذلك اعتبارا من تاريخ العمل بهذا القانون ودون حاجة إلى أي إجراء آخر.</p>
<p>وتنتقل إلى الشركات القابضة والشركات التابعة لها بحسب الأحوال كافة ما لهيئات القطاع العام وشركاته الملغاة من حقوق بما فيها حقوق الانتفاع والإيجار، كما تتحمل جميع التزاماتها وتسأل مسئولية كاملة عنها.</p>
<p>وينشر النظام الأساسي لكل شركة من الشركات القابضة والتابعة على نفقتها في الوقائع المصرية وتقيد في السجل التجاري.</p>
<p> </p>
<p>(المادة الثالثة)</p>
<p>تشكل مجالس إدارة الشركات القابضة والشركات التابعة وفق أحكام القانون المرافق خلال ستة أشهر من تاريخ العمل به.</p>
<p>ويتولى رؤساء وأعضاء مجالس الإدارة الحاليون لكل من هيئات القطاع العام وشركاته بحسب الأحوال إدارة الشركات المذكورة وذلك حتى يتم تشكيل مجالس الإدارة الجديد لها.</p>
<p> </p>
<p>(المادة الرابعة)</p>
<p>ينقل العاملون بكل من هيئات القطاع العام وشركاته الموجودون بالخدمة في تاريخ العمل بهذا القانون إلى الشركات القابضة أو الشركات التابعة لها بذات أوضاعهم الوظيفية وأجورهم وبدلاتهم وأجازاتهم ومزاياهم النقدية والعينية والتعويضات.</p>
<p>وتستمر معاملة هؤلاء العاملين بجميع الأنظمة والقواعد التي تنظم شئونهم الوظيفية وذلك إلى أن تصدر لوائح أنظمة العاملين بالشركات المنقولين إليها طبقا لأحكام القانون المرافق خلال سنة من التاريخ المذكور.</p>
<p>ويحتفظ العامل المنقول بصفة شخصية بما يحصل عليه من أجور وبدلات وأجازات ومزايا نقدية وعينية وتعويضات ولو كانت تزيد على ما يستحقه طبقا لهذه اللوائح دون أن يؤثر ذلك على ما يستحقه مستقبلا من أية علاوات أو مزايا.</p>
<p> </p>
<p> </p>
<p>(المادة الخامسة)</p>
<p>مع عدم الإخلال بما ورد في شأنه نص خاص في هذا القانون أو في القانون المرافق لا يسري نظام العاملين بالقطاع العام الصادر بالقانون رقم 48 لسنة 1978 على العاملين بالشركات الخاضعة لأحكام القانون المرافق وذلك اعتبارا من تاريخ العمل باللوائح المشار إليها.</p>
<p> </p>
<p>(المادة السادسة)</p>
<p>تستمر محاكم مجلس الدولة في نظر الدعاوى والطعون الآتية التي رفعت إليها إلى أن يتم الفصل فيها بحكم بات وفقا للقواعد المعمول بها حاليا وذلك دون حاجة إلى أي إجراء أخر.</p>
<p>أولا: الدعاوى التأديبية وطعون الجزاءات التأديبية وغيرها من الدعاوى المتعلقة بالعاملين بالشركات الخاضعة لأحكام هذا القانون متى كانت قد رفعت قبل العمل باللوائح المنصوص عليها في المادة السابقة.</p>
<p>ثانيا: الدعاوى والطعون الأخرى التي تكون تلك الشركات طرفا فيها متى كانت قد رفعت قبل العمل بهذا القانون.</p>
<p> </p>
<p>(المادة السابعة)</p>
<p>لا يجوز حرمان الشركات الخاضعة لأحكام هذا القانون من أية مزايا أو تحميلها بأية أعباء تخل بالمساواة بينها وبين شركات المساهمة الخاضعة لأحكام القانون رقم 159 لسنة 1981 المشار إليه التي تعمل في ذات النشاط، ويلغى البند (1) من المادة السادسة من القانون رقم 66 لسنة 1971 بإنشاء هيئة عامة باسم "بنك ناصر الإجتماعي" كما تلغى عبارة "وفي حدود الموازنة النقدية السارية" الواردة في الفقرة الأولى من المادة (1) من القانون رقم 118 لسنة 1975 في شأن الاستيراد والتصدير.</p>
<p> </p>
<p> </p>
<p>(المادة الثامنة)</p>
<p>يصدر رئيس الجمهورية قرارا بتحديد الوزير المختص في تطبيق أحكام هذا القانون. وعليه أن يقدم إلى مجلس الوزراء تقارير دورية وفقا لما تحدده اللائحة التنفيذية عن نتائج أعمال الشركات الخاضعة لأحكام هذا القانون.</p>
<p> </p>
<p>(المادة التاسعة)</p>
<p>يجوز بقرار من رئيس الجمهورية بعد موافقة مجلس الوزراء تحويل إحدى الهيئات الإقتصادية أو المؤسسات العامة أو شركات القطاع العام المقرر لها أنظمة خاصة إلى شركة قابضة أو شركة تابعة تخضع لأحكام هذا القانون.</p>
<p> </p>
<p>(المادة العاشرة)</p>
<p>لا تخل أحكام المادة الثامنة من هذا القانون بالاختصاصات والسلطات الإدارية والتنفيذية المقررة للوزراء بمقتضى القوانين واللوائح والقرارات الجمهورية.</p>
<p> </p>
<p>(المادة الحادية عشرة)</p>
<p>يختص مجلس الدولة دون غيره بمراجعة نموذج العقد الابتدائي ونماذج النظام الأساسي للشركات الخاضعة لأحكام هذا القانون.</p>
<p>وللشركات المشار إليها أن تطلب إلى مجلس الدولة عن طريق الوزير المختص إبداء الرأي مسببا في المسائل المتعلقة بشئون العاملين فيها أو أعضاء مجالس إدارتها أو بغير ذلك من المسائل التي تتعلق بأي شأن آخر من شئونها.</p>
<p> </p>
<p> </p>
<p>(المادة الثانية عشرة)</p>
<p>يصدر رئيس مجلس الوزراء اللائحة التنفيذية للقانون المرافق خلال ثلاثة أشهر من تاريخ العمل بهذا القانون.</p>
<p>كما يصدر نموذج العقد الابتدائي ونموذج النظام الأساسي للشركات الخاضعة لأحكام هذا القانون وذلك خلال ثلاثة أشهر من تاريخ العمل باللائحة التنفيذية.</p>
<p>ويجوز أن تتعدد نماذج النظم الأساسية للشركات بحسب طبيعة أنشطتها.</p>
<p> </p>
<p>(المادة الثالثة عشرة)</p>
<p>ينشر هذا القانون في الجريدة الرسمية، ويعمل به بعد ثلاثين يوما من تاريخ نشره؛</p>
<p>يبصم هذا القانون بخاتم الدولة, وينفذ كقانون من قوانينها.</p>
<p> </p>
<p>صدر برياسة الجمهورية في 7 ذي الحجة سنة 1411 هـ</p>
<p>(19 يونية 1991م )</p>
<p> </p>
<p>حسني مبارك</p>
<p> </p>
<p> الباب الأول: الشركات القابضة</p>
<p>الفصل الأول: التأسيس</p>
<p> </p>
<p>مادة 1</p>
<p>يصدر بتأسيس الشركة القابضة قرار من رئيس مجلس الوزراء بناء على اقتراح الوزير المختص، ويكون رأسمالها مملوكا بالكامل للدولة أو للأشخاص الاعتبارية العامة، وتثبت لها الشخصية الاعتبارية من تاريخ قيدها في السجل التجاري.</p>
<p>وتأخذ الشركة القابضة شكل شركة المساهمة، وتعتبر من أشخاص القانون الخاص ويحدد القرار الصادر بتأسيسها اسمها ومركزها الرئيسي ومدتها والغرض الذي أنشئت من أجله ورأس مالها.</p>
<p>وينشر القرار الصادر بتأسيس الشركة مع نظامها الأساسي على نفقتها في الوقائع المصرية وتقيد الشركة في السجل التجاري.</p>
<p> </p>
<p>مادة 2</p>
<p>تتولى الشركة القابضة من خلال الشركات التابعة لها استثمار أموالها، كما يكون لها عند الاقتضاء أن تقوم بالاستثمار بنفسها.</p>
<p>وتتولى الشركة القابضة في مجال نشاطها ومن خلال الشركات التابعة لها المشاركة في تنمية الاقتصاد القومي في إطار السياسة العامة للدولة.</p>
<p>وللشركة أيضا في سبيل تحقيق أغراضها القيام بالأعمال الآتية:</p>
<p>1- تأسيس شركات مساهمة بمفردها أو بالاشتراك مع الأشخاص الاعتبارية العامة أو الخاصة أو الأفراد.</p>
<p>2- شراء أسهم شركات المساهمة أو بيعها أو المساهمة في رأسمالها.</p>
<p>3- تكوين وإدارة محفظة الأوراق المالية للشركة بما تتضمنه من أسهم وصكوك تمويل وسندات وأية أدوات أو أصول مالية أخرى.</p>
<p>4- إجراء جميع التصرفات التي من شأنها أن تساعد في تحقيق كل أو بعض أغراضها.</p>
<p> </p>
<p> </p>
<p> الفصل الثاني: مجلس الإدارة</p>
<p> </p>
<p>مادة 3</p>
<p> </p>
<p>يتولى إدارة الشركة القابضة مجلس إدارة يصدر بتشكيله قرار من الجمعية العامة بناء على اقتراح رئيسها لمدة ثلاث سنوات قابلة للتجديد على أن يعكس التشكيل حقوق الملكية بالشركة، ويتكون من عدد من الأعضاء يحدده النظام الأساسي لا يقل عن خمسة ولا يزيد على تسعة، ويشكل على الوجه الآتي:</p>
<p>1- رئيس غير تنفيذي لمجلس إدارة الشركة.</p>
<p>?- أعضاء يمثلون المساهمين بالشركة وفقا لهيكل ملكية الشركة بمراعاة قواعد التمثيل النسبي، منهم عضو يمثل وزارة المالية يرشحه وزير المالية، وتعينهم الجمعية العامة، دون الإخلال بحق الشخص الاعتباري المساهم في الشركة في تغيير ممثليه خلال مدة المجلس.</p>
<p>3- ممثل عن الاتحاد النقابي العمالي الأكثر تمثيلا يختاره مجلس إدارة الاتحاد بمراعاة طبيعة نشاط الشركة.</p>
<p>ويجوز أن يتضمن النظام الأساسي للشركة أعضاء مستقلين إضافيين من ذوي الخبرة بمجلس الإدارة لا يزيد عددهم على عضوين تختارهم الجمعية العامة بناء على ترشيح لوزير المختص.</p>
<p>ولا يعتبر رئيس وأعضاء مجلس الإدارة من العاملين بالشركة.</p>
<p>ويحدد القرار الصادر بتشكيل المجلس العضو المنتدب التنفيذي وغيره من الأعضاء المنتدبين للإدارة.</p>
<p>وتحدد الجمعية العامة سنويا من يتقاضاه كل من رئيس وأعضاء المجلس المشار إليهم من بدلات الحضور والانتقال للجلسات، وكذا ما يتقاضاه رئيس المجلس والأعضاء المنتدبون من رواتب والمكافأة السنوية التي يستحقها مجلس الإدارة بمراعاة نص المادة (24) من هذا القانون، كما تحدد الجمعية العامة المزايا الأخرى للأعضاء المنتدبين للإدارة بما في ذلك التأمين الطبي ووسائل الانتقال.</p>
<p>وفي جميع الأحوال، لا يجوز أن يزيد بدل الحضور والانتقال لرئيس وأعضاء المجلس على الحد الذي يصدر به قرار من رئيس مجلس الوزراء.</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 4</p>
<p>لا يجوز أن يكون رئيسا أو عضوا بمجلس إدارة الشركة من حكم عليه بعقوبة جناية، أو بعقوبة مقيدة للحرية في جريمة مخلة بالشرف أو الأمانة أو تفالس أو بعقوبة من العقوبات المنصوص عليها في المواد 49 و50 و51 من هذا القانون.</p>
<p> </p>
<p>مادة 5</p>
<p>يكون الجمعية العامة العادية للشركة عند مناقشتها لنتائج أعمال الشركة السنوية تغيير رئيس وأعضاء مجلس إدارة الشركة كلهم أو بعضهم أثناء مدة العضوية، وفي حالة تغيير المجلس بأكمله يجوز للجمعية العامة تعيين مفوض لإدارة الشركة بصفة مؤقتة ولمدة لا تجاوز ثلاثة أشهر لحين تشكيل مجلس إدارة جديد طبقا لأحكام هذا القانون.</p>
<p>ويسري حكم هذه الفقرة على الشركات التابعة الخاضعة لأحكلم هذا القانون.</p>
<p>كما لا يجوز تجديد تعيين رئيس وأعضاء مجلس إدارة الشركة إذا لم تحقق الشركة الأهداف المحددة لها خلال مدة العضوية.</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 6</p>
<p>لمجلس إدارة الشركة مباشرة كل السلطات اللازمة لتصريف أمور الشركة والقيام بكافة الأعمال اللازمة لتحقيق الغرض الذي أنشئت من أجله، وذلك فيما عدا ما تختص به الجمعية العامة للشركة، ولمجلس الإدارة في سبيل ذلك على الأخص ما يأتي:</p>
<p>1- وضع السياسات العامة وتحديد الوسائل اللازمة لتحقيقها.</p>
<p>2- إدارة محفظة الأوراق المالية للشركة بيعا وشراء بما تتضمنه من أسهم وصكوك تمويل وسندات وأية أدوات وأصول مالية أخرى.</p>
<p>3- اقتراح تأسيس شركات مساهمة تؤسسها الشركة بمفردها أو بالاشتراك مع الأشخاص الاعتبارية العامة أو الخاصة أو الأفراد.</p>
<p>4- شراء أسهم الشركات المساهمة أو بيعها أو المساهمة في رأس مالها.</p>
<p>5- القيام بكافة الأعمال اللازمة لتصحيح الهياكل التمويلية ومسار الشركات المتعثرة التابعة لها وتعظيم ربحية هذه الشركات، وترشيد التكلفة.</p>
<p>6- إقرار مشروع الميزانية والحسابات الختامية تمهيدا لعرضها على الجمعية العامة للشركة.</p>
<p>7- وضع معايير الأداء وتقييمها وفحص التقارير التي تقدم عن سير العمل بالشركة.</p>
<p>8- اعتماد الهيكل التنظيمي للشركة ووضع اللوائح الداخلية المتعلقة بالنواحي المالية والإدارية والفنية وغيرها.</p>
<p>9- ما يرى رئيس الجمعية العامة أو رئيس مجلس الإدارة عرضه على المجلس.</p>
<p> </p>
<p>مادة 6 مكرر</p>
<p>يضع مجلس إدارة الشركة القابضة الضوابط الواجب الالتزام بها لقيام الشركة القابضة أو شركاتها التابعة بتأسيس أو المشاركة في تأسيس الشركات الأخرى، ويتم اعتمادها من الوزير المختص، على أن تتضمن هذه الضوابط على الأخص ما يأتي:</p>
<p>1- وجود جدوي اقتصادية من إنشاء الشركة الجديدة، أو المساهمة فيها.</p>
<p>2- عدم وجود شركة تابعة أخرى يمكن لها القيام بالنشاط المطلوب تأسيس شركة لممارسته.</p>
<p>3- أن تكون الجمعية العامة للشركة أو مجلس إدارتها هو السلطة المختص بالموافقة على إنشاء الشركة الجديدة أو المساهمة فيها، وفقا لما يحدده النظام الأساسي للشركة.</p>
<p>وعلى الشركة القابضة وضع نظام لمتابعة أداء الشركات التي تساهم فيها الشركة القابضة وشركاتها التابعة يتضمن على الأخص لتقارير الدورية لمتابعة أداء هذه الشركات والرقابة عليها، وقواعد اختيار ممثلي الشركة القابضة أو التابعة في عضوية مجالس إدارات هذه الشركات، والتأكد من اتباع الإجراءات الواجبة عند التخارج من هذه الشركات، وذلك كله وفقا للقواعد التي يضعها الوزير المختص</p>
<p>.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p>مادة 7</p>
<p>يجتمع مجلس الإدارة مرة على الأقل كل شهر بدعوة من رئيسه، وفي حالة غيابه يندب رئيس الجمعية العامة من بين أعضاء المجلس من يرأس الاجتماع.</p>
<p>ولا يكون انعقاد المجلس صحيحا إلا بحضور أغلبية الأعضاء، وتصدر قراراته بأغلبية أصوات الحاضرين، وعند التساوي يرجح الجانب الذي منه الرئيس. وللمجلس أن يدعو لحضور اجتماعاته من يرى الاستعانة بهم من ذوي الخبرة من العاملين بالشركة أو غيرهم دون أن يكون لهم صوت معدود فيما يتخذه المجلس من قرارات.</p>
<p>ويجوز للمجلس أن يشكل من بين أعضائه لجنة أو لجانا يعهد إليها ببعض اختصاصاته؛ كما يجوز له أن يعهد إلى رئيس مجلس الإدارة أو أحد المديرين ببعض اختصاصاته، وللمجلس أن يفوض أحد أعضائه أو أحد المديرين في القيام بمهمة محددة على أن يعرض على المجلس تقريرا بما قام به من أعمال.</p>
<p> </p>
<p>مادة 8</p>
<p>يمثل الشركة أمام القضاء وفي صلاتها بالغير رئيس مجلس الإدارة أو العضو المنتدب التنفيذي بحسب النظام الأساسي للشركة، ويختص العضو المنتدب التنفيذي بما يأتي:</p>
<p>1- تنفيذ قرارات مجلس الإدارة.</p>
<p>2- إدارة الشركة وتصريف شئونها.</p>
<p>وله أن يفوض واحدا أو أكثر من أعضاء مجلس الإدارة في بعض اختصاصاته.</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>الفصل الثالث: الجمعية العامة</p>
<p>مادة 9</p>
<p>تتكون الجمعية العامة للشركة على النحو الآتي:</p>
<p>1- الوزير المختص رئيسا.</p>
<p>2- أعضاء من ذوي الخبرة في مجال الأنشطة التي تقوم بها الشركة القابضة وشركاتها التابعة لا يقل عددهم عن اثني عشر ولا يزيد على أربعة عشرة من بينهم ممثل واحد على الأقل يرشحه الاتحاد النقابي العمالي الأكثر تمثيلا وممثل عن وزارة المالية يرشحه وزير المالية يصدر باختيارهم قرار من رئيس مجلس الوزراء، ويحدد القرار ما يتقاضونه من بدل الحضور والانتقال وفقا للقواعد التي تحددها اللائحة التنفيذية لهذا القانون.</p>
<p>ويحضر اجتماعات الجمعية العامة رئيس وأعضاء مجلس إدارة الشركة، ومراقبو الحسابات بالجهاز المركزي للمحاسبات، ومراقب الحسابات المعين من الجمعية العامة للشركة إن وجد، دون أن يكون لهم صوت معدود. ويجوز للوزير المختص تفويض غيره في حضور الجمعية العامة.</p>
<p>وتصدر قرارات الجمعية العامة بأغلبية أصوات الحاضرين بالاجتماع فيما عدا الأحوال التي تتطلب فيها اللائحة التنفيذية لهذا القانون أو النظام الأساسي للشركة أغلبية خاصة.</p>
<p>وتبين اللائحة التنفيذية لهذا القانون شروط صحة انعقاد الجمعية العامة ونظام التصويت على المسائل المعروضة عليها بحسب ما إذا كانت الجمعية العامة منعقدة في اجتماع عادي أو غير عادي</p>
<p>.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 10</p>
<p>مع مراعاة أحكام هذا القانون ولائحته التنفيذية والنظام الأساسي للشركة تختص الجمعية العامة العادية بما يأتي:</p>
<p>(‌أ) التصديق على تقرير مجلس الإدارة عن نشاط الشركة والنظر في إخلاء المجلس من المسئولية عن الفترة المقدم عنها التقرير.</p>
<p>(‌ب) التصديق على الميزانية والحسابات الختامية للشركة.</p>
<p>(‌ج) الموافقة على استمرار رئيس وأعضاء مجلس الإدارة لمدة تالية أو عزلهم، ويكون التصويت على ذلك بطريق الاقتراع السري.</p>
<p>(‌د) الموافقة على توزيع الأرباح.</p>
<p>(‌هـ) كل ما يرى رئيس الجمعية العامة أو مجلس الإدارة عرضه عليها.</p>
<p>ولا يجوز التصرف بالبيع في أصل من خطوط الإنتاج الرئيسية إلا بعد موافقة الجمعية العامة وطبقا للقواعد التي تحددها اللائحة التنفيذية.</p>
<p>مادة 11</p>
<p>لا يجوز تعديل نظام الشركة إلا بموافقة الجمعية العامة غير العادية وفقا لأحكام اللائحة التنفيذية.</p>
<p> </p>
<p>الفصل الرابع: النظام المالي للشركة ومراقبة حساباتها</p>
<p>مادة 12</p>
<p>يحدد النظام الأساسي بداية ونهاية السنة المالية للشركة.</p>
<p>وتعتبر أموال الشركة من الأموال المملوكة للدولة ملكية خاصة، وتودع الشركة مواردها بالنقد المحلي والأجنبي في حساب مصرفي بالبنك المركزي أو أحد البنوك التجارية.</p>
<p> </p>
<p>مادة 13</p>
<p>تعد الشركة القابضة قوائم مالية مجمعة تعرض أصول والتزامات وحقوق المساهمين وإيرادات ومصروفات واستخدامات الشركة والشركات التابعة لها وفقا للأوضاع والشروط والبيانات التي تحددها اللائحة التنفيذية.</p>
<p> </p>
<p>مادة 14</p>
<p>تحدد الأرباح الصافية للشركة، ويتم توزيعها بقرار من الجمعية العامة طبقا لأحكام هذا القانون ولائحته التنفيذية.</p>
<p>ويؤول نصيب الدولة في هذه الأرباح إلى الخزانة العامة.</p>
<p>ولوزير المالية أو من يفوضه الخصم من خلال البنك المركزي المصري بمستحقات الدولة في أرباح الشركات القابضة طبقا لحساب التوزيع المصادق عليه من الجمعية العامة على حسابات هذه الشركات بالجهاز المصرفي، وذلك في حالة تخلفها عن السداد في موعد غايته ستة أشهر من قرار الجمعية العامة بتوزيع الأرباح.</p>
<p>التعديل الأخير بموجب القانون 149 لسنة 2001</p>
<p> </p>
<p>مادة 15</p>
<p>يتولى الجهاز المركزي للمحاسبات مراقبة حسابات الشركة وتقييم أدائها طبقا لقانونه.</p>
<p>ويجوز للجمعية العامة للشركة تعيين مراقب حسابات آخر للشركة، بالإضافة إلى مراقب الجهاز المركزي للمحاسبات من مراقبي الحسابات المقيدين بسجلات الهيئة العامة للرقابة المالية، على أن تحدد الجمعية العامة أتعابه</p>
<p>.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p> الباب الثاني: الشركات التابعة للشركات القابضة</p>
<p> الفصل الأول: التأسيس</p>
<p>مادة 16</p>
<p> </p>
<p>مع عدم الإخلال بحكم المادة (39 مكررا) من هذا القانون، تعتبر شركة تابعة في تطبيق أحكام هذا القانون الشركة التي يكون لإحدى الشركات القابضة أكثر من (50%) من رأسها أو من حقوق التصويت بها.</p>
<p>فإذا اشترك في هذه النسبة أكثر من شركة من الشركات القابضة أو الأشخاص الاعتبارية العامة أوالبنوك المملوكة أسهمها بالكامل للدولة يصدر رئيس مجلس الوزراء قرارا بتحديد الشركة القابضة التي تتبعها هذه الشركة.</p>
<p>وتتخذ الشركة التابعة شكل شركة المساهمة وتثبت لها الشخصية الاعتبارية من تاريخ قيدها في السجل التجاري</p>
<p>.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 17</p>
<p>يصدر بتأسيس الشركة التابعة قرار من الوزير المختص بناء على اقتراح مجلس إدارة الشركة القابضة، وينشر هذا القرار مرفقا به النظام الأساسي على نفقة الشركة في الوقائع المصرية وتقيد الشركة في السجل التجاري.</p>
<p> </p>
<p> </p>
<p> </p>
<p> الفصل الثاني: رأس مال الشركة وأسهمها</p>
<p>مادة 18</p>
<p>يقسم رأسمال الشركة إلى أسهم اسمية متساوية القيمة.</p>
<p>ويحدد النظام الأساسي القيمة الاسمية للسهم بما لا يقل عن الحد الأدنى وفقا لقانون سوق رأس المال الصادر بالقانون رقم 95 لسنة 1992</p>
<p>ويكون السهم غير قليل للتجزئة، ولا يجوز إصداره بأقل من قيمته الاسمية، كما لا يجوز إصداره بقيمة أعلى إلا في الأحوال وبالشروط المبينة في قانون سوق رأس المال المشار إليه ولائحته التنفيذية، على أن تضاف هذه الزيادة إلى الاحتياطي.</p>
<p>ولا يجوز بأي حال أن تجاوز مصاريف الإصدار الحد الذي يصدر به قرار من الهيئة العامة للرقابة المالية.</p>
<p>ويتم إيداع أسهم الشركة لدى إحدى الشركات أو الجهات المرخص لها بنشاط الإيداع والقيد المركزي للأوراق والأدوات المالية.</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 19</p>
<p> </p>
<p>تشكل بقرار من الوزير المختص لجنة أو أكثر برئاسة أحد أعضاء الجهات أو الهيئات القضائية بدرجة مستشار على الأقل تختاره جهة عمله، وعضوية ممثل عن كل من وزارة المالية، والجهاز المركزي للمحاسبات، وممثل عن المؤسسين أو المساهمين يختاره مجلس إدارة الشركة القابضة أو التابعة بحسب الأحوال، وأربعة على الأكثر من ذوي الخبرة الاقتصادية والمحاسبية والقانونية والفنية يختارهم الوزير المختص.</p>
<p>وتتولى اللجنة المشار إليها بالفقرة السابقة التحقق من صحة تقييم الأصول الآتية:</p>
<p>1- الحصص العينية المادية والمعنوية الداخلة في رأسمال الشركات الخاضعة لأحكام هذا القانون عند التأسيس أو الاندماج أو عند زيادة رأس المال.</p>
<p>2- الأسهم والحصص التي تملكها الدولة في الشركات القابضة.</p>
<p>3- الأسهم والحصص التي تمتلكها الشركة القابضة في شركاتها التابعة أو تمتلكها في غيرها من الشركات الأخرى قبل التصرف فيها.</p>
<p>4- أسهم الشركات الخاضعة لأحكام هذا القانون في حالات مبادلتها بأسهم في شركات أخرى.</p>
<p>5- الأصول العقارية غير المستغلة التي تقرر الشركة التصرف فيها.</p>
<p>وعلى اللجنة تقديم تقريرها إلى الوزير المختص أو الشركة بحسب الأحوال في مدة أقصاها ثلاثون يوما من تاريخ إحالة الأوراق إليها، وتحدد اللائحة التنفيذية لهذا القانون القواعد والإجراءات الواجب الالتزام بها عند القيام بأصل التقييم واعتماده بمراعاة معايير التقييم المالي للمنشآت والمعايير المصرية للتقييم العقاري</p>
<p>.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 20</p>
<p> </p>
<p>تكون أسهم الشركة قابلة التداول فور إصدارها، ويكون تداول أو نقل ملكية أسهم الشركات الخاضعة لهذا القانون بمراعاة أحكام هذا القانون ولائحته التنفيذية والقواعد المطبقة بالبورصات المصرية.</p>
<p>ويجوز تداول حصص التأسيس والأسهم التي تعطى مقابل حصة عينية والأسهم التي يكتتب فيها مؤسسو الشركة من تاريخ قيدها في السجل التجاري.</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>الفصل الثالث: مجلس الإدارة</p>
<p>مادة 21</p>
<p> </p>
<p>مع مراعاة حكم المادة (4) من هذا القانون، يتولى إدارة الشركة التابعة مجلس إدارة تختاره الجمعية العامة لمدة ثلاث سنوات قابلة للتجديد.</p>
<p>ويجتمع مجلس الإدارة مرة على الأقل كل شهر بدعوة من رئيسه، وفي حالة غيابه يندب رئيس الجمعية العامة من بين أعضاء المجلس من يرأس الاجتماع.</p>
<p>ويتكون مجلس الإدارة من عدد من الأعضاء يحدده النظام الأساسي لا يقل عن خمسة ولا يزيد على تسعة بمن فيهم رئيس المجلس على النحو الآتي:</p>
<p>(أ) رئيس غير تنفيذي، تختاره الجمعية العامة للشركة بناء على ترشيح مجلس إدارة الشركة القابضة.</p>
<p>(ب) أعضاء يمثلون المساهمين بالشركة وفقا لهيكل ملكية الشركة بمراعاة قواعد التمثيل النسبي، تعينهم الجمعية العامة مع عدم الإخلال بحق الشخص الاعتباري المساهم في الشركة في تغيير ممثليه خلال مدة المجلس.</p>
<p>(جـ) ممثل أو اثنان من العاملين بالشركة بحسب عدد أعضاء مجلس الإدارة، يتم انتخابه أو انتخابهما طبقا لأحكام القانون المنظم لذلك.</p>
<p>ويجوز أن يتضمن النظام الأساسي للشركة أعضاء مستقلين إضافيين من ذوي الخبرة بمجلس الإدارة لا يزيد عددهم على عضوين تختارها الجمعية العامة بناء على ترشيح مجلس إدارة الشركة القابضة.</p>
<p>وتحدد الجمعية العامة سنويا ما يتقاضاه كل من رئيس وأعضاء المجلس المشار إليهم من بدلات الحضور والانتقال للجلسات والمكافأة السنوية التي يستحقها مجلس الإدارة بمراعاة نص المادة (34) من هذا القانون، كما تحدد الجمعية العامة المزايا الأخرى للأعضاء المنتمين بما في ذلك التأمين الطبي ووسائل الانتقال.</p>
<p>وفي جميع الأحوال، لا يجوز أن يزيد بدل الحضور والانتقال لرئيس وأعضاء المجلس عن الحد الذي يصدر به قرار من رئيس مجلس الوزراء.</p>
<p>ويختار مجلس إدارة الشركة من بين أعضاء المجلس العضو المنتدب التنفيذي وغيره من الأعضاء المنتدبين للإدارة ويحدد ما يتقاضونه من راتب بالإضافة إلى ما يستحقونه من مبالغ طبقا للفقرة الخامسة من هذه المادة.</p>
<p>ويحدد المجلس من يحل محل العضو المنتدب في حالة غيابه أو خلو منصبه أو تغييره</p>
<p>.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 22</p>
<p>ملغاة.</p>
<p>.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 23</p>
<p>لعضو مجلس الإدارة المنتدب جميع السلطات المتعلقة بإدارة الشركة والقيام بكافة الأعمال اللازمة لتحقيق غرضها، وذلك فيما عدا ما يدخل في اختصاص الجمعية العامة ومجلس الإدارة طبقا لأحكام هذا القانون ولائحته التنفيذية ونظام الشركة.</p>
<p> </p>
<p>مادة 24</p>
<p>يمثل عضو مجلس الإدارة المنتدب الشركة أمام القضاء وفي صلاتها بالغير.</p>
<p> </p>
<p>الفصل الرابع: الجمعية العامة</p>
<p>مادة 25</p>
<p> </p>
<p>تتكون الجمعية العامة للشركة على النحو الآتي:</p>
<p>1- رئيس مجلس إدارة الشركة القابضة أو من يحل محله في حالة غيابه، رئيسا.</p>
<p>2- ممثلون للشركة القابضة والأشخاص الاعتبارية العامة المساهمة في الشركة تختارهم السلطة المختصة بكل منها.</p>
<p>3- المساهمون من الأشخاص الاعتبارية الخاصية أو الأفراد في رأسمال الشركة.</p>
<p>4- عضو تختاره اللجنة النقابية الشركة.</p>
<p>ويجوز أن يضم لتشكيل الجمعية العامة بالشركات التابعة التي تمتلكها الشركات القابضة بمفردها أو مع غيرها من الأشخاص الاعتبارية العامة أو البنوك المملوكة أسهمها بالكامل للدولة أربعة أعضاء على الأكثر من ذوي الخبرة تختارهم الجمعية العامة للشركة القابضة، وتحدد ما يتقاضونه من بدل الحضور والانتقال.</p>
<p>ويحضر اجتماعات الجمعية العامة رئيس وأعضاء مجلس إدارة الشركة، ومراقبو الحسابات من الجهاز المركزي للمحاسبات، ومراقب الحسابات المعين من الجمعية العامة للشركة إن وجد، دون أن يكون لهم صوت معدود.</p>
<p>ويكون لكل مساهم التصويت في الجمعية العامة بنسبة ما يمتلكه من أسهم في رأسمال الشركة أو حقوق تصويت فيها، وتصدر قرارات الجمعية العامة بأغلبية الأسهم الحاضرة بالاجتماع فيما عدا الأحوال التي تتطلب فيها اللائحة التنفيذية أو النظام الأساسي للشركة أغلبية خاصة.</p>
<p>وتبين اللائحة التنفيذية لهذا القانون شروط صحة انعقاد الجمعية العامة ونظام التصويت على المسائل المعروضة عليها بحسب ما إذا كانت الجمعية العامة منعقدة في اجتماع عادي أو غير عادي.</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p>.</p>
<p> </p>
<p>مادة 26</p>
<p>ملغاة.</p>
<p> </p>
<p>مادة 27</p>
<p>مع مراعاة أحكام هذا القانون ولائحته التنفيذية والنظام الأساسي تختص الجمعية العامة العادية بما يأتي:-</p>
<p>(‌أ) التصديق على الميزانية وحساب الأرباح والخسائر.</p>
<p>(‌ب) التصديق على تقرير مجلس الإدارة عن نشاط الشركة والنظر في إخلائه من المسئولية.</p>
<p>(‌ج) الموافقة على توزيع الأرباح.</p>
<p>(‌د) الموافقة على استمرار رئيس وأعضاء مجلس الإدارة لمدة تالية أو عزلهم ويكون التصويت على ذلك بطريق الاقتراع السري.</p>
<p>(‌هـ) كل ما يرى رئيس الجمعية العامة للشركة أو رئيس مجلس إدارة الشركة القابضة أو مجلس إدارة الشركة التابعة لها أو المساهمون من الأشخاص الاعتبارية من القطاع الخاص أو الأفراد الذين يملكون 10% من رأس المال عرضه على الجمعية العامة.</p>
<p> </p>
<p>مادة 28</p>
<p>لا يجوز تعديل النظام الأساسي للشركة إلا بموافقة الجمعية العامة غير العادية ووفقا لأحكام اللائحة التنفيذية.</p>
<p> </p>
<p>مادة 29</p>
<p>ملغاة.</p>
<p>.التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 30</p>
<p>مع عدم الإخلال بأحكام هذا القانون تسري على الجمعيات العامة للشركات التي يساهم فيها مع الشركة القابضة أشخاص اعتبارية من القطاع الخاص أو الأفراد أحكام المواد 59 إلى 76 من القانون رقم 159 لسنة 1981. المشار إليه.</p>
<p> </p>
<p> </p>
<p> </p>
<p> </p>
<p> </p>
<p> </p>
<p> </p>
<p> </p>
<p>مادة 30 مكرر</p>
<p> </p>
<p>لرئيس مجلس الوزراء بعد موافقة مجلس الوزراء نقل ملكية أسهم أي من الشركات التابعة المملوكة أسهمها بالكامل لإحدى الشركات القابضة الخاضعة لأحكام هذا القانون إلى أي من الأشخاص الاعتبارية العامة أو صندوق مصر السيادي للاستثمار والتنمية، على أن تخضع الشركات المنقولة ملكيتها لأحكام قانون شركات المساهمة وشركات التوصية بالأسهم والشركات ذات المسئولية المحدودة وشركات الشخص الواحد الصادر بالقانون رقم 159 لسنة 1981</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>الفصل الخامس: النظام المالي للشركة ومراقبة حساباتها</p>
<p>مادة 31</p>
<p>يحدد النظام الأساسي بداية ونهاية السنة المالية للشركة بمراعاة السنة المالية للشركة القابضة التي تتبعها.</p>
<p> </p>
<p>مادة 32</p>
<p> </p>
<p>الأرباح الصافية هي الأرباح الناتجة عن العمليات التي باشرتها الشركة وذلك بعد خصم جميع التكاليف اللازمة لتحقيق هذه الأرباح وبعد حساب وتجنيب كافة الإهلاكات والمخصصات التي تقضي الأصول المحاسبية بحسابها وتجنيبها قبل إجراء أي توزيع بأية صورة من الصور للأرباح.</p>
<p>ويجنب مجلس الإدارة من صافي الأرباح المشار إليها في الفقرة السابقة جزءا من عشرين على الأقل لتكوين احتياطي قانوني، ويجوز للجمعية العامة للشركة وقف تجنيب هذا الاحتياطي أو تخفيض نسبته إذا بلغ ما يساوي نصف رأس المال.</p>
<p>ويجوز استخدام الاحتياطي القانوني في تغطية خسائر الشركة وفي زيادة رأس المال. كما يجوز أن ينص في نظام الشركة على تجنيب نسبة معينة من الأرباح الصافية لتكوين احتياطي نظامي.</p>
<p>وإذا لم يكن الاحتياطي النظامي مخصصا لأغراض معينة منصوص عليها في نظام الشركة جاز للجمعية العامة بناء على اقتراح مجلس الإدارة أن تقرر استخدامه فيما يعود بالنفع على الشركة، أو على المساهمين.</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 33</p>
<p>يكون العاملين بالشركة نصيب في الأرباح السنوية القابلة للتوزيع، يصدر به قرار من الجمعية العامة بناء على عرض مجلس الإدارة بنسبة لا تقل عن (10%) ولا تزيد على (12%) من هذه الأرباح تصرف نقدا، بمراعاة أحكام قانون تنظيم استخدام وسائل الدفع غير النقدي الصادر بالقانون رقم 18 لسنة 2019</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 34</p>
<p> </p>
<p>يبين النظام الأساسي للشركة كيفية تحديد وتوزيع مكافأة أعضاء مجلس الإدارة، ولا يجوز تقدير مكافأة مجلس الإدارة بأكثر من (5%) بالنسبة للشركات القابضة و(10%) بالنسبة للشركات التابعة وذلك من أرباح الشركة السنوية القابلة للتوزيع مخصوصا منها نسبة (5%) من رأس المال المدفوع.</p>
<p>ويكون للجمعية العامة تقرير ما تراه مناسبا في شأن توزيع باقي أرباح الشركة بعد خصم حصة العاملين ومجلس الإدارة على المساهمين من عدمه وذلك في ضوء التزامات الشركة ومشروعاتها</p>
<p>.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p> </p>
<p> </p>
<p>مادة 35</p>
<p>يتولى الجهاز المركزي للمحاسبات مراقبة حسابات الشركة وتقييم أدائها طبقا لقانونه.</p>
<p>ويجوز للجمعية العامة للشركة تعيين مراقب حسابات آخر للشركة، بالإضافة إلى مراقب الجهاز المركزي للمحاسبات من مراقبي الحسابات المقيدين بسجلات الهيئة العامة للرقابة المالية، على أن تحدد الجمعية العامة أتعابه.</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>الباب الثالث: الأحكام العامة</p>
<p>الفصل الأول: الإفصاح وقواعد الحوكمة والإدارة الرشيدة</p>
<p>مادة 35 مكرر</p>
<p> </p>
<p>تلتزم الشركات الخاضعة لأحكام هذا القانون وغير المقيدة بالبورصات المصرية بالآتي:</p>
<p>1- نشر تقارير دورية نصف سنوية عن أدائها ونتائج أعمالها، متضمنة أداء ونتائج أعمال الشركات التي تساهم فيها، وأي أحداث جوهرية مرتبطة بهذه الشركة والشركات التي تساهم فيها ومجالس إدارتها خلال هذه الفترة.</p>
<p>2- نشر تقرير مجلس الإدارة السنوي المعروض على الجمعية العامة، وكذا القوائم المالية السنوية والإيضاحات المتممة لها، وتقرير مراقبي الحسابات في شأنها.</p>
<p>3- نشر القرارات الصادرة عن الجمعية العامة العادية وغير العادية للشركة.</p>
<p>4- البيانات والمعلومات الأخرى التي تحددها اللائحة التنفيذية لهذا القانون.</p>
<p>وتحدد اللائحة التنفيذية لهذا القانون وسائل ومواعيد الشر والقواعد الواجب اتباعها في شأن إعداد هذه التقارير</p>
<p>.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 35 مكرر 1</p>
<p> </p>
<p>تلتزم الشركات الخاضعة لأحكام هذا القانون بقواعد الحوكمة والإدارة الرشيدة التي يصدر بتحديدها قرار من رئيس مجلس الوزراء.</p>
<p>وعلى مجلس إدارة الشركة إعداد تقرير سنوي عن مدى تطبيق قواعد الحوكمة الإدارة الرشيدة بها وخطتها للامتثال لباقي متطلباتها، وذلك للعرض على الجمعية العامة.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>الفصل الثاني: اندماج وتقسيم وانقضاء وتصفية الشركات القابضة والشركات التابعة لها</p>
<p>مادة 36</p>
<p>يجوز بقرار من رئيس مجلس الوزراء تقسيم وإدماج الشركات القابضة بناء على عرض الوزير المختص، كما يجوز تقسيم وإدماج الشركات التابعة لها وذلك بقرار من مجلس إدارة الشركة أو الشركات القابضة واعتماد الجمعيات العامة للشركات المندمجة والمندمج فيها أو المقسمة حسب الأحوال.</p>
<p>ويكون لكل شركة نشأت من الاندماج أو التقسيم الشخصية الاعتبارية المستقلة مع ما يترتب على ذلك من آثار قانونية.</p>
<p>ومع مراعاة أحكام هذا القانون ولائحته التنفيذية تسري على حالات الاندماج أحكام المواد من 130 إلى 135 من القانون رقم 159 لسنة 1981 المشار إليه.</p>
<p> </p>
<p>مادة 37</p>
<p>تتولى تقدير صافي أصول الشركات في حالات الاندماج والتقسيم اللجنة المنصوص عليها في المادة (19) من هذا القانون، ويجب أن تعتمد قرارات اللجنة بالنسبة للشركات القابضة من الوزير المختص، وبالنسبة للشركات التابعة من الجمعية العامة للشركات المندمجة والشركة المندمج فيها أو الشركة المقسمة بحسب الأحوال.</p>
<p> </p>
<p> </p>
<p> </p>
<p>مادة 38</p>
<p>إذا بلغت خسائر الشركة نصف رأس المال المصدر وجب على مجلس الإدارة أن يبادر إلى دعوة الجمعية العامة غير العادية للنظر في حل الشركة أو استمرارها.</p>
<p>وفي جميع الأحوال إذا بلغت قيمة خسائر الشركة كامل حقوق المساهمين بالشركة يتم عرض الأمر على الجمعية العامة للشركة لزيادة رأسمالها لتغطية الخسائر المرحلة، وفي حال عدم زيادة رأسمال الشركة وفقا لما سبق وجب العرض على الجمعية العامة غير العادية لحل وتصفية الشركة أو دمجها في شركة أخرى مع الحفاظ على حقوق العاملين بها بما لا يقل عما تضمنه قانون العمل المشار إليه، ودون الإخلال بأحكام القانون رقم 125 لسنة 2010 بشان مرتبة امتياز حقوق العمال، وذلك كله وفقا للقواعد التي تحددها اللائحة التنفيذية لهذا القانون.</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 39</p>
<p> </p>
<p>تنقضي الشركة بأحد الأسباب الآتية:-</p>
<p>1- حل الشركة.</p>
<p>2- انتهاء المدة المحددة في نظام الشركة.</p>
<p>3- انتهاء الغرض الذي أسست الشركة من أجله.</p>
<p>4- الاندماج أو التقسيم.</p>
<p>وتكون الشركة المنقضية في حالة تصفية، وتطبق عليها أحكام المواد من 137 إلى 154 من القانون رقم 159 لسنة 1981 المشار إليه ولائحته التنفيذية.</p>
<p> </p>
<p>مادة 39 مكرر</p>
<p>في الأحوال التي يترتب فيها على طرح أسهم الشركة التابعة بإحدى البورصات المصرية وصول نسبة المساهمين بخلاف الدولة والشركات القابضة الخاضعة لهذا القانون والأشخاص الاعتبارية العامة وبنوك القطاع العام في ملكية الشركة التابعة إلى (25%) أو أكثر في رأسمال الشركة، يتم نقل تبعية الشركة لأحكام القانون رقم 159 لسنة 1981 وفقا للإجراءات وخلال المدة التي تحددها اللائحة التنفيذية لهذا القانون</p>
<p>.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>الفصل الثالث: التحكيم</p>
<p> </p>
<p>مادة 40</p>
<p>يجوز الاتفاق على التحكيم في المنازعات التي تقع فيما بين الشركات الخاضعة لأحكام هذا القانون أو بينها وبين الأشخاص الاعتبارية العامة أو الأشخاص الاعتبارية من القطاع الخاص أو الأفراد وطنيين كانوا أو أجانب وتطبق في هذا الشأن أحكام الباب الثالث من الكتاب الثالث من قانون المرافعات المدنية والتجارية.</p>
<p> </p>
<p>مادة 41</p>
<p>طلبات التحكيم بين شركات القطاع العام أو بينها وبين جهة حكومية مركزية أو محلية أو هيئة عامة أو هيئة قطاع عام أو مؤسسة عامة التي قدمت قبل تاريخ العمل بهذا القانون وكذلك منازعات التنفيذ الوقتية في الأحكام الصادرة فيها يستمر نظرها أمام هيئات التحكيم المشكلة طبقا لأحكام قانون هيئات القطاع العام وشركاته الصادر بالقانون رقم 97 لسنة 1983 وطبقا للأحكام والإجراءات المنصوص عليها فيه.</p>
<p> </p>
<p> </p>
<p> </p>
<p> </p>
<p> </p>
<p> </p>
<p> </p>
<p> </p>
<p>الفصل الرابع: في نظام العاملين في الشركات القابضة والشركات التابعة لها</p>
<p> </p>
<p>مادة 42</p>
<p>يضع مجلس إدارة الشركة بعد أخذ رأي النقابة العامة المختصة اللوائح المتعلقة بنظام العاملين بها.</p>
<p>وتتضمن هذه اللوائح على الأخص نظام الأجور والعلاوات والبدلات والإجازات طبقا لتنظيم الخاص بكل شركة، وتعتمد هذه اللوائح أو أي تعديل بها من الجمعية العامة للشركة، ويصدر بها قرار من الوزير المختص.</p>
<p>كما يضع مجلس إدارة الشركة لائحة العمل بالإدارة القانونية بها، متضمنة على الأخص نظام عملها وتحديد اختصاصات أعضائها وتأديبهم، بما يكفل لهم الحيدة والاستقلالية في مباشرة أعمالهم، وتعتمد هذه اللائحة وأي تعديل لها بقرار من الوزير المختص</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020.</p>
<p> </p>
<p>مادة 43</p>
<p>يراعى في وضع اللوائح المنظمة لشئون العاملين ما يأتي:-</p>
<p>أولا- أن يكون لكل شركة هيكل تنظيمي وجدول للوظائف بما يتفق مع طبيعة أنشطة الشركة وأهدافها.</p>
<p>ثانيا- التزام نظام الأجور بالحد الأدنى المقرر قانونا.</p>
<p>ثالثا- ربط الأجر ونظام الحوافز والبدلات والمكافآت وسائر التعويضات والمزايا المالية للعاملين في ضوء ما تحققه الشركة من إنتاج أو رقم أعمال وما تحققه من أرباح.</p>
<p> </p>
<p> </p>
<p>مادة 44</p>
<p>ملغاة</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 45</p>
<p>تنتهي خدمة العامل لأحد الأسباب الآتية:</p>
<p>1- فقد الجنسية المصرية أو انتفاء شرط المعاملة بالمثل بالنسبة لرعايا الدول الأخرى.</p>
<p>2- بلوغ سن الستين وذلك بمراعاة أحكام قانون التأمين الاجتماعي الصادر بالقانون رقم 79 لسنة 1975.</p>
<p>3- عدم اللياقة للخدمة صحيا.</p>
<p>4- صدور حكم بات بعقوبة جناية أو بعقوبة مقيدة للحرية في جريمة مخلة بالشرف أو الأمانة ما لم يكن الحكم مع وقف التنفيذ الشامل.</p>
<p>ودون إخلال بأحكام قانون العقوبات إذا كان قد حكم عليه لأول مرة فلا يؤدي ذلك إلى انتهاء الخدمة إلا إذا قدرت لجنة شئون العاملين بقرار مسبب من واقع أسباب الحكم وظروف الواقعة أن بقاءه في الخدمة يتعارض مع مقتضيات الوظيفة أو طبيعة العمل.</p>
<p>5- انتهاء العمل العرضي أو المؤقت أو الموسمي.</p>
<p>6- الاستقالة.</p>
<p>7- الإحالة إلى المعاش أو الفصل.</p>
<p>8- الوفاة.</p>
<p>وتحدد اللائحة التنفيذية الأوضاع والإجراءات الخاصة بانتهاء خدمة العامل بسبب الاستقالة أو عدم اللياقة للخدمة صحيا.</p>
<p> </p>
<p>مادة 46</p>
<p>يجوز عند الضرورة القصوى بقرار من رئيس مجلس الوزراء مد خدمة أي من العاملين من شاغلي الوظائف القيادية أصحاب الخبرة الفنية النادرة بالشركة لمدة سنة قابلة للتجديد لمدة أقصاها سنتان.</p>
<p>وأما بالنسبة للعاملين بالشركات من غير شاغلي الوظائف القيادية فلا يجوز مد خدمة أي منهم بعد بلوغ سن التقاعد أو تكليفه أو التعاقد معه بأي مسمى أو صفة في أي من الشركات الخاضعة لأحكام هذا القانون إلا بموافقة الوزير المختص عند الضرورة وبما لا يجاوز عامين.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p>مادة 47</p>
<p>يكون نقل رؤساء وحدات الأمن في الشركات الخاضعة لأحكام هذا القانون والعاملين بهذه الوحدات بقرار من السلطة المختصة في الشركة دون حاجة لأي إجراء آخر.</p>
<p> </p>
<p>مادة 48</p>
<p>تسري في شأن واجبات العاملين بالشركات القابضة والتابعة والتحقيق معهم وتأديبهم أحكام قانون العمل الصادر بالقانون رقم 12 لسنة 2003</p>
<p>كما تسري أحكام قانون العمل لمشار إليه، بما في ذلك الأحكام المنظمة للمفاوضة الجماعية على العاملين بالشركات القابضة والتابعة، فيما لم يرد في شأنه نص خاص في هذا القانون أو اللوائح الصادرة تنفيذا له.</p>
<p>ويجوز للوزير المختص أو مجلس إدارة الشركة أن يطلب من النيابة الإدارية التحقيق مع أي من العاملين بهذه الشركات في الحالات التي يقدرونها، ويتم موافاة الوزير المختص أو مجلس إدارة الشركة بنتيجة هذا التحقيق لتتولى السلطة المختصة بالشركة اتخاذ الإجراء أو القرار التأديبي المناسب</p>
<p>.</p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020</p>
<p> </p>
<p> </p>
<p> </p>
<p> الفصل الخامس: العقوبات</p>
<p> </p>
<p>مادة 49</p>
<p>مع عدم الإخلال بأية عقوبة أو وصف قانوني أشد ينص عليها قانون العقوبات أو أي قانون آخر يعاقب بالحبس مدة لا تقل عن سنتين وبغرامة لا تقل عن ألفي جنيه ولا تجاوز عشرة آلاف جنيه أو بإحدى هاتين العقوبتين كل من ارتكب أحد الأفعال الآتية:-</p>
<p>1- كل من عبث عمدا في نظام الشركة أو في نشرات الاكتتاب أو في غير ذلك من وثائق الشركة أو أثبت بها بيانات غير صحيحة أو مخالفة لأحكام هذا القانون أو قانون شركات المساهمة المشار إليه وكل من وقع هذه الوثائق أو وزعها مع علمه بذلك.</p>
<p>2- كل من قوم بسوء قصد الحصص العينية المقدمة من الشركة بأكثر من قيمتها الحقيقية.</p>
<p>3- كل مدير أو عضو مجلس إدارة وزع على المساهمين أو غيرهم أرباحا أو فوائد على خلاف أحكام هذا القانون أو نظام الشركة وكل مراقب حسابات أقر هذا التوزيع.</p>
<p>4- كل مدير أو عضو مجلس إدارة أو مصف ذكر عمدا بيانات غير صحيحة في الميزانية أو في حساب الأرباح والخسائر أو أغفل عمدا ذكر وقائع جوهرية في هذه الوثائق.</p>
<p>5- كل مراقب حسابات تعمد وضع تقرير غير صحيح عن نتيجة مراجعته أو أخفى عمدا وقائع جوهرية في هذا التقرير.</p>
<p>6- كل مدير أو عضو مجلس إدارة أو مراقب حسابات أو معاون له أو عامل لديه وكل شخص يعهد إليه بالتفتيش على الشركة أفشى ما يحصل عليه بحكم عمله من أسرار الشركة أو استغل هذه الأسرار لجلب نفع خاص له أو لغيره.</p>
<p>7- كل شخص عين من قبل الجهة الإدارية المختصة للتفتيش على الشركة أثبت عمدا في تقريره عن نتيجة التفتيش وقائع كاذبة أو أغفل عمدا في تقريره وقائع جوهرية من شأنها أن تؤثر في نتيجة التفتيش.</p>
<p> </p>
<p>مادة 50</p>
<p>مع عدم الإخلال بأية عقوبة أشد ينص عليها قانون العقوبات أو أي قانون آخر يعاقب بغرامة لا تقل عن خمسمائة جنيه ولا تجاوز خمسة آلاف جنيه كل من يصدر أسهما أو صكوكا أو سندات أو إيصالات اكتتاب أو شهادات مؤقتة أو يعرضها للتداول على خلاف الأحكام المقررة في هذا القانون.</p>
<p> </p>
<p> </p>
<p>مادة 51</p>
<p>تضاعف في حالة العود الغرامات المنصوص عليها في المادتين السابقتين في حديها الأدنى والأقصى.</p>
<p> </p>
<p>مادة 52</p>
<p>تعتبر أموال الشركات الخاضعة لأحكام هذا القانون في حكم الأموال العامة، كما يعد القائمون على إدارتها والعاملون فيها في حكم الموظفين العموميين وذلك في تطبيق أحكام البابين الثالث والرابع من الكتاب الثاني من قانون العقوبات.</p>
<p> </p>
<p>مادة 53</p>
<p>لا يجوز إحالة الدعوى الجنائية إلى المحكمة في الجرائم المشار إليها في المواد 116 مكررا و116 مكررا (أ) و116 مكررا (ب) من قانون العقوبات بالنسبة إلى أعضاء مجالس إدارة الشركات الخاضعة لأحكام هذا القانون إلا بناء على أمر من النائب العام أو من النائب العام المساعد أو من المحامي العام الأول.</p>
<p> </p>
<p>مادة 54</p>
<p>يكون للمكلفين بإثبات الجرائم التي تقع بالمخالفة لأحكام هذا القانون والقرارات الصادرة تنفيذا له الذين يصدر بتحديدهم قرار من وزير العدل بالاتفاق مع الوزير المختص حق الاطلاع على جميع سجلات ودفاتر الشركة القابضة أو الشركات التابعة لها.</p>
<p>وعلى رئيس وأعضاء مجلس الإدارة والعضو المنتدب ومراقب الحسابات وسائر العاملين بهذه الشركات أن يقدموا إليهم جميع البيانات والمعلومات والمستندات والوثائق والسجلات والدفاتر التي يطلبونها لأداء عملهم.</p>
<p> </p>
<p>مادة 55</p>
<p> </p>
<p>مع عدم الإخلال بحكم المادة (54) من هذا القانون أو بأي حكم وارد في أي قانون آخر، لا يجوز لأي جهة رقابية بالدولة عدا الجهاز المركزي للمحاسبات وهيئة الرقابة الإدارية أن تباشر أي عمل من أعمال الرقابة داخل المقر الرئيسي أو المقار الفرعية لأي شركة من الشركات الخاضعة لأحكام هذا القانون إلا بعد الحصول على إذن بذلك من الوزير المختص أو رئيس مجلس إدارة الشركة القابضة</p>
<p> </p>
<p>التعديل الأخير بموجب القانون 185 لسنة 2020.</p>
<p> </p>
<p> </p>
<p><strong>اللائحة التنفيذية للقانون</strong></p>
<p><strong>الصادرة بقرار رئيس مجلس الوزراء رقم 1590 لسنة 1991 بتاريخ 28 أكتوبر 1991</strong></p>
<p><strong>وفقا لتعديلاتها المتتابعة وآخرها بقرار رئيس الوزراء 949 لسنة 2008</strong></p>
<p>رئيس مجلس الوزراء</p>
<p>بعد الاطلاع على الدستور؛</p>
<p>وعلى قانون التجارة؛</p>
<p>وعلى القانون المدني؛</p>
<p>وعلى القانون رقم 55 لسنة 1951 بشأن الأسماء التجارية؛</p>
<p>وعلى القانون رقم 388 لسنة 1953 بشأن الدفاتر التجارية؛</p>
<p>وعلى القانون رقم 161 لسنة 1957 بشأن اللائحة العامة للبورصات؛</p>
<p>وعلى القانون رقم 47 لسنة 1972 بشأن مجلس الدولة؛</p>
<p>وعلى القانون رقم 73 لسنة 1973 بتحديد شروط وإجراءات انتخاب ممثلي العمال في مجالس إدارة وحدات القطاع العام وشركات المساهمة والجمعيات والمؤسسات الخاصة؛</p>
<p>وعلى القانون رقم 34 لسنة 1976 في شأن السجل التجاري؛</p>
<p>وعلى القانون رقم 137 لسنة 1981 بإصدار قانون العمل؛</p>
<p>وعلى القانون رقم 159 لسنة 1981 بإصدار قانون شركات المساهمة وشركات التوصية بالأسهم والشركات ذات المسئولية المحدودة؛</p>
<p>وعلى القانون رقم 144 لسنة 1988 بشأن إصدار قانون الجهاز المركزي للمحاسبات؛</p>
<p>وعلى القانون رقم 146 لسنة 1988 بإصدار قانون في شأن الشركات العاملة في مجال تلقي الأموال لاستثمارها؛</p>
<p>وعلى القانون رقم 203 لسنة 1991 بإصدار قانون شركات قطاع الأعمال العام؛</p>
<p>وعلى قرار رئيس الجمهورية رقم 4723 لسنة 1966 باعتماد النظام المحاسبي الموحد؛</p>
<p>وعلى اللائحة التنفيذية لقانون شركات المساهمة وشركات التوصية بالأسهم والشركات ذات المسئولية المحدودة الصادرة بقرار نائب رئيس مجلس الوزراء للشئون الاقتصادية والمالية ووزير شئون الاستثمار والتعاون الدولي رقم 96 لسنة 1982؛</p>
<p>وبناء على ما ارتآه مجلس الدولة؛</p>
<p>قرر:</p>
<p> </p>
<p> </p>
<p>(المادة الأولى)</p>
<p>يعمل بأحكام اللائحة التنفيذية لقانون شركات قطاع الأعمال العام الصادر بالقانون رقم 203 لسنة 1991 المرفقة.</p>
<p>وتسري أحكام اللائحة التنفيذية لقانون شركات المساهمة وشركات التوصية بالأسهم والشركات ذات المسئولية المحدودة الصادرة بقرار نائب رئيس مجلس الوزراء للشئون الاقتصادية والمالية ووزير شئون الاستثمار والتعاون الدولي رقم 96 لسنة 1982 وذلك فيما لم يرد به نص خاص بهذه اللائحة وبما لا يتعارض مع أحكامها.</p>
<p> </p>
<p>(المادة الثانية)</p>
<p>في تطبيق أحكام هذه اللائحة يقصد:</p>
<p>1- بالقانون: قانون شركات قطاع الأعمال العام المشار إليه.</p>
<p>2- بالوزير: الوزير المختص في تطبيق أحكام ذلك القانون.</p>
<p>ويتولى الوزير جميع الاختصاصات المقررة في القانون رقم 159 لسنة 1981 المشار إليه ولائحته التنفيذية لكل من الوزير المختص, والجهة الإدارية المختصة.</p>
<p> </p>
<p>(المادة الثالثة)</p>
<p>تطبق هذه اللائحة على:</p>
<p>1- الشركات القابضة والشركات التابعة التي تؤسس وفقا لأحكام القانون.</p>
<p>2- الشركات القابضة والشركات التابعة التي حلت محل هيئات القطاع العام والشركات التي كانت تشرف عليها.</p>
<p> </p>
<p>(المادة الرابعة)</p>
<p>تضع الجمعية العامة لكل من الشركات القابضة والشركات التابعة لها التي حلت محل هيئات القطاع العام والشركات التي كانت تشرف عليها نظاما أساسياً للشركة طبقاً للنموذج الذي يصدر به قرار من رئيس مجلس الوزراء.</p>
<p>وينشر هذا النظام على نفقة الشركة في الوقائع المصرية كما تقيد في السجل التجاري.</p>
<p> </p>
<p>(المادة الخامسة)</p>
<p>يتم تقويم صافي أصول كل شركة من الشركات القابضة والشركات التابعة التي حلت في تاريخ العمل بالقانون محل هيئات القطاع العام وشركاته التي كانت خاضعة للقانون رقم 97 لسنة 1983 وفقاً للقواعد التي تقررها الجمعية العامة لكل شركة من هذه الشركات.</p>
<p> </p>
<p>(المادة السادسة)</p>
<p>ينشر هذا القرار في الجريدة الرسمية, ويعمل به من اليوم التالي لتاريخ نشره</p>
<p>صدر برئاسة مجلس الوزراء في 20 ربيع الآخر سنة 1412هـ</p>
<p>الموافق 28 أكتوبر 1991 م</p>
<p>رئيس مجلس الوزراء</p>
<p>دكتور/ عاطف صدقي.</p>
<p> </p>
<p> الباب الأول: الشركات القابضة</p>
<p> الفصل الأول:  تأسيس الشركة القابضة</p>
<p> </p>
<p>مادة 1</p>
<p>تؤسس الشركة القابضة من شخص اعتباري عام واحد أو أكثر.</p>
<p> </p>
<p>مادة 2</p>
<p>يكون للشركة رأس مال مصدر. ويجوز أن يحدد النظام رأس مال مرخص به يجاوز رأس المال المصدر.</p>
<p>ويجب ألا يقل رأس المال المصدر للشركة عن عشرين مليوناً من الجنيهات وألا يقل المدفوع منه عند التأسيس عن 50%, ولا يسري ذلك على الشركات التي حلت محل هيئات القطاع العام التي كانت خاضعة للقانون رقم 97 لسنة 1983.</p>
<p>ويتم تأسيس الشركة عن طريق الاكتتاب المغلق.</p>
<p> </p>
<p>مادة 3</p>
<p>يقسم رأس مال الشركة إلى أسهم اسمية متساوية القيمة ولا يجوز تداول هذه الأسهم إلا فيما بين الأشخاص الاعتبارية العامة.</p>
<p>ويحدد النظام الأساسي القيمة الاسمية لكل سهم بما لا يقل عن مائة جنيه ولا يزيد على ألف جنيه.</p>
<p> </p>
<p>مادة 4</p>
<p>يقدم المؤسسون طلب التأسيس إلى الوزير مبيناً به اسم الشركة ومدتها والغرض من إنشائها وقيمة رأس المال اللازم لتحقيق هذا الغرض وطريقة أدائه. ويرفق بالطلب المستندات الآتية:</p>
<p>(أ) العقد الابتدائي بالنسبة للشركات التي يشترك في تأسيسها أكثر من شخص اعتباري عام.</p>
<p>(ب) مشروع النظام الأساسي للشركة.</p>
<p>ويجب أن يكون كل من العقد الابتدائي ومشروع النظام الأساسي مطابقاً للنموذج الذي يصدر به قرار من رئيس مجلس الوزراء.</p>
<p>(ج) شهادة من مصلحة السجل التجاري تفيد عدم التباس الاسم التجاري للشركة مع غيرها من الشركات.</p>
<p>(د) شهادة من أحد البنوك المعتمدة تفيد تمام الاكتتاب في جميع أسهم الشركة وأن النسبة الواجب سدادها من قيمة الأسهم قد تم أداؤها بالكامل وأن هذه القيمة قد وضعت لحساب الشركة إلى أن يتم قيدها بالسجل التجاري.</p>
<p>(هـ) إقرار من السلطات المختصة في الأشخاص الاعتبارية العامة المشتركة في التأسيس بالموافقة على الاشتراك في التأسيس وقيمة مساهمتها في رأس مال الشركة.</p>
<p> </p>
<p>مادة 5</p>
<p>يتولى الوزير متابعة واستيفاء إجراءات ومستندات تأسيس الشركة وبوجه خاص:</p>
<p>(أ) مراجعة مشروع النظام الأساسي والعقد الابتدائي إن وجد.</p>
<p>(ب) اتخاذ إجراءات التحقق من أن الحصص العينية في حالة وجودها قد قدرت تقديراً صحيحاً.</p>
<p> </p>
<p>مادة 6</p>
<p>يصدر بتأسيس الشركة قرار من رئيس مجلس الوزراء بناء على اقتراح الوزير.</p>
<p>وينشر النظام الأساسي للشركة على نفقتها في الوقائع المصرية وتقيد في السجل التجاري.</p>
<p> </p>
<p>مادة 7</p>
<p>جميع العقود والأوراق الصادرة عن الشركة والموجهة إلى الغير كالمكاتبات والفواتير والإعلانات والمطبوعات يجب أن تحمل اسم الشركة وعنوانها مسبوقاً أو مردفاً بعبارة «شركة مساهمة قابضة مصرية ش.م.ق.م» وذلك بحروف واضحة مقروءة مع بيان مركز الشركة الرئيسي ورأس المال المصدر.</p>
<p>ويسري ما تقدم على اللافتات التي توضع للإعلان عن الشركة بمقرها الرئيسي أو بفروعها أو بأي مكان آخر.</p>
<p> </p>
<p>الفصل الثاني:  مجلس إدارة الشركة القابضة</p>
<p>مادة 8</p>
<p>يعرض رئيس الجمعية العامة للشركة علي الجمعية الترشيحات لاختيار رئيس مجلس الإدارة والأعضاء من ذوي الخبرة، ويراعى بقدر الإمكان أن يكون من بينهم أحد العاملين بوزارة المالية.</p>
<p>ويرفق بالترشيح بيان مختصر عن الخبرة والسيرة الذاتية لكل مرشح والإنجازات التي حققها في أعماله السابقة وما كان يتقاضاه مقابل قيامه بهذه الأعمال.</p>
<p>وتصدر الجمعية العامة للشركة قرارها بتشكيل مجلس الإدارة على ألا يقل عن سبعة ولا يزيد على أحد عشر بما فيهم ممثل الاتحاد العام لنقابات عمال مصر.</p>
<p>ويجب إرسال صورة من إخطار الدعوة للجمعية العامة قبل موعد انعقادها بأسبوع على الأقل إلى الاتحاد العام لنقابات عمال مصر لاختيار ممثل الاتحاد في مجلس الإدارة.</p>
<p>وتكون مدة مجلس الإدارة ثلاث سنوات، ويجوز تجديد عضوية رئيس وأعضاء مجلس الإدارة الذين انتهت عضويتهم كلهم أو بعضهم لمدة أو لمدد أخرى بناء على اقتراح رئيس الجمعية العامة. ويرفق باقتراح التجديد بيان مختصر بالإنجازات التي حققها المجلس أو الأعضاء المطلوب تجديد مدة عضويتهم ومبررات التجديد.</p>
<p> </p>
<p>مادة 9</p>
<p>يجوز لمجلس إدارة الشركة القابضة دعوة رئيس مجلس إدارة أية شركة تابعة أو عضو مجلس إدارتها المنتدب لحضور اجتماعات مجلس إدارة الشركة القابضة عند نظر الموضوعات المتعلقة بالشركات التابعة وذلك لإبداء ما يراه من ملاحظات، أو آراء أو لتقديم ما يطلب منه من إيضاحات أو بيانات وله الاشتراك في المناقشة دون أن يكون له صوت معدود.</p>
<p> </p>
<p>مادة 10</p>
<p>لا يجوز لأي شخص أن يكون رئيساً أو عضواً متفرغاً للإدارة بمجلس إدارة أكثر من شركة واحدة من الشركات الخاضعة لأحكام القانون.</p>
<p>وفي جميع الأحوال لا يجوز الجمع بين عضوية مجلس إدارة أكثر من شركتين من تلك الشركات.</p>
<p>وتبطل العضوية في مجلس الإدارة التي يتجاوز بها العضو النصاب المقرر.</p>
<p> </p>
<p>مادة 11</p>
<p>لا يجوز أن يكون لرئيس مجلس الإدارة أو لأحد أعضاء المجلس أو لأحد مديري الشركة أو لأي من أزواجهم أو أقاربهم أو أصهارهم حتى الدرجة الرابعة أية مصلحة مباشرة في الأعمال والعقود التي تتم باسمها أو لحسابها، كما لا يجوز لرئيس مجلس إدارة الشركة أو لأي عضو بهذا المجلس أن يحضر مداولاته أو يشترك في التصويت على القرارات المتعلقة بأية مسألة معروضة على المجلس إذا كان لأي منهم أو لمن لهم صلة قرابة أو مصاهرة بهم إلى الدرجة الرابعة مصلحة مباشرة أو غير مباشرة فيها.</p>
<p> </p>
<p>مادة 12</p>
<p>لا يجوز لرئيس مجلس الإدارة أو لأي عضو من الأعضاء أن يشترك في أي عمل من شأنه منافسة الشركة أو أن يتجر لحسابه, أو لحساب غيره في أحد فروع النشاط الذي تزاوله, ولا يجوز كذلك لرئيس مجلس الإدارة أو لأي من أعضائه إفشاء أسرار الشركة أو تسهيل حصول الغير على أي من تقارير الأجهزة الرقابية أو أية تقارير داخلية تتعلق بأعمالها.</p>
<p> </p>
<p>مادة 13</p>
<p>رئيس وأعضاء مجلس الإدارة مسئولون عن أعمالهم أمام الجمعية العامة للشركة وذلك دون إخلال بمسئوليتهم الجنائية أو المدنية.</p>
<p> </p>
<p>مادة 14</p>
<p>يضع مجلس إدارة الشركة القابضة نماذج أشكال ومحتويات وتوقيتات التقارير الدورية التي تلتزم الشركات التابعة بإعدادها وإرسالها إلى الشركة القابضة لإجراء تقويم مستمر ومنتظم لنتائج أعمال تلك الشركات.</p>
<p>ويضع مجلس الإدارة النظام الذي يلتزم به ممثلو الشركة في مجالس إدارة الشركات التابعة لها، والموضوعات التي يتم متابعتها والتقارير التي يلتزمون بإعدادها.</p>
<p> </p>
<p>مادة 15</p>
<p>تعرض المسائل التالية على مجلس إدارة الشركة القابضة دورياً للنظر فيها واتخاذ القرار المناسب بشأنها:</p>
<p>(1) مشروع القوائم المالية التقديرية للشركة القابضة.</p>
<p>(2) التقارير الدورية عن تقويم الأداء والحسابات والقوائم الختامية ونتائج الأعمال للشركة القابضة.</p>
<p>(3) التقارير التي يعدها ممثلو الشركة القابضة في مجالس إدارة الشركات التابعة.</p>
<p>(4) مقترحات الاستثمار للشركة القابضة والدراسات التي أعدت عن كل منها وبرامج تمويلها.</p>
<p>(5) مقترحات تشكيل اللجان التي يعهد إليها المجلس ببعض اختصاصاته أو بمهام محددة.</p>
<p>(6) قوائم نتائج الأعمال والقوائم المالية الأخرى السنوية للشركات التابعة وتقارير مراقبي الحسابات.</p>
<p>(7) مؤشرات الاستثمار في الشركات التابعة.</p>
<p>(8) الدراسات التي تعد لتصحيح مسار الشركات التابعة ومقترحات تصحيح المسار.</p>
<p>(9) المركز المالي للشركات التابعة كل ثلاثة أشهر مصدقاً عليه من مراقب الحسابات.</p>
<p>(10) الترشيحات لشغل مناصب رؤساء وأعضاء مجالس الإدارة من ذوي الخبرة بالشركات التابعة ولشغل مناصب الأعضاء المنتدبين في هذه الشركات.</p>
<p>(11) جميع الموضوعات التي تحتاج إلى التنسيق أو التعاون المشترك بين الشركات التابعة، وفي هذه الحالات يدعى رؤساء مجالس إدارة هذه الشركات وأعضاؤها المنتدبون عند النظر في هذه الموضوعات للاشتراك في المداولات وتقديم المقترحات دون أن يكون لهم صوت معدود.</p>
<p>(12) أية موضوعات أخرى يرى رئيس مجلس الإدارة عرضها.</p>
<p> </p>
<p> </p>
<p>مادة 16</p>
<p>يرسل رئيس مجلس الإدارة إلى الوزير قبل بدء السنة المالية بستة أشهر القوائم التقديرية لنتائج أعمال الشركة للعام التالي، وموازنة الاستثمار والبرامج التي سيجرى تنفيذها لتصحيح مسار الشركات التابعة.</p>
<p>كما يرسل إليه أيضاً كل ثلاثة أشهر تقريراً يبين فيه نتائج أعمال الشركة وموقف الاستثمارات المالية التي تنفذها الشركة بنفسها أو من خلال الغير، والجهود التي بذلت لتصحيح مسار الشركات التابعة وبيانا مقارناً يوضح النتائج التي تحققت من محفظة الاستثمار والنتائج المتوقعة.</p>
<p> </p>
<p>مادة 17</p>
<p>يختص مجلس إدارة الشركة القابضة بتكوين وإدارة محفظة الأوراق المالية للشركة واستثمار أموالها سواء بنفسها أو من خلال الشركات التابعة لها وذلك في أي مجال يراه المجلس محققا لأغراض الشركة وتنمية مواردها.</p>
<p>وتتكون محفظة الأوراق المالية من الاستثمارات الآتية:</p>
<p>(1) تأسيس الشركات التابعة وغيرها من الشركات المساهمة سواء كان ذلك بمفردها أو بالاشتراك مع الغير من الأشخاص الاعتبارية العامة أو الخاصة أو الأفراد.</p>
<p>(2) شراء أسهم الشركات المساهمة أو بيعها أو المساهمة في رأسمالها.</p>
<p>(3) التصرف بالبيع في الأسهم التي تملكها في الشركات التابعة وغيرها من الشركات.</p>
<p>(4) شراء وبيع أية أصول مالية أخرى.</p>
<p>(5) إصدار صكوك تمويل أو سندات لتجميع الأموال وإعادة استثمارها.</p>
<p>(6) القيام بجميع الإجراءات التي يراها المجلس لازمة لزيادة قيمة الاستثمارات التي تديرها الشركة أو زيادة الأرباح التي تتحقق منها.</p>
<p> </p>
<p> </p>
<p>الفصل الثالث: الجمعية العامة للشركة القابضة</p>
<p>مادة 18</p>
<p>يحدد النظام الأساسي للشركة عدد أعضاء الجمعية العامة على أن لا يقل عن اثني عشر ولا يزيد على أربعة عشر من بينهم ممثل واحد على الأقل يرشحه الاتحاد العام لنقابات عمال مصر, ويكون عدد أعضاء أول جمعية عامة اثني عشر عضوا.</p>
<p>ويعرض الوزير على رئيس مجلس الوزراء الترشيحات لأعضاء الجمعية العامة مرفقا بها بيان مختصر عن الخبرة والسيرة الذاتية لكل مرشح والإنجازات التي حققها في أعماله السابقة.</p>
<p>ويصدر باختيار أعضاء الجمعية العامة للشركة قرار من رئيس مجلس الوزراء، ويجب أن يكونوا من ذوي الخبرة في مجال الأنشطة التي تقوم بها الشركات التابعة لها، ويراعى بقدر الإمكان أن يكون أحدهم من بين العاملين بوزارة المالية.</p>
<p>ويحدد قرار رئيس مجلس الوزراء باختيار أعضاء الجمعية العامة ما يتقاضونه من بدل حضور جلسات الجمعية بما لا يقل عن مائة جنيه ولا يزيد على مائتي جنيه في الجلسة الواحدة وذلك بحسب حجم نشاط الشركة وطبيعة أعمالها.</p>
<p> </p>
<p>مادة 19</p>
<p>يرأس الجمعية العامة للشركة القابضة الوزير.</p>
<p>ويحضر اجتماعات الجمعية العامة رئيس وأعضاء مجلس إدارة الشركة القابضة ومراقبو الحسابات بالجهاز المركزي للمحاسبات دون أن يكون لهم صوت معدود.</p>
<p> </p>
<p> </p>
<p>مادة 20</p>
<p>تكون مدة العضوية للجمعية العامة ثلاث سنوات.</p>
<p>ويجوز تجديد العضوية لأعضاء الجمعية العامة الذين انتهت مدتهم كلهم أو بعضهم لمدة أو لمدد أخرى وذلك في ضوء ما تسفر عنه نتائج أعمال الشركة.</p>
<p>مادة 21</p>
<p>تحدد الجمعية العامة الأهداف التي تلتزم الشركة بتحقيقها ويتم تقويم أدائها وفقا لما يتحقق من هذه الأهداف وفي ضوء تقرير الجهاز المركزي للمحاسبات في هذا الشأن.</p>
<p> </p>
<p>مادة 22</p>
<p>تجتمع الجمعية العامة العادية مرتين على الأقل سنويا إحداهما قبل بداية السنة المالية بثلاثة أشهر وذلك للنظر في الموازنة التقديرية للشركة والأخرى خلال ستة أشهر من انتهاء السنة المالية للنظر في المسائل الآتية:</p>
<p>(1) تقرير مراقب الحسابات.</p>
<p>(2) التصديق على تقرير مجلس الإدارة عن نشاط الشركة والنظر في إخلاء مسئوليته عن الفترة المقدم عنها التقرير.</p>
<p>(3) التصديق على الميزانية وحساب الأرباح والخسائر والحسابات الختامية للشركة.</p>
<p>(4) الموافقة على توزيع الأرباح.</p>
<p>(5) الموافقة على استمرار رئيس وأعضاء مجلس الإدارة لمدة تالية.</p>
<p>(6) تشكيل مجلس إدارة الشركة.</p>
<p>(7) النظر في تقارير الجهاز المركزي للمحاسبات بشأن مراقبة حسابات الشركة وتقويم أدائها واتخاذ ما يلزم في شأنها من قرارات.</p>
<p>(8) كل ما يرى رئيس الجمعية أو مجلس الإدارة عرضه عليها.</p>
<p> </p>
<p>مادة 23</p>
<p>لرئيس الجمعية العامة دعوتها للانعقاد في اجتماع عادي أو غير عادي كلما رأى مقتضى لذلك.</p>
<p>وعليه دعوتها للانعقاد إذا طلب ذلك مجلس إدارة الشركة أو مراقب الحسابات على أن يوضح بالطلب الأسباب الداعية إلى عقد الاجتماع والمسائل المطلوب عرضها.</p>
<p> </p>
<p>مادة 24</p>
<p>مع مراعاة أحكام المادة السابقة وما تقضي به نصوص النظام الأساسي تختص الجمعية العامة العادية بنظر المسائل التالية في أي من الاجتماعين المشار إليهما في المادة (22) أو في أي اجتماع آخر تعقده لهذا الغرض خلال السنة المالية:</p>
<p>(1) وقف تجنيب الاحتياطي القانوني إذا بلغ ما يساوي نصف رأس المال.</p>
<p>(2) استخدام الاحتياطي النظامي فيما يعود بالنفع على الشركة بناء على اقتراح مجلس الإدارة إذا لم يكن هذا الاحتياطي مخصصا لأغراض معينة منصوص عليها في نظام الشركة.</p>
<p>(3) التصرف في الاحتياطيات والمخصصات في غير الأبواب المخصصة لها.</p>
<p>(4) الموافقة على إصدار سندات أو صكوك تمويل وعلى الضمانات التي تقرر عليها.</p>
<p>(5) النظر في قرارات وتوصيات جماعة حملة السندات.</p>
<p> </p>
<p>مادة 25</p>
<p>  النص النهائى للمادة بتاريخ :  15/09/2020</p>
<p>تختص الجمعية العامة غير العادية بما يأتي:</p>
<p>أولاً: تعديل نظام الشركة بمراعاة ألا يترتب على ذلك زيادة التزامات المساهمين ما لم يوافق على ذلك جميع المساهمين ويقع باطلا كل قرار يصدر من الجمعية العامة يكون من شأنه المساس بحقوق المساهم الأساسية التي يستمدها بصفته شريكا.</p>
<p>وتنظر الجمعية العامة غير العادية بصفة خاصة التعديلات التالية في نظام الشركة:</p>
<p>(1) زيادة رأس المال المرخص به أو المصدر في حالة عدم وجود رأس مال مرخص به.</p>
<p>(2) إضافة أية أغراض مكملة أو مرتبطة أو قريبة من غرض الشركة الأصلي ولا تكون الموافقة على تغيير الغرض الأصلي نافذة إلا بموافقة رئيس مجلس الوزراء.</p>
<p>(3) إطالة أمد الشركة أو تقصيره أو حلها قبل انتهاء مدتها أو تغيير نسبة الخسارة التي يتعين عند تحققها دعوة الجمعية العامة غير العادية للنظر في تصفية الشركة أو استمرارها.</p>
<p>ثانياً: اقتراح إدماج الشركة في غيرها من الشركات القابضة.</p>
<p>ثالثا: اقتراح تقسيم الشركة.</p>
<p>رابعا: النظر في تصفية الشركة أو استمرارها إذا بلغت خسائرها نصف رأس المال أو أية نسبة أقل يحددها النظام.</p>
<p>خامسا: بيع كل أو بعض أسهم الشركة التابعة بما يؤدي إلى خفض حصة الشركة القابضة أو الأشخاص الاعتبارية العامة و البنوك المملوكة أسهمها بالكامل للدولة في رأس مالها عن 51%.</p>
<p> </p>
<p>مادة 26</p>
<p>لا يجوز للشركة التصرف بالبيع في أصل من خطوط الإنتاج الرئيسية إلا بموافقة الجمعية العامة غير العادية وطبقاً لما يأتي:</p>
<p>(1) أن تكون الشركة عاجزة عن تشغيل هذه الخطوط تشغيلا اقتصاديا أو أن يؤدي الاستمرار في تشغيلها إلى تحميل الشركة خسائر مؤكدة.</p>
<p>(2) ألا يقل سعر البيع عن القيمة التي تقدرها اللجنة المنصوص عليها في المادة (19) من القانون.</p>
<p> </p>
<p>مادة 26 مكرر</p>
<p>يتم طرح الشركات التابعة في إطار برنامج إدارة الأصول المعتمد أيا كانت وسيلة البيع.</p>
<p>ويكون طرح الشركة التابعة بالكامل أو بأغلبية الأسهم فيها بطريق الاكتتاب أو بنظام عروض الشراء من خلال بورصة الأوراق المالية - بعد عرض الوزير المختص على المجموعة الوزارية للسياسات الاقتصادية بحضور وزير القوى العاملة والهجرة.</p>
<p>كما يتم العرض على المجموعة المشار إليها لاستكمال إجراءات البيع لمستثمر رئيسي أو أكثر بصفة نهائية.</p>
<p> </p>
<p>مادة27</p>
<p>إذا قررت الجمعية العامة غير العادية تصفية الشركة يتعين أن تتضمن قراراتها الصادرة في هذا الشأن المسائل الآتية:</p>
<p>(أ‌) تعيين المصفي أو المصفين وتحديد أتعابهم ومدة التصفية.</p>
<p>(ب‌) مد المدة المقررة للتصفية بعد الاطلاع على تقرير المصفي.</p>
<p>(جـ) النظر في الحساب المؤقت الذي يقدمه المصفي.</p>
<p>(د) التصديق على الحساب الختامي لأعمال التصفية.</p>
<p>(هـ) تعيين المكان الذي تحفظ فيه دفاتر الشركة ووثائقها بعد إتمام التصفية وشطبها من السجل التجاري.</p>
<p> </p>
<p>مادة 28</p>
<p>في جميع الأحوال لا يكون انعقاد الجمعية العامة صحيحا إلا إذا حضره نصف عدد أعضائها على الأقل بما فيهم رئيس الجمعية, وتصدر قرارات الجمعية العامة العادية بالأغلبية المطلقة لعدد أصوات الحاضرين أما قرارات الجمعية العامة غير العادية فلا تصدر إلا بأغلبية ثلثي عدد أصوات الحاضرين.</p>
<p> </p>
<p>مادة 29</p>
<p>مع مراعاة ما وردت بشأنه أحكام خاصة بهذه اللائحة تسري في شأن إجراءات وشروط صحة انعقاد الجمعية العامة العادية وغير العادية ونظام التصويت على المسائل المعروضة عليها أحكام المواد من 200 إلى 231 من اللائحة التنفيذية لقانون شركات المساهمة وشركات التوصية بالأسهم والشركات ذات المسئولية المحدودة المشار إليها.</p>
<p> </p>
<p>الفصل الرابع: النظام المالي للشركة القابضة ـ توزيع الأرباح ـ الاحتياطيات</p>
<p>مادة 30</p>
<p>(أ) تتكون أصول الشركة من:</p>
<p>(1) الأسهم التي تملكها في رؤوس أموال الشركات التابعة وغيرها من الشركات.</p>
<p>(2) الأوراق المالية الأخرى.</p>
<p>(3) الأصول الثابتة والأصول المتداولة الناتجة عن قيام الشركة بأنشطتها المتنوعة.</p>
<p>(ب) تتكون خصوم الشركة من:</p>
<p>(1) رأس المال الذي تساهم به الدولة والأشخاص الاعتبارية العامة الأخرى.</p>
<p>(2) الاحتياطيات والمخصصات التي تنشئها الشركة.</p>
<p>(3) القروض والتسهيلات التي تحصل عليها.</p>
<p>(4) الأرباح التي تحتجزها من الفائض الذي يؤول إليها من استثماراتها.</p>
<p>(5) الخصوم المتداولة الناتجة عن قيام الشركة بأنشطتها.</p>
<p> </p>
<p> </p>
<p>مادة 31</p>
<p>يعرض على مجلس إدارة الشركة تقرير ربع سنوي يتضمن حساب العمليات الجارية وفائض هذه العمليات ويجب أن يشمل هذا التقرير بيانا مقارنا عن الفترات المماثلة من السنة المالية السابقة وكذلك الأرقام المخططة والمدرجة في الموازنة التقديرية.</p>
<p> </p>
<p>مادة 32</p>
<p>يجب أن تتضمن اللوائح والنظم الإدارية والمالية للشركة الصلاحيات والضمانات الكافية للقائمين بأعمال المراقبة أو المراجعة الداخلية أو الخارجية وأن تتضمن تنظيم إجراءات الجرد المفاجئ.</p>
<p> </p>
<p>مادة 33</p>
<p>تعد الشركة في نهاية كل سنة مالية قوائم مالية مجمعة تعرض أصول والتزامات وحقوق المساهمين وإيرادات ومصروفات واستخدامات الشركة والشركات التابعة لها وفقا للأوضاع والشروط والبيانات الواردة بالملحق رقم (5) المرفق باللائحة التنفيذية لقانون شركات المساهمة وشركات التوصية بالأسهم والشركات ذات المسئولية المحدودة المشار إليه.</p>
<p> </p>
<p> </p>
<p>مادة 34</p>
<p>الأرباح الصافية هي الأرباح الناتجة عن حصة الشركة في أرباح الشركات التابعة لها أو غيرها أو من العمليات التي باشرتها الشركة بنفسها خلال السنة المالية وذلك بعد خصم جميع المصروفات اللازمة لتحقيق هذه الأرباح وبعد حساب وتجنيب جميع الاهلاكات والمخصصات التي تقضي الأصول المحاسبية بحسابها وتجنيبها قبل إجراء أي توزيع بأية صورة من الصور.</p>
<p>ويجب إجراء الاهلاكات وتجنيب المخصصات المشار إليها حتى في السنوات التي لا تحقق فيها الشركة أرباحا، أو تحقق أرباحا غير كافية.</p>
<p> </p>
<p>مادة 35</p>
<p>يجب على مجلس الإدارة لدى إعداده الميزانية وحساب الأرباح والخسائر أن يجنب من الأرباح الصافية جزءا من عشرين على الأقل لتكوين احتياطي قانوني. ويجوز للجمعية العامة وقف تجنيب هذا الاحتياطي القانوني إذا بلغ ما يساوي رأس المال المصدر ويجوز استخدام الاحتياطي القانوني في تغطية خسائر الشركة وفي زيادة رأس المال.</p>
<p> </p>
<p>مادة 36</p>
<p>يجوز أن ينص النظام الأساسي للشركة على تجنيب نسبة معينة من الأرباح الصافية لتكوين احتياطي نظامي بحد أقصى 10% بالنسبة للشركات التي لا تزاول النشاط بنفسها و20% بالنسبة لغيرها من الشركات وذلك لمواجهة الأغراض التي يحددها النظام.</p>
<p>وإذا لم يكن الاحتياطي النظامي مخصصا لأغراض معينة جاز للجمعية العامة العادية بناء على اقتراح من مجلس الإدارة مشفوعا بتقرير من مراقب الحسابات أن تقرر استخدامه فيما يعود بالنفع على الشركة أو على المساهمين.</p>
<p>وفي جميع الأحوال لا يجوز التصرف في الاحتياطيات والمخصصات الأخرى في غير الأبواب المخصصة لها إلا بقرار من الجمعية العامة بما يحقق أغراض الشركة, على أن يحدد القرار أوجه الاستخدام لهذه الاحتياطيات والمخصصات.</p>
<p> </p>
<p>مادة 37</p>
<p>يجوز للجمعية العامة بناء على اقتراح مجلس الإدارة أن تقرر تكوين احتياطيات أخرى غير الاحتياطي القانوني والنظامي بما لا يجاوز 10% من المتبقي من الأرباح الصافية بعد تجنيب الاحتياطي القانوني والنظامي، وتخصيص نسبة من الربح لا تقل عن 5% من رأس المال للمساهمين والعاملين كحصة أولى ونسبة لا تزيد على 5% لمكافأة أعضاء مجلس الإدارة.</p>
<p> </p>
<p>مادة 38</p>
<p>الأرباح القابلة للتوزيع هي الأرباح الصافية مستنزلا منها، ما يكون قد لحق برأس مال الشركة من خسائر في سنوات سابقة، وبعد تجنيب الاحتياطيات المنصوص عليها في المادتين (35 و36) من اللائحة.</p>
<p>ويجوز للجمعية العامة أن تقرر توزيع كل أو بعض الاحتياطيات التي تملك التصرف فيها بموجب نصوص القانون أو اللائحة أو النظام ويجب أن يتضمن قرار الجمعية في هذا الشأن بيانا بأوضاع المال الاحتياطي الذي يجرى التوزيع منه.</p>
<p> </p>
<p>مادة 39</p>
<p>لا يجوز توزيع الأرباح التي تحققها الشركة نتيجة التصرف في أصل من الأصول الثابتة أو التعويض عنه، وتكون الشركة من هذه الأرباح احتياطيا يخصص لإعادة أصولها إلى ما كانت عليه أو شراء أصول ثابتة جديدة, أو لسداد ديون الشركة.</p>
<p>ويسري هذا الحكم في حالة إعادة تقويم أصول الشركة.</p>
<p> </p>
<p>مادة 40</p>
<p>بمراعاة أحكام القانون واللائحة والنظام الأساسي للشركة، تحدد الجمعية العامة بعد إقرار الميزانية وحساب الأرباح والخسائر، الأرباح القابلة للتوزيع وتعلن ما يخص العاملين والمساهمين ومجلس الإدارة منها وذلك بمراعاة ما يأتي:</p>
<p>أولا: ألا يقل نصيب العاملين بالشركة في الأرباح التي يتقرر توزيعها في الشركات التي تزاول النشاط بنفسها عن 10%.</p>
<p>ولا يجوز أن يزيد ما يصرف للعاملين نقدا على مجموع أجورهم الأساسية السنوية، ويجنب ما يزيد على ذلك في حساب خاص لإنشاء مشروعات إسكان للعاملين بالشركة وتقديم الخدمات الاجتماعية لهم وفقا لما تقرره الجمعية العامة للشركة.</p>
<p>ثانيا: ألا يزيد نصيب العاملين في الأرباح التي يتقرر توزيعها في الشركات التي لا تزاول النشاط بنفسها على مجموع أجورهم الأساسية السنوية.</p>
<p>ثالثا: ألا يجاوز تقدير مكافأة مجلس الإدارة بنسبة معينة في الأرباح أكثر من 5% من الربح القابل للتوزيع بعد تخصيص نسبة من الربح لا تقل عن 5% من رأس المال للمساهمين والعاملين كحصة أولى.</p>
<p>ويراعى في تحديد ما يصرف من مكافآت لرئيس وأعضاء مجلس الإدارة الجهود التي بذلوها لزيادة إنتاج الشركة عن السنة المالية السابقة وتخفيض خسائر الشركات التابعة لها.</p>
<p> </p>
<p> </p>
<p>مادة 41</p>
<p>يجوز للجمعية العامة بناء على اقتراح مجلس إدارة الشركة القابضة تخصيص نسبة من الاحتياطي النظامي والاحتياطيات الأخرى المنصوص عليها في المادة 37 لتمويل البرامج المالية التي تكفل تصحيح مسار الشركات التابعة وذلك وفقا للقواعد وبالشروط والأوضاع التي تحددها الجمعية العامة.</p>
<p> </p>
<p>مادة 42</p>
<p>يوزع الربح المتبقي من الربح القابل للتوزيع على الدولة وغيرها من الأشخاص الاعتبارية العامة المساهمة في رأس مال الشركة.</p>
<p> </p>
<p>مادة 43</p>
<p>يجوز للجمعية العامة بناء على اقتراح مجلس الإدارة وفي ضوء تقرير مراقب الحسابات عدم توزيع الأرباح إذا كان ذلك ضروريا لاستمرار نشاط الشركة أو المحافظة على مركزها المالي وذلك دون الإخلال بتخصيص ربح لا يقل عن 5% من رأس المال للمساهمين والعاملين وخصم مكافأة مجلس الإدارة.</p>
<p> </p>
<p> الباب الثاني: الشركات التابعة للشركات القابضة</p>
<p> الفصل الأول: التأسيس</p>
<p> </p>
<p>مادة 44</p>
<p>  النص النهائى للمادة بتاريخ :  15/09/2020</p>
<p>يؤسس الشركة التابعة إحدى الشركات القابضة بمفردها أو بالاشتراك مع شركات قابضة أخرى أو أشخاص اعتبارية عامة أو البنوك المملوكة أسهمها بالكامل للدولة أو أفراد أو أشخاص اعتبارية من القطاع الخاص.</p>
<p>ويجوز أن تكون الأنشطة التي تزاولها الشركات التابعة لأية شركة قابضة متماثلة أو متكاملة أو متباينة.</p>
<p> </p>
<p>مادة 45</p>
<p>يعرض رئيس مجلس إدارة الشركة القابضة على الوزير قرار مجلس إدارة الشركة القابضة باقتراح تأسيس الشركة التابعة وطلب التأسيس مبيناً به اسم الشركة التابعة ومدتها والغرض من إنشائها وقيمة رأس المال اللازم لتحقيق هذا الغرض وطريقة أدائه مع مذكرة مشتملة على جميع البيانات الواجب توافرها قانونا لتأسيس الشركة.</p>
<p>ويرفق بطلب التأسيس المستندات الآتية:</p>
<p>(أ) العقد الابتدائي بالنسبة للشركات التي يشترك في تأسيسها أكثر من شخص طبيعي أو اعتباري.</p>
<p>(ب) مشروع النظام الأساسي للشركة.</p>
<p>(جـ) شهادة من مصلحة السجل التجاري تفيد عدم التباس الاسم التجاري للشركة مع غيرها من الشركات.</p>
<p>(د) شهادة من أحد البنوك المعتمدة تفيد تمام الاكتتاب في جميع أسهم الشركة، وأن النسبة الواجب سدادها من قيمة الأسهم قد تم أداؤها بالكامل وأن هذه القيمة قد وضعت لحساب الشركة إلى أن يتم قيدها بالسجل التجاري.</p>
<p>(هـ) إقرار من السلطات المختصة في الأشخاص الاعتبارية المشتركة في التأسيس بالموافقة على التأسيس وقيمة مساهمتها في رأس مال الشركة.</p>
<p>(و) نموذج الإقرار المرفق بهذه اللائحة مستوفيا بالنسبة للمؤسسين من الأشخاص الطبيعيين.</p>
<p> </p>
<p>مادة 46</p>
<p>يجب أن يكون رأس مال الشركة كافيا لتحقيق أغراضها، وأن يكون مكتتبا فيه بالكامل وألا يقل المدفوع منه نقدا عند التأسيس عن الربع.</p>
<p> </p>
<p>مادة 47</p>
<p>يكون للشركة رأس مال مصدر، ويجوز أن يحدد النظام رأس مال مرخصا به بما يجاوز رأس المال المصدر.</p>
<p>ويكون الاكتتاب في رأس المال المصدر للشركات التابعة إما بطرح الأسهم للاكتتاب العام، أو بالاكتتاب المغلق.</p>
<p> </p>
<p> </p>
<p>مادة 48</p>
<p>مع عدم الإخلال بما تنص عليه القوانين واللوائح الخاصة يجب ألا يقل رأس مال الشركة المصدر عن مليون جنيه ولا يسري ذلك على الشركات التي كانت تشرف عليها هيئات القطاع العام.</p>
<p> </p>
<p>مادة 49</p>
<p>يجب أن يكون كل من العقد الابتدائي والنظام الأساسي للشركة موقعا من المؤسسين ومطابقا للنموذج الذي يصدر به قرار من رئيس مجلس الوزراء.</p>
<p>ولا يجوز للمؤسسين إغفال إدراج البيانات المتعلقة باسم الشركة وغرضها وقيمة رأس مالها وعدد الأسهم التي ينقسم إليها ومراتبها والقيمة الاسمية للسهم وما يرد من قيود على تداولها، وغير ذلك من البيانات الإلزامية التي ينص النموذج على وجوب إدراجها.</p>
<p> </p>
<p>مادة 50</p>
<p>يصدر الوزير قرار التأسيس بعد التأكد من استيفاء جميع أوراق ومستندات التأسيس ومراجعة مشروع النظام الأساسي للشركة والتحقق من أن الحصص العينية ـ في حالة وجودها ـ قد قدرت تقديرا صحيحا. طبقا لنص المادة 19 من القانون.</p>
<p>ويجوز لذوي الشأن التظلم من قرار لجنة التقييم إلى الوزير بالشروط والأوضاع المنصوص عليها في قانون مجلس الدولة الصادر بالقانون رقم 47 لسنة 1972، فإذا ما رأى الوزير أن التظلم يقوم على أسباب جدية أحال الأمر إلى لجنة أخرى، لإعادة التقييم ويكون القرار الصادر في هذا الشأن بعد اعتماد الوزير نهائيا.</p>
<p> </p>
<p>مادة 51</p>
<p>يسري في شأن إصدار أسهم الزيادة في رأس المال بقيمة اسمية أعلى والبيانات التي تتضمنها شهادات الأسهم وكيفية استبدال الشهادات المفقودة والتالفة وما يتبع بالنسبة لهذه الشهادات عند تعديل نظام الشركة أحكام المواد 94 و125 و126 و129 من اللائحة التنفيذية لقانون شركات المساهمة وشركات التوصية بالأسهم والشركات ذات المسئولية المحدودة المشار إليه.</p>
<p> </p>
<p>مادة 52</p>
<p>جميع العقود والأوراق الصادرة عن الشركة والموجهة إلى الغير كالمكاتبات والفواتير والإعلانات والمطبوعات يجب أن تحمل اسم الشركة وعنوانها مسبوقا أو مردفا بعبارة (شركة تابعة مساهمة مصرية - ش.ت.م.م) وذلك بحروف واضحة مقروءة، مع بيان اسم الشركة القابضة التي تتبعها الشركة ومركز الشركة الرئيسي.</p>
<p>ويسري ما تقدم على اللافتات التي توضع للإعلان عن الشركة سواء في مقرها الرئيسي أو في فروعها أو بأي مكان آخر.</p>
<p> </p>
<p> </p>
<p> </p>
<p> </p>
<p>الفصل الثاني:  مجلس إدارة الشركة التابعة</p>
<p> </p>
<p>مادة 53</p>
<p>يبين النظام الأساسي للشركة عدد أعضاء مجلس الإدارة على ألا يقل عن خمسة ولا يزيد على تسعة بما فيهم الرئيس ويراعى في تحديد العدد حجم نشاط الشركة وطبيعته، ويكون عدد أعضاء مجلس الإدارة الأول خمسة أعضاء.</p>
<p> </p>
<p> </p>
<p>مادة 54</p>
<p>يعرض مجلس إدارة الشركة القابضة على الجمعية العامة للشركة التابعة التي لا يساهم القطاع الخاص في رأس مالها أو على رئيس الجمعية العامة للشركة التي يساهم القطاع الخاص في رأس مالها بحسب الأحوال الترشيح لشغل منصب رئيس مجلس إدارة الشركة التابعة.</p>
<p>كما يختار مجلس إدارة الشركة القابضة أعضاء مجلس إدارة الشركة التابعة من ذوي الخبرة الذين يمثلون الجهات المساهمة في الشركة.</p>
<p>وفي جميع الأحوال يرفق بالترشيح لاختيار رؤساء وأعضاء مجالس إدارة الشركات المشار إليها بيان مختصر بالخبرة والسيرة الذاتية لكل مرشح والإنجازات التي حققها في أعماله السابقة، وما كان يتقاضاه قبل قيامه بهذه الأعمال.</p>
<p>ويختار ممثلو الأفراد والأشخاص الاعتبارية من القطاع الخاص في الجمعية العامة ممثليهم في مجلس الإدارة وذلك بالنسبة للشركات التي يساهم القطاع الخاص في رأسمالها.</p>
<p> </p>
<p> </p>
<p>مادة 55</p>
<p>ينتخب العاملون في الشركة من بينهم أعضاء غير متفرغين بمجلس الإدارة طبقا للقانون رقم 73 لسنة 1973 المشار إليه. ويكون عددهم مساويا لعدد الأعضاء من ذوي الخبرة وممثلي القطاع الخاص وذلك بالنسبة للشركات التي يساهم القطاع الخاص في رأس مالها أما بالنسبة للشركات التي لا يساهم القطاع الخاص في رأس مالها فيكون عدد هؤلاء الأعضاء مساويا لعدد الأعضاء من ذوي الخبرة.</p>
<p>ويتم الانتخاب في الأسبوع السابق لموعد انعقاد الجمعية العامة والمدرج في جدول أعمالها إعلان تشكيل مجلس الإدارة.</p>
<p>وترسل صورة من إخطار الدعوى للجمعية العامة إلى الجهات المعنية وفقا لأحكام القانون رقم 73 لسنة 1973 المشار إليه بخطاب يطلب فيه السير في إجراءات انتخاب العاملين وذلك قبل موعد انعقاد الجمعية بشهر على الأقل.</p>
<p> </p>
<p>مادة 56</p>
<p>يتم تشكيل مجلس الإدارة من الأعضاء المشار إليهم في المواد السابقة وتكون مدة المجلس ثلاث سنوات تبدأ من اليوم التالي لتاريخ إعلان تشكيل المجلس.</p>
<p>ويجوز تجديد عضوية رئيس وأعضاء مجلس الإدارة الذين انتهت مدة عضويتهم كلهم أو بعضهم لمدة أو لمدد أخرى.</p>
<p>ويعتبر عضوا في المجلس رئيس اللجنة النقابية بالشركة التابعة دون أن يكون له صوت معدود.</p>
<p>وإذا تعددت اللجان النقابية في الشركة تخطر النقابة العامة بموعد انعقاد الجمعية العامة ويطلب منها تسمية أحد رؤساء اللجان للانضمام لعضوية المجلس وذلك قبل موعد الانعقاد بشهر على الأقل.</p>
<p> </p>
<p>مادة 57</p>
<p>تسري أحكام المواد 10، 11، 12، 13 من هذه اللائحة على رؤساء وأعضاء مجالس إدارة الشركات التابعة والأعضاء المنتدبين.</p>
<p> </p>
<p>مادة 58</p>
<p>لمجلس الإدارة ولرئيسه أن يدعو لحضور اجتماعات المجلس من يرى الاستعانة بهم من ذوي الخبرة من غير أعضاء مجلس الإدارة، وذلك لتقديم ما يطلب منهم من آراء أو بيانات أو إيضاحات دون أن يكون لأي منهم صوت معدود فيما يتخذه المجلس من قرارات.</p>
<p> </p>
<p>مادة 59</p>
<p>لمجلس الإدارة أن يشكل من بين أعضائه لجنة أو لجانا يعهد إليها ببعض اختصاصاته، كما يجوز له أن يعهد إلى رئيس المجلس أو أحد المديرين ببعض اختصاصاته.</p>
<p>وللمجلس أن يفوض أحد أعضائه أو أحد المديرين في القيام بمهمة محددة على أن يعرض على المجلس تقريرا بما قام به من أعمال.</p>
<p> </p>
<p>مادة 60</p>
<p>يتولى رئيس مجلس الإدارة غير المتفرغ المهام الآتية:</p>
<p>(1) رئاسة جلسات مجلس الإدارة.</p>
<p>(2) وضع جدول الأعمال لجلسات المجلس بالتشاور مع العضو المنتدب.</p>
<p>(3) التأكد من تنفيذ العضو المنتدب لقرارات المجلس.</p>
<p>(4) التأكد من أن التقارير الشهرية عن نتائج أعمال الشركة والمشروعات الاستثمارية المطلوب تنفيذها وبرامج التمويل قد أعدت بالصورة التي حددها المجلس.</p>
<p> </p>
<p>مادة 61</p>
<p>يتولى عضو مجلس الإدارة المنتدب وحده رئاسة العمل التنفيذي للشركة وتصريف أمورها اليومية والإشراف على سير العمل في جميع قطاعات الشركة، ومتابعة الأداء لجميع الأنشطة واتخاذ ما يراه من قرارات لانتظام العمل وتحقيق الأهداف وله على الأخص مباشرة الاختصاصات الآتية:</p>
<p>(1) اقتراح الموضوعات التي تطرح في الاجتماعات الدورية لمجلس الإدارة ويتشاور بشأنها مع رئيس المجلس.</p>
<p>(2) مراجعة كافة التقارير الدورية التي تعد للعرض على مجلس الإدارة قبل إرسالها إلى رئيس المجلس.</p>
<p>(3) الإشراف على إعداد برنامج العمل التفصيلي للشركة للعام التالي والقوائم المالية المعبرة عنه والتقرير السنوي عن نتائج أعمال الشركة وتقويم أدائها ومراجعة كافة الردود على استفسارات مراقبي الحاسبات قبل إعداد هذا التقرير.</p>
<p>(4) الإشراف على إعداد الدراسات الخاصة بتصحيح الهيكل المالي للشركة.</p>
<p>(5) مراجعة الدراسات التي تعد عن المشروعات الاستثمارية اللازمة للإحلال والتجديد والتوسع.</p>
<p>(6) مراجعة مقترحات التطوير في جميع أنشطة الشركة ومتابعة تنفيذ ما يتم إقراره منها.</p>
<p>(7) تحديد اللجان المؤقتة أو الدائمة والتي قد يرى ضرورة تشكيلها لتنفيذ المهام التي يحددها واختيار أعضائها.</p>
<p>(8) التحقق من توافر شروط شغل الوظائف القيادية فيمن تقدموا لشغلها ويعرض ترشيحاته على مجلس الإدارة للنظر فيها وإقرارها.</p>
<p>(9) منح المكافآت الخاصة عن الأعمال المتميزة التي قام بها معاونوه من شاغلي الوظائف المختلفة طبقا للوائح والنظم المعمول بها في الشركة وفي حدود الاعتمادات المالية المخصصة لهذا الغرض.</p>
<p>(10) تمثيل الشركة في صلاتها مع الغير وأمام القضاء.</p>
<p>(11) تنفيذ ومتابعة ما يكلفه به مجلس الإدارة من مهام وأعمال.</p>
<p> </p>
<p> </p>
<p>الفصل الثالث: الجمعية العامة للشركات التابعة</p>
<p>مادة 62</p>
<p>  النص النهائى للمادة بتاريخ :  15/09/2020</p>
<p>تتكون الجمعية العامة للشركة التي تملك الشركة القابضة رأس مالها بأكمله أو تشترك في ملكيته مع شركات قابضة أخرى أو مع أشخاص عامة أو البنوك المملوكة أسهمها بالكامل للدولة على النحو الآتي:</p>
<p>(1) رئيس مجلس إدارة الشركة القابضة أو من يحل محله في حالة غيابه .. رئيسا.</p>
<p>(2) أعضاء مجلس إدارة الشركة القابضة التي تتبعها الشركة.</p>
<p>(3) عضوان تختارهما اللجنة النقابية.</p>
<p>(4) أعضاء من ذوي الخبرة في مجال نشاط الشركة على ألا يزيد عددهم على أربعة وفقا لما يحدده النظام الأساسي.</p>
<p>ويكون اختيار الأعضاء من ذوي الخبرة وممثلي اللجنة النقابية لمدة ثلاث سنوات.</p>
<p>ويجوز تجديد عضويتهم كلهم أو بعضهم في ضوء ما تسفر عنه نتائج أعمال الشركة.</p>
<p>ويحدد قرار الجمعية العامة للشركة القابضة باختيار أعضاء الجمعية العامة من ذوي الخبرة للشركة التابعة ما يتقاضونه من بدل حضور جلسات الجمعية العامة.</p>
<p> </p>
<p>مادة 63</p>
<p>  النص النهائى للمادة بتاريخ :  15/09/2020</p>
<p>تسري في شأن إجراءات ومواعيد وشروط صحة انعقاد الجمعية العامة العادية وغير العادية ونظام التصويت على المسائل المعروضة عليها واختصاصاتها أحكام المواد 22 و23 و24 و25 و27 و28 و29 من هذه اللائحة.</p>
<p>وعلى رئيس الجمعية العامة دعوتها إلى الانعقاد كلما طلب ذلك المساهمون الذين يملكون 10% من رأس المال على الأقل وعلى أن يوضح بالطلب الأسباب الداعية لعقد الاجتماع والمسائل المطلوب عرضها.</p>
<p>ولا يجوز لأعضاء الجمعية من ممثلي الشركات القابضة والأشخاص الاعتبارية العامة و البنوك المملوكة أسهمها بالكامل للدولة أن ينوبوا عن حملة الأسهم من ممثلي القطاع الخاص في حضور الجمعية العامة أو في التصويت، كما لا يجوز لحملة الأسهم من القطاع الخاص أن ينوبوا عن هؤلاء.</p>
<p> </p>
<p>مادة 64</p>
<p>تسري في شأن صحة انعقاد الجمعية العامة العادية وغير العادية للشركات التابعة التي يساهم القطاع الخاص في رأسمالها ونظام التصويت فيها أحكام المادتين 67، 70 من قانون شركات المساهمة وشركات التوصية بالأسهم والشركات ذات المسئولية المحدودة المشار إليه.</p>
<p> </p>
<p>مادة 65</p>
<p>يكون تعديل النظام الأساسي للشركة وفقا للأحكام المنصوص عليها في المادة 25 (أولا) من هذه اللائحة.</p>
<p>وإذا تناول تعديل النظام الأساسي الغرض الأصلي للشركة فلا يكون التعديل نافذا إلا بموافقة الوزير.</p>
<p> </p>
<p>الفصل الرابع: النظام المالي للشركات التابعة ـ توزيع الأرباح ـ الاحتياطيات</p>
<p> </p>
<p>مادة 66</p>
<p>يعد مجلس إدارة الشركة النظم واللوائح والتعليمات الإدارية والمالية التي تكفل انتظام العمل وإعداد وتنفيذ البرامج والسياسات ومتابعة تنفيذها وتوفير ضمانات المراقبة والمراجعة المالية طبقا للأصول والقواعد المحاسبية المقررة وحسب ما تقتضيه طبيعة وحجم نشاط الشركة.</p>
<p> </p>
<p>مادة 67</p>
<p>يجب أن تتضمن اللوائح والنظم والتعليمات الإدارية والمالية للشركة الصلاحيات المناسبة والضمانات الكافية للقائمين بأعمال المراقبة أو المراجعة الداخلية والخارجية وتنظيم إجراءات الجرد الدوري والسنوي والمفاجئ.</p>
<p> </p>
<p>مادة 68</p>
<p>تعرض على مجلس الإدارة شهريا القوائم المالية التالية، والتي تعتمد في إعدادها على تقديرات مستندة إلى أسس وضوابط يحددها النظام المالي:</p>
<p>(1) قائمة نتائج أعمال تبين الإيرادات الفعلية والأنشطة المختلفة والمصروفات المتنوعة والفائض قبل الضرائب.</p>
<p>(2) قائمة مركز مالي.</p>
<p>(3) مقارنات بين القوائم التقديرية السابق إعدادها قبل بداية العام عن نفس الشهر مع تحديد الاختلافات بالزيادة والنقص وأسبابها.</p>
<p>(4) تقرير مبسط من العضو المنتدب يبين التقدم المالي والفني الذي تحقق خلال الشهر المنقضي وتوقعاته للشهر المقبل.</p>
<p>(5) موقف المشروعات الاستثمارية الجاري تنفيذها مبينا به التكاليف التقديرية والفعلية للأعمال التي نفذت وموقف التنفيذ.</p>
<p>(6) التزامات وموارد الشركة من النقد الأجنبي خلال الشهر المنقضي والشهر المقبل. ومصادر تغطية العجز أن وجد وتأثير التغير في أسعار الصرف على نتائج أعمال الشركة، ومقترحات مواجهة هذه الآثار.</p>
<p>(7) موقف السيولة المقارن من خلال قائمة المقبوضات والمدفوعات النقدية الفعلية مقارنة بالأرقام لسابق تقديرها عن نفس الشهر.</p>
<p> </p>
<p>مادة 69</p>
<p>يعرض على مجلس الإدارة في اجتماع خاص يعقد قبل بدء السنة المالية بثلاثة أشهر برنامج العمل التفصيلي للعام التالي موزعا على شهور السنة ومرفقا به القوائم المالية المعبرة عن هذا البرنامج كما يخصص مجلس الإدارة أحد اجتماعاته بعد انتهاء السنة المالية لفحص ومناقشة القوائم المالية التي أعدها الجهاز المالي للشركة وراجعها مراقب الحسابات والآتي بيانها:</p>
<p>1ـ الميزانية العمومية.</p>
<p>2ـ حساب الأرباح والخسائر والحسابات والقوائم الختامية.</p>
<p>3ـ تقرير كتابي عن موقف الشركة خلال السنة.</p>
<p>ويعتمد المجلس البرنامج التفصيلي والقوائم المشار إليها في الفقرة السابقة تمهيدا لعرضها على الجمعية العامة.</p>
<p> </p>
<p>مادة 70</p>
<p>يناقش مجلس الإدارة في ذات الاجتماع المخصص لمناقشة القوائم المالية المشار إليها في المادة السابقة وفي اجتماعات لاحقة إذا لزم الأمر تقرير العضو المنتدب عن الأداء المالي للشركة وتقويمه لنتائج الأعمال والمركز المالي كما تظهره القوائم المالية المشار إليها في المادة السابقة</p>
<p>.</p>
<p>مادة 71</p>
<p>ترسل نسخة من الميزانية وحساب الأرباح والخسائر قبل اعتمادها من مجلس إدارة الشركة وملخص لمناقشات المجلس لها إلى رئيس مجلس إدارة الشركة القابضة لدراستها وإبداء ما يراه من ملاحظات عليها.</p>
<p> </p>
<p>مادة 72</p>
<p>يعتمد مجلس الإدارة الصيغة النهائية لكل من البرنامج التفصيلي للعام التالي والميزانية وحساب الأرباح والخسائر والتقرير المعد عن إنجازات الشركة ومركزها المالي.</p>
<p> </p>
<p>مادة 73</p>
<p>يجب أن تشتمل الميزانية وحساب الأرباح والخسائر على البيانات الواردة بالنظام المحاسبي الموحد الصادر باعتماده قرار رئيس الجمهورية رقم 4723 لسنة 1966.</p>
<p>ويجب أن يتضمن التقرير الذي يعده مجلس الإدارة عن إنجازات الشركة ومركزها المالي البيانات الواردة بالملحق رقم (4) المرفق باللائحة التنفيذية لقانون شركات المساهمة وشركات التوصية بالأسهم والشركات ذات المسئولية المحدودة المشار إليه.</p>
<p> </p>
<p>مادة 74</p>
<p>يجب أن تكون كلا من الميزانية وحساب الأرباح والخسائر وتقرير مجلس الإدارة معدا قبل الموعد المقرر لاجتماع الجمعية العامة للشركة بثلاثة أشهر على الأقل ويتعين وضع هذه الوثائق تحت تصرف مراقبي الحسابات خلال هذه الفترة.</p>
<p> </p>
<p>مادة 75</p>
<p>تسري في شأن الأرباح القابلة للتوزيع في الشركات التابعة وقواعد توزيعها أحكام المادتين (38) و(39) وأولا وثالثا من المادة (40) والمادة (43) من هذه اللائحة.</p>
<p> </p>
<p>مادة 76</p>
<p>في حالة وجود حصص تأسيس أو حصص أرباح لا يجوز أن يخصص لها ما يزيد على 10% من الأرباح القابلة للتوزيع وذلك بعد تخصيص نسبة من الربح لا تقل عن 5% من رأس المال للمساهمين والعاملين كحصة أولى، وخصم مكافأة مجلس الإدارة.</p>
<p> </p>
<p>الباب الثالث: أحكام عامة</p>
<p> الفصل الأول: مراقبة حسابات الشركات القابضة والشركات التابعة وتقويم أدائها</p>
<p>مادة 77</p>
<p>يباشر الجهاز المركزي للمحاسبات اختصاصه بشأن الرقابة على الشركات القابضة والشركات التابعة لها وتقويم أدائها وفقا لأحكام قانونه.</p>
<p>وتتضمن هذه الرقابة على الأخص مراجعة الحسابات الختامية والمراكز المالية والميزانيات للوقوف على مدى صحتها وتمثيلها لحقيقة النشاط وذلك وفقا للمبادئ المحاسبية المتعارف عليها.</p>
<p>ولمراقب حسابات الجهاز أن يبدي ملاحظاته بشأن الأخطاء والمخالفات والقصور في تطبيق القوانين واللوائح والقرارات والتثبت من صحة تطبيق النظام المحاسبي الموحد وسلامة الدفاتر والتأكد من سلامة إثبات توجيه العمليات المختلفة بالدفاتر بما يتفق مع الأصول المحاسبية لتحقيق النتائج المالية السليمة.</p>
<p> </p>
<p> ا الفصل الثاني: الرقابة على الشركات وحقوق الاطلاع على السجلات والدفاتر</p>
<p>مادة 78</p>
<p>يقدم الوزير إلى مجلس الوزراء كل ستة أشهر تقريرا عن نتائج أعمال الشركات الخاضعة للقانون.</p>
<p>وللوزير من خلال خبراء يعينهم حق الإطلاع على سجلات الشركات المشار إليها وطلب كافة البيانات التي تتطلبها طبيعة عملهم للتحقق من تنفيذ أحكام القانون ولائحته التنفيذية.</p>
<p> </p>
<p>مادة 79</p>
<p>للشركة القابضة الإطلاع على سجلات الشركة التابعة وطلب بيانات تفصيلية عن ميزانيتها وحسابات أرباحها وخسائرها وتقرير مراقبي الحسابات عن الثلاث سنوات السابقة وكافة الأوراق والمستندات الأخرى.</p>
<p>ويتم الإطلاع من خلال ممثلي الشركة القابضة في مجلس إدارة الشركة التابعة وفي مقر هذه الشركة.</p>
<p>ويجوز لهم اصطحاب خبراء والحصول على مستخرجات من الأوراق موضوع الإطلاع.</p>
<p> </p>
<p>مادة 80</p>
<p>يجوز لباقي المساهمين الإطلاع على سجلات الشركة التابعة فيما عدا الدفتر الذي تدون فيه محاضر مجلس الإدارة والدفاتر المحاسبية للشركة. كما يجوز لهم الإطلاع على ميزانيات الشركة وحسابات أرباحها وخسائرها وتقارير مراقبي الحسابات وذلك عن الثلاث سنوات المالية السابقة على السنة التي يتم فيها الإطلاع.</p>
<p>ولهؤلاء المساهمين الإطلاع على جميع الأوراق والمستندات الأخرى التي لا يكون في إذاعة ما ورد بها من بيانات أضرار بمركز الشركة أو بالغير.</p>
<p>ويتم إطلاع المساهمين بأنفسهم، كما يجوز له اصطحاب خبراء، على أن يتم الإطلاع بمقر الشركة في المواعيد التي تحددها بشرط ألا تقل عن يوم في كل أسبوع.</p>
<p>ويجوز لهؤلاء المساهمين الحصول على مستخرجات من الأوراق موضوع الإطلاع مقابل أداء (عشرة قروش) على الأقل عن الصفحة الواحدة.</p>
<p> </p>
<p>الفصل الثالث: إدماج وتقسيم الشركات القابضة والشركات التابعة لها</p>
<p>مادة 81</p>
<p>يجوز إدماج أو تقسيم الشركات القابضة أو التابعة لتحقيق واحد أو أكثر من الأغراض الآتية:</p>
<p>(1) تحقيق التكامل بين الأنشطة التي تقوم بها الشركة.</p>
<p>(2) تحقيق قدر أكبر من المنافسة بين الشركات.</p>
<p>(3) دعم المركز المالي للشركات المندمجة أو المقسمة.</p>
<p>(4) الاستفادة من الطاقات العاطلة في بعض الشركات.</p>
<p>(5) الاستفادة من كفاءة وخبرة الإدارة الموجودة في بعض المواقع.</p>
<p>(6) زيادة قدرة الشركة على الحصول على الائتمان والتسهيلات من المؤسسات المالية الداخلية والخارجية.</p>
<p>(7) تجميع الوحدات المتقاربة جغرافيا تحت إشراف واحد لتوفير إشراف أكثر فاعلية.</p>
<p>(8) زيادة ربحية الشركات المندمجة أو الشركات المقسمة.</p>
<p>(9) أية أغراض أخرى تساهم في دعم أنشطة الشركات وزيادة فرص النجاح لها.</p>
<p> </p>
<p>مادة 82</p>
<p>يسري في شأن إدماج الشركات القابضة والشركات التابعة لها أحكام المواد من 289 إلى 298 من اللائحة التنفيذية لقانون شركات المساهمة وشركات التوصية بالأسهم والشركات ذات المسئولية المحدودة.</p>
<p> </p>
<p>مادة 83</p>
<p>يجوز تقسيم الشركة القابضة أو الشركة التابعة إلى شركتين أو أكثر.</p>
<p>ويحدد النظام الأساسي للشركة الإجراءات والأوضاع التي تتبع في تقسيمها.</p>
<p> </p>
<p> الفصل الرابع: أوضاع وإجراءات إنهاء خدمة العاملين بالشركات الخاضعة للقانون بسبب الاستقالة أو عدم اللياقة للخدمة صحيا</p>
<p>مادة 84</p>
<p>للعامل أن يقدم استقالته من وظيفته وتكون الاستقالة مكتوبة، ولا تنتهي خدمة العامل إلا بالقرار الصادر بقبول الاستقالة ويجب البت في الطلب خلال ثلاثين يوما من تاريخ تقديمه وإلا اعتبرت الاستقالة مقبولة بحكم القانون ما لم يكن طلب الاستقالة معلقا على شرط أو مقترنا بقيد، وفي هذه الحالة لا تنتهي خدمة العامل إلا إذا تضمن قرار قبول الاستقالة إجابته إلى طلبه ويجوز خلال هذه المدة إرجاء قبول الاستقالة لأسباب تتعلق بمصلحة العمل مع إخطار العامل بذلك على ألا تزيد مدة الإرجاء على أسبوعين بالإضافة إلى مدة الثلاثين يوما السابق الإشارة إليها.</p>
<p> </p>
<p>مادة 85</p>
<p>يعتبر العامل مقدما استقالته في الحالتين الآتيتين:</p>
<p>(1) إذا انقطع عن عمله بغير إذن أكثر من عشرة أيام متتالية ما لم يقدم خلال الخمسة عشر يوما التالية ما يثبت أن انقطاعه بعذر يقبله رئيس مجلس الإدارة أو العضو المنتدب أو من يفوضه أي منهما حسب الأحوال، ويجوز لمن قبل العذر أن يقرر عدم حرمان العامل من أجره عن مدة الانقطاع إذا كان له رصيد من الإجازات يسمح بذلك وإلا وجب حرمانه من أجره عن هذه المدة. فإذا لم يقدم العامل أسبابا تبرر الانقطاع أو قدم هذه الأسباب ورفضت اعتبرت خدمته منتهية من تاريخ انقطاعه عن العمل.</p>
<p>(2) إذا انقطع عن عمله دون عذر يقبله رئيس مجلس الإدارة أو العضو المنتدب أو من يفوضه أي منهما حسب الأحوال أكثر من عشرين يوما غير متصلة في السنة، وتعتبر خدمته منتهية في هذه الحالة من اليوم التالي لاكتمال هذه المدة.</p>
<p>ويتعين إنذار العامل كتابة بعد انقطاعه لمدة سبعة أيام في الحالة الأولى وخمسة عشر يوما في الحالة الثانية.</p>
<p> </p>
<p>مادة 86</p>
<p>تثبت عدم اللياقة للخدمة صحيا بقرار من الجهة الطبية المختصة سواء كان ذلك راجعا إلى وجود عجز كلي عن أداء العمل الأصلي أو عجز جزئي مستديم متى ثبت عدم وجود أي عمل آخر يمكنه القيام به طبقا لأحكام قانون التأمين الاجتماعي والقرارات الصادرة تنفيذا له.</p>
<p>وفي جميع الأحوال لا يجوز فصل العامل لعدم اللياقة للخدمة صحيا قبل نفاذ إجازاته المرضية والاعتيادية ما لم يطلب هو نفسه إنهاء خدمته دون انتظار انتهاء إجازاته.</p>
<p> </p>
<p> </p>
<p>مادة 87</p>
<p>يصرف للعامل أجره إلى اليوم الذي تنتهي فيه خدمته لأحد الأسباب المبينة بالمادة 45 من القانون، على أنه في حالة الفصل لعدم اللياقة الصحية يستحق العامل الأجر كاملا أو منقوصا حسب الأحوال حتى يستنفد إجازاته المرضية والاعتيادية أو إحالته إلى المعاش بناء على طلبه وذلك وفق ما يقرره قانون التأمين الاجتماعي أو لائحة نظام العاملين بالشركة أيهما أفضل للعامل.</p>
<p>وإذا كان انتهاء الخدمة بسبب استقالة العامل استحق أجره حتى تاريخ صدور قرار قبول الاستقالة أو انقضاء المدة التي تعتبر الاستقالة بعدها مقبولة.</p>
<p> </p>"""
    clean_text = remove_html_tags(html_paragraph)
    print(clean_text)


(المادة الأولى) يعمل في شأن قطاع الأعمال العام بأحكام القانون المرافق، ويقصد بهذا القطاع الشركات القابضة والشركات التابعة لها الخاضعة لأحكام هذا القانون، وتتخذ هذه الشركات بنوعيها شكل شركات المساهمة، ويسري عليها فيما لم يرد بشأنه نص خاص في هذا القانون وبما لا يتعارض مع أحكامه نصوص قانون شركات المساهمة وشركات التوصية بالأسهم والشركات ذات المسئولية المحدودة الصادر بالقانون رقم 159 لسنة 1981. لا تسري أحكام قانون هيئات القطاع العام وشركاته الصادر بالقانون رقم 97 لسنة 1983 على الشركات المشار إليها. (المادة الثانية) تحل الشركات القابضة محل هيئات القطاع العام الخاضعة لأحكام القانون رقم 97 لسنة 1983 المشار إليه، كما تحل الشركات التابعة محل الشركات التي تشرف عليها هذه الهيئات وذلك اعتبارا من تاريخ العمل بهذا القانون ودون حاجة إلى أي إجراء آخر. وتنتقل إلى الشركات القابضة والشركات التابعة لها بحسب الأحوال كافة ما لهيئات القطاع العام وشركاته الملغاة من حقوق بما فيها حقوق الانتفاع والإيجار، كما تتحمل جميع التزاماتها وتسأل مسئولية كاملة عنها. وينشر النظام الأساسي لكل شركة من الشركات القابضة والتابعة

In [ ]:
# (take this)
clean_text

'المادة 1 يعمل في شأن نظام التأمين الصحي الشامل بأحكام القانون المرافق، وتسري أحكامه إلزاميا على جميع المواطنين المقيمين داخل جمهورية مصر العربية، واختياريا على المصريين العاملين بالخارج وكذلك المقيمين مع أسرهم بالخارج. وتسري قواعد التأمين الصحي والرعاية الطبية المقررة بالقوات المسلحة على أفرادها بالخدمة أو بالمعاش وأسرهم المقرر علاجهم على نفقتها. المادة 2 فيما عدا خدمات الصحة العامة، والخدمات الوقائية، والخدمات الإسعافية، وخدمات تنظيم الأسرة، والخدمات الصحية الخاصة بتغطية الكوارث بكافة أنواعها، والأوبئة، وما يماثلها من خدمات تلتزم بتقديمها سائر أجهزة الدولة مجانا، تطبق أحكام القانون المرافق على الخدمات الصحية التأمينية وما ينتج عن إصابات العمل وذلك كله وفقا للتعريفات الواردة فيه. المادة 3 تسري أحكام القانون المرافق تدريجيا على المحافظات استرشادا بالمراحل المبينة بالجدول رقم (5) المرافق، وبما يضمن استدامة الملاءة المالية للنظام وبمراعاة توازنه الاكتواري. ويستمر انتفاع المؤمن عليهم بخدمات التأمين الصحي ومرافقه وفقا للقوانين واللوائح والقرارات المعمول بها حاليا، حتى تاريخ سريان أحكام الق

In [ ]:
# Specify the URL directly in the code
URL = "https://manshurat.org/node/63712"  

def fetch_page_content(url):
    """Fetch the page content for the given URL."""
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.content, 'html.parser')
    return None

def parse_normal_page(soup):
    """Parse the content of a normal page to extract articles and laws."""
    articles = []
    
    # Locate the div containing the articles
    content_div = soup.select_one(".field.field-name-body .field-items")
    if not content_div:
        print("No content div found. Check the structure of the page.")
        return articles

    # Find all <p> tags inside the content div
    paragraphs = content_div.find_all("p")
    print(f"Found {len(paragraphs)} <p> tags.")  # Debugging: Log the number of <p> tags found

    current_article = None
    current_law = []

    for i, paragraph in enumerate(paragraphs):
        text = paragraph.get_text(strip=True)  # Extract text
        print(f"Processing Paragraph {i+1}: {text}")  # Debugging: Log the text of each paragraph

        # Handle multiple lines inside a single paragraph due to <br> tags
        parts = paragraph.decode_contents().split("<br/>")
        for part in parts:
            part_text = BeautifulSoup(part, "html.parser").get_text(strip=True)

            # Detect new article
            if part_text.startswith("المادة"):
                if current_article and current_law:
                    # Store the previous article before processing the new one
                    articles.append({"Article": current_article, "Law": " ".join(current_law)})
                
                # Start a new article
                split_text = part_text.split("المادة", 1)
                if len(split_text) > 1:
                    article_number = split_text[1].strip().split(" ")[0]  # Extract article number
                    current_article = f"المادة {article_number}"
                    current_law = [split_text[1].strip()[len(article_number):].strip()]
                else:
                    current_article = None
                    current_law = []
            elif current_article:
                # Add additional text to the current law
                current_law.append(part_text)

    # Save the last article if there is one
    if current_article and current_law:
        articles.append({"Article": current_article, "Law": " ".join(current_law)})

    return articles

def save_csv(filename, rows, fieldnames):
    """Save the extracted data into a CSV file."""
    # Overwrite the file if it exists
    if os.path.exists(filename):
        os.remove(filename)
        print(f"Overwriting existing file: {filename}")
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

def main():
    print(f"Processing URL: {URL}")
    soup = fetch_page_content(URL)
    if not soup:
        print(f"Failed to fetch the page. Please check the URL and try again.")
        return

    # Extract the page title
    page_title_element = soup.find("h2")
    if not page_title_element:
        print(f"The page has no title. Skipping...")
        return

    page_title = page_title_element.text.strip()

    # Skip pages with "الصفحة غير موجودة"
    if "الصفحة غير موجودة" in page_title:
        print(f"The page does not exist. Skipping...")
        return

    # Process normal pages
    articles = parse_normal_page(soup)
    if articles:
        # Sanitize the title and create a unique filename
        sanitized_title = page_title.replace("/", "-").replace("\\", "-").replace(":", "-")
        csv_filename = f"{sanitized_title}.csv"

        # Save the extracted data into a CSV file
        save_csv(csv_filename, articles, ["Article", "Law"])
        print(f"Saved articles for the page in {csv_filename}")
    else:
        print(f"No articles found on the page.")

if __name__ == "__main__":
    main()


Processing URL: https://manshurat.org/node/63712
Overwriting existing file: قانون التأمين الصحي الشامل رقم 2 لسنة 2018.csv
Saved articles for the page in قانون التأمين الصحي الشامل رقم 2 لسنة 2018.csv


In [ ]:
# Specify the URL directly in the code
URL = "https://manshurat.org/node/14675"

def fetch_page_content(url):
    """Fetch the page content for the given URL."""
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.content, 'html.parser')
    return None

def parse_normal_page(soup):
    """Parse the content of a normal page to extract articles and laws."""
    articles = []
    
    # Locate the div containing the articles
    content_div = soup.select_one(".field.field-name-body .field-items")
    if not content_div:
        print("No content div found. Check the structure of the page.")
        return articles

    # Find all <p> tags inside the content div
    paragraphs = content_div.find_all("p")
    print(f"Found {len(paragraphs)} <p> tags.")  # Debugging: Log the number of <p> tags found

    current_article = None
    current_law = []

    for i, paragraph in enumerate(paragraphs):
        text = paragraph.get_text(strip=True)  # Extract text
        print(f"Processing Paragraph {i+1}: {text}")  # Debugging: Log the text of each paragraph

        # Handle multiple lines inside a single paragraph due to <br> tags
        parts = paragraph.decode_contents().split("<br/>")
        for part in parts:
            part_text = BeautifulSoup(part, "html.parser").get_text(strip=True)

            # Detect new article using regex (to capture only the number)
            match = re.match(r"المادة\s+(\d+)", part_text)
            if match:
                if current_article and current_law:
                    # Store the previous article before processing the new one
                    articles.append({"Article": current_article, "Law": " ".join(current_law)})
                
                # Start a new article with only the number
                article_number = match.group(1)
                current_article = f"المادة {article_number}"
                
                # Extract law text without the article number
                law_start_pos = part_text.find(article_number) + len(article_number)
                current_law = [part_text[law_start_pos:].strip()]
            elif current_article:
                # Add additional text to the current law
                current_law.append(part_text)

    # Save the last article if there is one
    if current_article and current_law:
        articles.append({"Article": current_article, "Law": " ".join(current_law)})

    return articles

def save_csv(filename, rows, fieldnames):
    """Save the extracted data into a CSV file."""
    # Overwrite the file if it exists
    if os.path.exists(filename):
        os.remove(filename)
        print(f"Overwriting existing file: {filename}")
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

def main():
    print(f"Processing URL: {URL}")
    soup = fetch_page_content(URL)
    if not soup:
        print(f"Failed to fetch the page. Please check the URL and try again.")
        return

    # Extract the page title
    page_title_element = soup.find("h2")
    if not page_title_element:
        print(f"The page has no title. Skipping...")
        return

    page_title = page_title_element.text.strip()

    # Skip pages with "الصفحة غير موجودة"
    if "الصفحة غير موجودة" in page_title:
        print(f"The page does not exist. Skipping...")
        return

    # Process normal pages
    articles = parse_normal_page(soup)
    if articles:
        # Sanitize the title and create a unique filename
        sanitized_title = page_title.replace("/", "-").replace("\\", "-").replace(":", "-")
        csv_filename = f"{sanitized_title}.csv"

        # Save the extracted data into a CSV file
        save_csv(csv_filename, articles, ["Article", "Law"])
        print(f"Saved articles for the page in {csv_filename}")
    else:
        print(f"No articles found on the page.")

if __name__ == "__main__":
    main()


Processing URL: https://manshurat.org/node/14675
Found 310 <p> tags.
Processing Paragraph 1: 
Processing Paragraph 2: بسم الله الرحمن الرحيم
Processing Paragraph 3: هذا دستورنا مصر هبة النيل للمصريين، وهبة المصريين للإنسانية.
Processing Paragraph 4: مصر العربية - بعبقرية موقعها وتاريخها - قلب العالم كله، فهي ملتقى حضاراته وثقافاته،
Processing Paragraph 5: ومفترق طرق مواصلاته البحرية واتصالاته، وهي رأس أفريقيا المطل على المتوسط، ومصب أعظم أنهارها: النيل. هذه مصر، وطن خالد للمصريين، ورسالة سلام ومحبة لكل الشعوب.
Processing Paragraph 6: في مطلع التاريخ، لاح فجر الضمير الإنساني وتجلى في قلوب أجدادنا العظام فاتحدت إرادتهم الخيرة، وأسسوا أول دولة مركزية، ضبطت ونظمت حياة المصريين على ضفاف النيل، وأبدعوا أروع آيات الحضارة، وتطلعت قلوبهم إلى السماء قبل أن تعرف الأرض الأديان السماوية الثلاثة.
Processing Paragraph 7: مصر مهد الدين، وراية مجد الأديان السماوية.
Processing Paragraph 8: في أرضها شب كليم الله موسى عليه السلام، وتجلى له النور الإلهي، وتنزلت عليه الرسالة في طور سنين.وعلى أرضها احتضن الم

In [ ]:
# Specify the URL directly in the code
URL = "https://manshurat.org/node/14677"  
def fetch_page_content(url):
    """Fetch the page content for the given URL."""
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.content, 'html.parser')
    return None

def parse_normal_page(soup):
    """Parse the content of a normal page to extract articles and laws."""
    articles = []
    
    # Locate the div containing the articles
    content_div = soup.select_one(".field.field-name-body .field-items")
    if not content_div:
        print("No content div found. Check the structure of the page.")
        return articles

    # Find all <p> tags inside the content div
    paragraphs = content_div.find_all("p")
    print(f"Found {len(paragraphs)} <p> tags.")  # Debugging: Log the number of <p> tags found

    current_article = None
    current_law = []

    for i, paragraph in enumerate(paragraphs):
        text = paragraph.get_text(strip=True)  # Extract text
        print(f"Processing Paragraph {i+1}: {text}")  # Debugging: Log the text of each paragraph

        # Handle multiple lines inside a single paragraph due to <br> tags
        parts = paragraph.decode_contents().split("<br/>")
        for part in parts:
            part_text = BeautifulSoup(part, "html.parser").get_text(strip=True)

            # Detect new article using regex (to capture only the number)
            match = re.match(r"المادة\s+(\d+)", part_text)
            if match:
                if current_article and current_law:
                    # Store the previous article before processing the new one
                    articles.append({"Article": current_article, "Law": " ".join(current_law)})

                # Extract only the article number
                article_number = match.group(1)
                current_article = f"المادة {article_number}"
                
                # Extract the law text without the article number
                law_start_pos = part_text.find(article_number) + len(article_number)
                law_text = part_text[law_start_pos:].strip()
                
                # Ensure the extracted law text does not start with extra words
                if law_text.startswith("–") or law_text.startswith("-") or law_text.startswith(":"):
                    law_text = law_text[1:].strip()

                current_law = [law_text] if law_text else []
            elif current_article:
                # Add additional text to the current law
                current_law.append(part_text)

    # Save the last article if there is one
    if current_article and current_law:
        articles.append({"Article": current_article, "Law": " ".join(current_law)})

    return articles

def save_csv(filename, rows, fieldnames):
    """Save the extracted data into a CSV file."""
    # Overwrite the file if it exists
    if os.path.exists(filename):
        os.remove(filename)
        print(f"Overwriting existing file: {filename}")
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

def main():
    print(f"Processing URL: {URL}")
    soup = fetch_page_content(URL)
    if not soup:
        print(f"Failed to fetch the page. Please check the URL and try again.")
        return

    # Extract the page title
    page_title_element = soup.find("h2")
    if not page_title_element:
        print(f"The page has no title. Skipping...")
        return

    page_title = page_title_element.text.strip()

    # Skip pages with "الصفحة غير موجودة"
    if "الصفحة غير موجودة" in page_title:
        print(f"The page does not exist. Skipping...")
        return

    # Process normal pages
    articles = parse_normal_page(soup)
    if articles:
        # Sanitize the title and create a unique filename
        sanitized_title = page_title.replace("/", "-").replace("\\", "-").replace(":", "-")
        csv_filename = f"{sanitized_title}.csv"

        # Save the extracted data into a CSV file
        save_csv(csv_filename, articles, ["Article", "Law"])
        print(f"Saved articles for the page in {csv_filename}")
    else:
        print(f"No articles found on the page.")

if __name__ == "__main__":
    main()


Processing URL: https://manshurat.org/node/14677
Found 557 <p> tags.
Processing Paragraph 1: **آخر تعديل: 15 أغسطس 2021 بالقانون 141 لسنة 2021 **
Processing Paragraph 2: القانون رقم 58 لسنة 1937إصدار قانون العقوباتنحن فاروق الأول ملك مصرقرر مجلس الشيوخ ومجلس النواب القانون الآتي نصه وقد صدقنا عليه وأصدرناه:
Processing Paragraph 3: المادة 1يلغى قانون العقوبات الجاري العمل به أمام المحاكم الأهلية وقانون العقوبات الذي تطبقه المحاكم المختلطة ويستعاض عنهما بقانون العقوبات المرافق لهذا القانون.
Processing Paragraph 4: المادة 2على وزير الحقانية تنفيذ هذا القانون ويعمل به من 15 أكتوبر سنة 1937.نأمر بأن يبصم هذا القانون بخاتم الدولة وأن ينشر في الجريدة الرسمية وينفذ كقانون من قوانين الدولة.
Processing Paragraph 5: صدر بسراي عابدين في 23 جمادي الأول ( 21 يوليو سنة 1937 )فاروقبأمر حضرة صاحب الجلالةرئيس الوزراءمصطفي النحاس
Processing Paragraph 6: قانون العقوباتالكتاب الأول أحكام ابتدائيةالباب الأول – قواعد عمومية
Processing Paragraph 7: المادة 1تسري أحكام هذا القانون على كل من يرتكب في القطر المصر